<div style="text-align: center;">
<span style="color:#C1D3FE;font-size: 32px; font-weight: bold;">Notebook de Análise: "Originadora" de 05/08</span>
<br> <span style="color:#FFD1DC;"> Feito por Lucas Andrade entre 07 e 11 de Agosto de 2025 </span>
<br> <span style="color:#FFF5BA;"> revisado por Felipe Bastos


<span style="color:#C1D3FE;">Porto Real Asset</span>

## <span style="color:#AEE5F9;">Análise de Carteira de Originadores

Este notebook realiza uma análise completa da carteira de recebíveis, consolidando dados de diferentes fontes, realizando verificações de qualidade, calculando métricas de risco e, por fim, gerando um relatório HTML interativo. Feito por Lucas Andrade, com auxílio de Felipe Bastos

### <span style="color:#AEE5F9;"> Bibliotecas e Configurações Iniciais

In [2]:
# =============================================================================
# Bibliotecas   ===============================================================
# =============================================================================
import pandas as pd
import numpy as np
import os
from scipy.optimize import brentq
import base64
import os

from IPython.display import display

pd.options.display.max_columns = 100
pd.options.display.max_rows = 200

##### <span style="color:#CFFFE5;">NOVIDADE versão 1.02</span>

In [3]:
DIAS_ATRASO_DEFINICAO_VENCIDO = 30

### <span style="color:#AEE5F9;">  Leitura e Preparação dos Dados
<span style="color: #FFB3B3; font-size: 15px; font-weight: bold;">
  ATENÇÃO: REDIFINIR AQUI OS PATHS
</span>

Defino os caminhos dos arquivos de entrada, carrego os dados, uno as duas fontes (`StarCard.xlsx` e `Originadores.xlsx`) usando a coluna `CCB` como chave e realizamos uma limpeza inicial, tratando colunas monetárias e de data.

In [4]:
# =============================================================================
# LER DADOS   =================================================================
# =============================================================================

#! PATHS ----------------------------------------------------------------------
#! ----------------------------------------------------------------------------
# Dados IN : 
path_starcard = r'C:\Users\Leo\Desktop\Porto_Real\portoauto\src\originadores\data\StarCard.xlsx'
path_originadores = r'C:\Users\Leo\Desktop\Porto_Real\portoauto\src\originadores\data\Originadores.xlsx'
caminho_feriados = r'C:\Users\Leo\Desktop\Porto_Real\portoauto\feriados_nacionais.xls'

logo_path = r'C:\Users\Leo\Desktop\Porto_Real\portoauto\images\logo_inv.png'

#SAÍDA LOCAL:
output_path = r'C:\Users\Leo\Desktop\Porto_Real\portoauto\src\originadores\output'
#! ----------------------------------------------------------------------------

print(f"Lendo arquivo principal: {path_starcard}")
df_starcard = pd.read_excel(path_starcard)

print(f"Lendo arquivo de detalhe: {path_originadores}")

# ?<obs> essas foram deduções próprias, q dei minha própria interpretação
cols_originadores = [
    'CCB', 'Prazo', 'Valor Parcela', 'Valor IOF', 'Valor Liquido Cliente',
    'Data Primeiro Vencimento', 'Data Último Vencimento', 'Data de Inclusão',
    'CET Mensal', 'Taxa CCB', 'Produto', 'Tabela', 'Promotora',
    'Valor Split Originador', 'Valor Split FIDC', 'Valor Split Compra de Divida',
    'Taxa Originador Split', 'Taxa Split FIDC'
]
df_originadores = pd.read_excel(path_originadores, usecols=cols_originadores)

# aproveitei que o CCB parece repetir em ambos os lados, pra dar um LEFT JOIN 
#!---------------------------------------------
#TODO :: verificar se essa leitura está correta
# fiz essa verificação que deveria ser sufieciente ao meu entendimento
#!---------------------------------------------
print("Unindo as duas fontes de dados...")
print("Verificando a unicidade da chave 'CCB' em df_originadores...")
if not df_originadores['CCB'].is_unique:
    print("[WARNING] A coluna 'CCB' não é única em Originadores.xlsx. Isso causa duplicação de linhas!")
    #** mostrar os duplicados pra investigr
    duplicados = df_originadores[df_originadores.duplicated(subset='CCB', keep=False)]
    print("CCBs duplicados :")
    display(duplicados.sort_values('CCB'))
else:
    print("'CCB' é uma chave única. A junção tá segura.")



df_merged = pd.merge(df_starcard, df_originadores, on='CCB', how='left', suffixes=('', '_orig'))

print("Iniciando limpeza e preparação dos dados...")

# Aqui eu renomeei para funcionar no script anterior
df_merged = df_merged.rename(columns={
    'Data Referencia': 'DataGeracao',
    'Data Aquisicao': 'DataAquisicao',
    'Data Vencimento': 'DataVencimento',
    'Status': 'Situacao',
    'PDD Total': 'PDDTotal',
    'Valor Nominal': 'ValorNominal',
    'Valor Presente': 'ValorPresente',
    'Valor Aquisicao': 'ValorAquisicao',
    'ID Cliente': 'SacadoID', # obs: nao especifica o doc
    'Pagamento Parcial': 'PagamentoParcial'
})

# remove 'R$ ' -->>> vira float #*(note que os valores vem assim em StarCard)
cols_monetarias = ['ValorAquisicao', 'ValorNominal', 'ValorPresente', 'PDDTotal']
for col in cols_monetarias:
    if df_merged[col].dtype == 'object':
        df_merged[col] = df_merged[col].astype(str).str.replace('R$', '', regex=False).str.replace('.', '', regex=False).str.replace(',', '.', regex=False).str.strip()
        df_merged[col] = pd.to_numeric(df_merged[col], errors='coerce')

# cols de data
cols_data = ['DataGeracao', 'DataAquisicao', 'DataVencimento', 'Data de Nascimento']
for col in cols_data:
    df_merged[col] = pd.to_datetime(df_merged[col], errors='coerce')

# ? df_final2 Criado AQUI <<<<
df_final2 = df_merged.copy()
# Libera memória
del df_starcard, df_originadores, df_merged

print("Leitura, junção e limpeza concluídas.")
print(f"DataFrame final com {df_final2.shape[0]} linhas e {df_final2.shape[1]} colunas.")

Lendo arquivo principal: C:\Users\Leo\Desktop\Porto_Real\portoauto\src\originadores\data\StarCard.xlsx
Lendo arquivo de detalhe: C:\Users\Leo\Desktop\Porto_Real\portoauto\src\originadores\data\Originadores.xlsx
Unindo as duas fontes de dados...
Verificando a unicidade da chave 'CCB' em df_originadores...
'CCB' é uma chave única. A junção tá segura.
Iniciando limpeza e preparação dos dados...
Leitura, junção e limpeza concluídas.
DataFrame final com 248413 linhas e 39 colunas.


### <span style="color:#AEE5F9;">  Colunas Auxiliares

Crio aqui colunas derivadas que capturam informações importantes, como:
- **_ValorLiquido**: Valor Presente descontado do PDD.
- **_ValorVencido**: Valor Presente de parcelas já vencidas.
- **_MuitosContratos**: Flag para clientes com 3 ou mais contratos.
- **_MuitosEntes**: Flag para clientes com contratos em 3 ou mais convênios.
- **_IdadeCliente**: Idade do cliente calculada na data de geração do relatório.

CHANGELOG: <span style="color:#CFFFE5;">ATUALIZADO na versão 1.02</span>

In [5]:
# =============================================================================
#  Colunas Auxiliares =======================================================
# =============================================================================

df_final2['_ValorLiquido'] = df_final2['ValorPresente'] - df_final2['PDDTotal']
df_final2['_ValorVencido'] = (df_final2['DataVencimento'] <= df_final2['DataGeracao']).astype('int') * df_final2['ValorPresente']

#* [NOVIDADE] ..............................................................................
# ID PARCELAS individuais que estão vencidas conforme a nossa regra.
print(f"Passo 1: Identificando parcelas com {DIAS_ATRASO_DEFINICAO_VENCIDO} ou mais dias de atraso.")
dias_de_atraso = (df_final2['DataGeracao'] - df_final2['DataVencimento']).dt.days
df_final2['_ParcelaVencida_Flag'] = ((dias_de_atraso >= DIAS_ATRASO_DEFINICAO_VENCIDO)).astype(int)
# IDtodos os CCBs  que contêm PELO MENOS UMA parcela vencida.
print("Passo 2: Identificando todos os contratos que possuem ao menos uma parcela vencida.")
contratos_com_parcela_vencida = df_final2.groupby('CCB')['_ParcelaVencida_Flag'].max()
# Filtramos para obter uma lista apenas dos CCBs que de fato estão "contaminados".
lista_ccbs_vencidos = contratos_com_parcela_vencida[contratos_com_parcela_vencida == 1].index
# flag final a nível de CONTRATO.
print("Passo 3: Marcando todas as parcelas de um contrato vencido com a flag final.")
df_final2['_ContratoVencido_Flag'] = df_final2['CCB'].isin(lista_ccbs_vencidos).astype(int)
#*..........................................................................................


# uso 'SacadoID', já que nao tem cpf
sacado_contratos = df_final2.groupby('SacadoID')['CCB'].nunique() # vou usar SacadoID pra achar os sacados com muitos contratos
k = 3
mask_contratos = sacado_contratos >= k
sacado_contratos_alto = sacado_contratos[mask_contratos].index
df_final2['_MuitosContratos'] = df_final2['SacadoID'].isin(sacado_contratos_alto).astype(str)

sacados_entes = df_final2.groupby('SacadoID')['Convênio'].nunique() # muitos entes com sacadoid dnv
k2 = 3
mask_entes = sacados_entes >= k2
sacados_entes_alto = sacados_entes[mask_entes].index
df_final2['_MuitosEntes'] = df_final2['SacadoID'].isin(sacados_entes_alto).astype(str)

#* NOVIDADE: idade do cliente
if 'Data de Nascimento' in df_final2.columns and 'DataGeracao' in df_final2.columns:
    df_final2['_IdadeCliente'] = ((df_final2['DataGeracao'] - df_final2['Data de Nascimento']).dt.days / 365.25).astype(int)
    print("Coluna '_IdadeCliente' criada.")

print("Criação de colunas auxiliares concluída.")

Passo 1: Identificando parcelas com 30 ou mais dias de atraso.
Passo 2: Identificando todos os contratos que possuem ao menos uma parcela vencida.
Passo 3: Marcando todas as parcelas de um contrato vencido com a flag final.
Coluna '_IdadeCliente' criada.
Criação de colunas auxiliares concluída.


### <span style="color:#AEE5F9;"> Qualidade e Consistência dos Dados

Verifico inconsistências, dados faltantes e características gerais da carteira, armazenando os resultados para exibição no relatório final.

In [6]:
# =============================================================================
# Qualidade e Consistência dos Dados ==========================================
# =============================================================================
# inconsistencias, dados faltantes e caracters gerais da carteira.
#* resultados armazenados

print("\n" + "="*80)
print("INICIANDO VERIFICAÇÕES DE SANIDADE E QUALIDADE DOS DADOS")
print("="*80)

#
checks_results = {}
"""dic para armznr os resltd das verfcc"""

#* char temprario 'x' para fazr a troca de seprdr.
valor_presente_formatado = f"R$ {df_final2['ValorPresente'].sum():,.2f}".replace(',', 'X').replace('.', ',').replace('X', '.')
total_registros_formatado = f"{len(df_final2):,}".replace(',', '.')
clientes_unicos_formatado = f"{df_final2['SacadoID'].nunique():,}".replace(',', '.')
ccbs_unicos_formatado = f"{df_final2['CCB'].nunique():,}".replace(',', '.')

checks_results['Número Total de Registros'] = total_registros_formatado
checks_results['Valor Presente Total da Carteira'] = valor_presente_formatado
checks_results['Período da Carteira (Data de Aquisição)'] = f"{df_final2['DataAquisicao'].min().strftime('%d/%m/%Y')} a {df_final2['DataAquisicao'].max().strftime('%d/%m/%Y')}"
checks_results['Número de Clientes Únicos'] = clientes_unicos_formatado
checks_results['Número de CCBs Únicos'] = ccbs_unicos_formatado
checks_results['Duplicidade de CCBs'] = f"{df_final2.duplicated(subset='CCB').sum()} registros"

#* Verif Valores
checks_results['Valores Monetários Negativos'] = (df_final2[['ValorAquisicao', 'ValorNominal', 'ValorPresente', 'PDDTotal']] < 0).any(axis=1).sum()
checks_results['VP > Valor Nominal'] = (df_final2['ValorPresente'] > df_final2['ValorNominal']).sum()
checks_results['Valor Aquisição > Valor Nominal'] = (df_final2['ValorAquisicao'] > df_final2['ValorNominal']).sum()

#* Verif Datas
checks_results['Data Aquisição > Data Vencimento'] = (df_final2['DataAquisicao'] > df_final2['DataVencimento']).sum()

#* ver se faltan dados
critical_cols_nulls = ['DataGeracao', 'DataAquisicao', 'DataVencimento', 'ValorPresente', 'ValorNominal', 'PDDTotal', 'SacadoID', 'Originador', 'Convênio']
null_counts = df_final2[critical_cols_nulls].isnull().sum().reset_index()
null_counts.columns = ['Coluna Crítica', 'Registros Faltantes']
null_counts = null_counts[null_counts['Registros Faltantes'] > 0].copy() #  apenas cols com dados faltantes
if not null_counts.empty:
    null_counts['% Faltante'] = (null_counts['Registros Faltantes'] / len(df_final2) * 100).map('{:,.2f}%'.format)
    # Convert a tabela de nulos para um HTML que eh inserido diretamente
    checks_results['[TABELA] Dados Faltantes em Colunas Críticas'] = null_counts.to_html(index=False, classes='dataframe dataframe-checks')
else:
    checks_results['Dados Faltantes em Colunas Críticas'] = "Nenhum dado faltante encontrado."
    
#* ver idades (novidade)
if '_IdadeCliente' in df_final2.columns:
    checks_results['Idade Mínima do Cliente'] = f"{df_final2['_IdadeCliente'].min()} anos"
    checks_results['Idade Máxima do Cliente'] = f"{df_final2['_IdadeCliente'].max()} anos"
    checks_results['Clientes com Idade < 18 ou > 95'] = ((df_final2['_IdadeCliente'] < 18) | (df_final2['_IdadeCliente'] > 95)).sum()

print("Verificações de sanidade concluídas. Os resultados foram armazenados.")


INICIANDO VERIFICAÇÕES DE SANIDADE E QUALIDADE DOS DADOS
Verificações de sanidade concluídas. Os resultados foram armazenados.


### <span style="color:#AEE5F9;"> Análise Exploratória

Análise inicial para entender a composição da carteira. Verifico o uso de memória, o valor total do estoque e a distribuição de valores das principais variáveis categóricas.

In [7]:
memoria_mb = df_final2.memory_usage(deep=True).sum() / 1024**2
print(f"Uso de memória do DataFrame: {memoria_mb:.2f} MB")

valor_total_estoque = df_final2["ValorPresente"].sum()
print(f"Valor Presente Total do Estoque: R$ {valor_total_estoque:_.2f}".replace('.', ',').replace('_', '.'))

print("\n" + "-"*80)
# Contagem de Valores para Variáveis categóricas
# Itero sobre as colunas de texto para entender a distribuição.
print("Analisando a contagem de valores para colunas de texto (geral):")
colunas_interesse_texto = ['Situacao', 'Cedente', 'PagamentoParcial', 'Convênio', 'Originador', 'UF', 'CAPAG', 'Produto', 'Promotora']
for col in colunas_interesse_texto:
    if col in df_final2.columns:
        print(f"\n--- Análise da coluna: {col} ---")
        display(df_final2[col].value_counts(dropna=False).head(20)) #  top 20

Uso de memória do DataFrame: 271.19 MB
Valor Presente Total do Estoque: R$ 35.160.953,80

--------------------------------------------------------------------------------
Analisando a contagem de valores para colunas de texto (geral):

--- Análise da coluna: Situacao ---


Situacao
A vencer    229375
Vencido      18031
Previsto      1007
Name: count, dtype: int64


--- Análise da coluna: Cedente ---


Cedente
BMP MONEY PLUS SOCIEDADE DE CREDITO DIRETO S.A    248413
Name: count, dtype: int64


--- Análise da coluna: PagamentoParcial ---


...

CAPAG
C       154664
A        72131
N.D.     16231
B         5387
Name: count, dtype: int64


--- Análise da coluna: Produto ---


Produto
Cartão RMC - S/T Efetivo                     89422
Empréstimo - S/T Efetivo                     36088
Empréstimo - S/T Temporário                  22578
Cartão Benefício - S/T Temporário            19419
Cartão Benefício S/T EFETIVO                 18332
Cartão Benefício - S/T Comissionado          10705
Cartão RMC - S/T Efetivio                     8424
Cartão Benefício - S/T Efetivo                8145
Cartão Benefício - S/T CONTRATADO             7398
Empréstimo - S/T CONTRATADO                   7116
Empréstimo - Temporário                       5130
Empréstimo - S/T Comissionado                 3264
Cartão RMC - S/T CONTRATADO                   2424
Cartão RMC - C/T Efetivo                      2382
Cartão Benefício*                             2014
Empréstimo - Efetivo                          1848
Cartão RMC - S/T Comissionado                 1778
Empréstimo - C/T Efetivo (3 Lançamentos )      815
Cartão RMC - S/T Temporário                    752
Empréstimo - C/T Efetiv


--- Análise da coluna: Promotora ---


Promotora
STARCARD ANTICIPAY SERVICOS FINANCEIROS LTDA             52664
BARROS PROMOTORA                                         22131
 GP CRED EMPRESTIMOS E FINANCIAMENTOS                    20147
 AVANTE ESTILOS FACILITADORA DE CREDITO                  18664
TG MULTI NEGOCIOS                                        14214
MTT NEGOCIOS LTDA                                        11515
START PROMOTORA - MTT NEGOCIOS LTDA                      11435
ATACRED - E&E SOLUÇÕES FINANCEIRAS LTDA                  11274
BR8 CENTRAL DE CREDITO                                    7555
RAYANNE RODRIGUES                                         5081
LINSCRED SOLUÇÕES FINANCEIRAS                             4662
LUCRA CRED - LUCRACRED ASSESSORIA FINANCEIRA LTDA ME      4549
02081977109 Danielleferrazdamaia                          4207
4BX NEGOCIOS EIRELLI                                      4119
Yeshuah & Figueiredo Soluções Fin. Ltda                   3971
D. S. RAMOS                                  

### <span style="color:#AEE5F9;"> PDD e Vencidos

PDD e o percentual de títulos vencidos, quebrando a análise pelas diversas variáveis categóricas para identificar os segmentos de maior risco.

In [8]:
# =============================================================================
# analise do PDD e Vencidos   ===============================================>>
# (por variável categórica)   ===============================================>>
# =============================================================================
# PDD e a inadimplência por diversas categorias.
cat_cols = [
    'Situacao', 'Cedente', 'PagamentoParcial',
    '_MuitosContratos', '_MuitosEntes', 'Convênio', 'Originador', 'Produto',
    'UF', 'CAPAG', 'Promotora'
]

cat_cols = [col for col in cat_cols if col in df_final2.columns] # tiro cols que nao estejam no df 

# PDD -----------------------------------------------------------------------------------
print("--- Análise de PDD por Categoria ---")
pdd_ref = (1 - df_final2['_ValorLiquido'].sum() / df_final2['ValorPresente'].sum()) * 100
print(f"PDD de Referência (Total): {pdd_ref:.2f}%\n")

for col in cat_cols:
    print(f"Análise de PDD por '{col}':")
    aux_ = df_final2.groupby(col)[['_ValorLiquido', 'ValorPresente']].sum() / 1e6
    aux_['%PDD'] = (1 - aux_['_ValorLiquido'] / aux_['ValorPresente']) * 100
    display(aux_.sort_values('ValorPresente', ascending=False).head(20))

# Vencidos --------------------------------------------------------------------------
print("\n" + "="*80 + "\n")
print("--- Análise de Vencidos por Categoria ---")
venc_ref = (df_final2['_ValorVencido'].sum() / df_final2['ValorPresente'].sum()) * 100
print(f"Percentual de Vencidos de Referência (Total): {venc_ref:.2f}%\n")

for col in cat_cols:
    print(f"Análise de Vencidos por '{col}':")
    aux_ = df_final2.groupby(col)[['_ValorVencido', 'ValorPresente']].sum() / 1e6
    aux_['%Vencido'] = (aux_['_ValorVencido'] / aux_['ValorPresente']) * 100
    display(aux_.sort_values('%Vencido', ascending=False).head(20))

--- Análise de PDD por Categoria ---
PDD de Referência (Total): 23.40%

Análise de PDD por 'Situacao':


,_ValorLiquido,ValorPresente,%PDD
Situacao,,,
A vencer,24.165965,30.588039,20.995377
Vencido,2.611387,4.376060,40.325620
Previsto,0.154540,0.196854,21.495355


Análise de PDD por 'Cedente':


,_ValorLiquido,ValorPresente,%PDD
Cedente,,,
BMP MONEY PLUS SOCIEDADE DE CREDITO DIRETO S.A,26.931892,35.160954,23.403979


Análise de PDD por 'PagamentoParcial':


...

,_ValorVencido,ValorPresente,%Vencido
UF,,,
GO,2.859188,18.340010,15.589891
RN,0.078199,0.551887,14.169328
PE,0.083260,0.718529,11.587607
MA,0.047473,0.499191,9.510032
SP,1.213036,12.907580,9.397860
PR,0.020613,0.238650,8.637238
CE,0.055201,1.149046,4.804074
PA,0.019090,0.756060,2.524936


Análise de Vencidos por 'CAPAG':


,_ValorVencido,ValorPresente,%Vencido
CAPAG,,,
C,3.358749,24.189972,13.884884
A,0.827552,7.999844,10.344599
N.D.,0.129760,2.015493,6.438119
B,0.059999,0.955646,6.278370


Análise de Vencidos por 'Promotora':


,_ValorVencido,ValorPresente,%Vencido
Promotora,,,
MTT NEGOCIOS LTDA,0.670273,2.184963,30.676620
SOLAR CONSULTORIA E SERVIÇOS LTDA,0.030838,0.105866,29.129421
SOLUÇÕES ASSESSORIA - ALEXSANDRA GROSSER LTDA,0.001156,0.004493,25.728683
Yeshuah & Figueiredo Soluções Fin. Ltda - GEICEL HENRIQUE PEREIRA DE SOUSA E SILVA,0.000508,0.001976,25.723486
GERAR PARTICIPAÇAO E INTERMEDIAÇAO,0.014865,0.058010,25.624068
X CONNECT CONSULTORIA LTDA,0.007975,0.032161,24.798241
GL CREDITOS INTELIGENTES,0.002702,0.011137,24.265391
REAL CREDITO EMPRESTIMO CONSIGNADO,0.037840,0.176913,21.388959
02081977109 Danielleferrazdamaia,0.135417,0.644157,21.022320


### <span style="color:#AEE5F9;"> Verificações de Consistência Adicionais

Algumas verificações focadas na lógica do negócio, como a presença de clientes em múltiplos convênios e a consistência entre datas e valores.

In [9]:
# =============================================================================
# Verificações de Consistência básica  ========================================
# =============================================================================
print("Analisando sacados presentes em múltiplos convênios (usando SacadoID)...")
sacados_multi_entes = df_final2.groupby('SacadoID')['Convênio'].agg(['nunique', pd.unique])
display(sacados_multi_entes.sort_values('nunique', ascending=False).head(20))

print("\nVerificando consistência das datas...")
check2 = (df_final2['DataAquisicao'] > df_final2['DataVencimento']).sum()
print(f"Registros com Data de Aquisição > Data de Vencimento: {check2}")

print("\nVerificando consistência dos valores...")
check_v1 = (df_final2['ValorAquisicao'] > df_final2['ValorNominal']).sum()
print(f"Registros com Valor de Aquisição > Valor Nominal: {check_v1}")
check_v2 = (df_final2['ValorAquisicao'] > df_final2['ValorPresente']).sum()
print(f"Registros com Valor de Aquisição > Valor Presente: {check_v2}")
check_v3 = (df_final2['ValorPresente'] > df_final2['ValorNominal']).sum()
print(f"Registros com Valor Presente > Valor Nominal: {check_v3}")

Analisando sacados presentes em múltiplos convênios (usando SacadoID)...


,nunique,unique
SacadoID,,
1,1,[GOV. GOIAS]
2885,1,[PREF. ARAÇATUBA]
3081,1,[PREF. ARAÇATUBA]
3080,1,[GOV. GOIAS]
3079,1,[PREF. TABOAO DA SERRA]
3078,1,[PREF. ARAÇATUBA]
3077,1,[GOV. GOIAS]
3076,1,[GOV. GOIAS]
3075,1,[PREF. ARAÇATUBA]



Verificando consistência das datas...
Registros com Data de Aquisição > Data de Vencimento: 0

Verificando consistência dos valores...
Registros com Valor de Aquisição > Valor Nominal: 248
Registros com Valor de Aquisição > Valor Presente: 248
Registros com Valor Presente > Valor Nominal: 0


### <span style="color:#AEE5F9;"> Geração do Relatório Final

Consolido todas as análises anteriores em um único  HTML. O processo é dividido em subpartes.

#### <span style="color:#FFDAC1;"> Cálculo de Métricas 

Aqui preparo as tabelas de métricas (PDD, Vencido, Ticket Médio) que serão exibidas no relatório. Agrupo os dados pelas dimensões de análise e calculamos o PDD, o % Vencido e o Ticket Médio Ponderado para cada segmento.
<br> CHANGELOG: <span style="color:#CFFFE5;">ATUALIZADO na versão 1.02</span>

In [10]:
# =============================================================================
# Geração do Relatório  ======================================================
# =============================================================================

dimensoes_analise = {
    'Cedentes': 'Cedente',
    'Originadores': 'Originador',
    'Promotoras': 'Promotora',
    'Produtos': 'Produto',
    'Convênios': 'Convênio',
    'Situação': 'Situacao',
    'UF': 'UF',
    'CAPAG': 'CAPAG',
    'Pagamento Parcial': 'PagamentoParcial',
    'Tem Muitos Contratos':'_MuitosContratos',
    'Tem Muitos Entes':'_MuitosEntes'
}
dimensoes_analise = {k: v for k, v in dimensoes_analise.items() if v in df_final2.columns} # prova se colunas tao no df

# DIC DE EXEMPLO
COST_DICT = {
    'GOV. GOIAS': [0.035, 5.92],
    'PREF. COTIA': [0.03, 2.14],
}
DEFAULT_COST = [0.035, 5.92]

os.makedirs(output_path, exist_ok=True)


"""
#ANTIGO
#***********************
#* CÁLCULO DAS MÉTRICAS
#***********************
print("\n" + "="*80)
print("INICIANDO CÁLCULO DAS MÉTRICAS DE RISCO E INADIMPLÊNCIA")
print("="*80)

# Nomes das colunas para o relatório final (mantido)
vp_col_name = 'Valor Presente \n(R$ MM)'
vl_col_name = 'Valor Líquido \n(R$ MM)'

tabelas_pdd = {}
tabelas_vencido = {}

# Risco: % PDD
for nome_analise, coluna in dimensoes_analise.items():
    if coluna not in df_final2.columns: continue
    
    aux_pdd = df_final2.groupby(coluna, observed=False)[['_ValorLiquido', 'ValorPresente']].sum()
   
    aux_pdd['%PDD'] = (1 - aux_pdd['_ValorLiquido'] / aux_pdd['ValorPresente']) * 100
    

    #renomeio as colunas para o relatório
    aux_pdd = aux_pdd.rename(columns={'ValorPresente': vp_col_name, '_ValorLiquido': vl_col_name})
    
    # escalar os valores para milhões
    aux_pdd[[vp_col_name, vl_col_name]] /= 1e6
    
    tabelas_pdd[nome_analise] = aux_pdd

# Inadimplencia --- #* % Vencido
for nome_analise, coluna in dimensoes_analise.items():
    if coluna not in df_final2.columns: continue
    
    # somas brutas
    aux_venc = df_final2.groupby(coluna, observed=False)[['_ValorVencido', 'ValorPresente']].sum()

    # Calcular o percentual USANDO OS VALORES BRUTO
    # Isso garante que a proporção esteja crta
    aux_venc['%Vencido'] = (aux_venc['_ValorVencido'] / aux_venc['ValorPresente']) * 100
    
    # renomear as colunas
    aux_venc = aux_venc.rename(columns={'ValorPresente': vp_col_name, '_ValorVencido': 'ValorVencido (M)'})
    
    # escalar valores para milhões
    aux_venc[[vp_col_name, 'ValorVencido (M)']] /= 1e6
    
    tabelas_vencido[nome_analise] = aux_venc

print("Métricas de PDD e Inadimplência calculadas corretamente.")
"""
#***********************
#* CÁLCULO MÉTRICAS ( NOVIDADE: contagem de CONTRATOS vencidos)
#***********************
print("\n" + "="*80)
print("INICIANDO CÁLCULO UNIFICADO DAS MÉTRICAS")
print("="*80)

vp_col_name = 'Valor Presente \n(R$ MM)'
vl_col_name = 'Valor Líquido \n(R$ MM)'
tabelas_metricas = {}

for nome_analise, coluna in dimensoes_analise.items():
    if coluna not in df_final2.columns: continue
    print(f"Calculando métricas para a dimensão: '{nome_analise}'...")

    grouped = df_final2.groupby(coluna, observed=False)
    
    # [NOVIDADE] a contagem agora eh baseada em CCBs unicos.
    # 
    total_contratos_unicos = grouped['CCB'].nunique()
    
    # agg os ccbs unicos depois de filtrar os vencidos: 
    contratos_vencidos_unicos = df_final2[df_final2['_ContratoVencido_Flag'] == 1].groupby(coluna, observed=False)['CCB'].nunique()

    df_metricas = pd.DataFrame({'Nº Contratos Únicos': total_contratos_unicos}) # < aqui comeco a base da tabela
    df_metricas = df_metricas.join(pd.DataFrame({'Nº Contratos Vencidos': contratos_vencidos_unicos}))
    df_metricas['Nº Contratos Vencidos'] = df_metricas['Nº Contratos Vencidos'].fillna(0).astype(int) # Preenche com 0 se não houver vencidos

    # junto somas como antes
    somas_financeiras = grouped[['_ValorLiquido', 'ValorPresente', '_ValorVencido']].sum()
    df_metricas = df_metricas.join(somas_financeiras)
    
    # calc % : 
    df_metricas['%PDD'] = (1 - df_metricas['_ValorLiquido'] / df_metricas['ValorPresente']) * 100
    df_metricas['% Vol. Vencido'] = (df_metricas['_ValorVencido'] / df_metricas['ValorPresente']) * 100
    df_metricas['% Contratos Vencidos'] = (df_metricas['Nº Contratos Vencidos'] / df_metricas['Nº Contratos Únicos']) * 100 # Agora a fórmula está correta
    
    # organizo as colunas
    df_metricas = df_metricas.rename(columns={'ValorPresente': vp_col_name, '_ValorLiquido': vl_col_name, 'Nº Contratos Únicos': 'Nº Contratos'})
    df_metricas[[vp_col_name, vl_col_name]] /= 1e6
    df_metricas = df_metricas.drop(columns=['_ValorLiquido', '_ValorVencido'], errors='ignore')

    tabelas_metricas[nome_analise] = df_metricas

print("Cálculo unificado de métricas concluído.")


#***********************
#* TICKET MÉDIO 
#***********************
tabelas_ticket = {}
for nome_analise, coluna in dimensoes_analise.items():
    df_temp = df_final2.dropna(subset=[coluna, 'ValorPresente', 'ValorNominal'])
    if df_temp.empty: continue
    grouped = df_temp.groupby(coluna, observed=False)
    numerador = grouped.apply(lambda g: (g['ValorNominal'] * g['ValorPresente']).sum(), include_groups=False)
    denominador = grouped['ValorPresente'].sum()
    ticket_ponderado = (numerador / denominador).replace([np.inf, -np.inf], 0)
    ticket_ponderado.name = "Ticket Ponderado (R$)"
    tabelas_ticket[nome_analise] = pd.DataFrame(ticket_ponderado)

print("Ticket médio calculado.")


INICIANDO CÁLCULO UNIFICADO DAS MÉTRICAS
Calculando métricas para a dimensão: 'Cedentes'...
Calculando métricas para a dimensão: 'Originadores'...
Calculando métricas para a dimensão: 'Promotoras'...
Calculando métricas para a dimensão: 'Produtos'...
Calculando métricas para a dimensão: 'Convênios'...
Calculando métricas para a dimensão: 'Situação'...
Calculando métricas para a dimensão: 'UF'...
Calculando métricas para a dimensão: 'CAPAG'...
Calculando métricas para a dimensão: 'Pagamento Parcial'...
Calculando métricas para a dimensão: 'Tem Muitos Contratos'...
Calculando métricas para a dimensão: 'Tem Muitos Entes'...
Cálculo unificado de métricas concluído.
Ticket médio calculado.


#### <span style="color:#FFDAC1;"> Cálculo da TIR 

Implemento a lógica para o cálculo da TIR (XIRR). Uso uma função para encontrar a taxa que zera o VPL do fluxo de caixa e a aplico para cada segmento da carteira. TIR em diferentes cenários: Bruta, Líquida de PDD, Líquida de Custos e Completa.

In [11]:
#******************
#* TIR com brentq
#*******************
def calculate_xirr(cash_flows, days):
    cash_flows = np.array(cash_flows)
    days = np.array(days)
    def npv(rate):
        if rate <= -1: return float('inf')
        with np.errstate(divide='ignore', over='ignore'):
            return np.sum(cash_flows / (1 + rate) ** (days / 21.0))
    try:
        return brentq(npv, 0, 1.0)
    except ValueError:
        try:
            return brentq(npv, -0.9999, 0)
        except (RuntimeError, ValueError):
            return np.nan


print("\n" + "="*80)
print("INICIANDO CÁLCULO DA TAXA INTERNA DE RETORNO (TIR)")
print("="*80)

ref_date = df_final2['DataGeracao'].max()
print(f"Data de Referência para o cálculo da TIR: {ref_date.strftime('%d/%m/%Y')}")
try:
    df_feriados = pd.read_excel(caminho_feriados)
    holidays = pd.to_datetime(df_feriados['Data']).values.astype('datetime64[D]')
    print(f"Sucesso: {len(holidays)} feriados carregados.")
except Exception as e:
    print(f"[AVISO] Não foi possível carregar feriados: {e}")
    holidays = []

df_avencer = df_final2[df_final2['DataVencimento'] > ref_date].copy()
try:
    start_dates = np.datetime64(ref_date.date())
    end_dates = df_avencer['DataVencimento'].values.astype('datetime64[D]')
    df_avencer.loc[:, '_DIAS_UTEIS_'] = np.busday_count(start_dates, end_dates, holidays=holidays)
    df_avencer = df_avencer[df_avencer['_DIAS_UTEIS_'] > 0]
except Exception as e:
    print(f"[ERRO] Falha ao calcular dias úteis: {e}")
    df_avencer.loc[:, '_DIAS_UTEIS_'] = np.nan

df_avencer['CustoVariavel'] = df_avencer['Convênio'].map(lambda x: COST_DICT.get(x, DEFAULT_COST)[0])
df_avencer['CustoFixo'] = df_avencer['Convênio'].map(lambda x: COST_DICT.get(x, DEFAULT_COST)[1])
# Receita líquida
df_avencer['ReceitaLiquida'] = df_avencer['ValorNominal'] - (df_avencer['CustoFixo'] + (df_avencer['CustoVariavel'] * df_avencer['ValorNominal']))


all_tirs = []
segmentos_para_analise = [('Carteira Total', 'Todos')] + \
                         [(col, seg) for col in cat_cols if col in df_avencer.columns for seg in df_avencer[col].dropna().unique()]

# ==============================================================================
# LOOP DA TIR ================
# ==============================================================================
for tipo_dimensao, segmento in segmentos_para_analise:
    if tipo_dimensao == 'Carteira Total':
        df_segmento = df_avencer.copy() # Usar .copy() para evitar SettingWithCopyWarning
    else:
        df_segmento = df_avencer[df_avencer[tipo_dimensao] == segmento].copy()

    # fallback herdado (provavelmente lixo)
    if df_segmento.empty or df_segmento['_DIAS_UTEIS_'].isnull().all():
        continue

    vp_bruto = df_segmento['ValorPresente'].sum()
    tir_bruta, tir_pdd, tir_custos, tir_completa = np.nan, np.nan, np.nan, np.nan

    if vp_bruto > 0:
        # =Calculo TIR Bruta 
        fluxos_brutos = df_segmento.groupby('_DIAS_UTEIS_')['ValorNominal'].sum()
        tir_bruta = calculate_xirr([-vp_bruto] + fluxos_brutos.values.tolist(), [0] + fluxos_brutos.index.tolist())

        # taxa de PDD robusta
        pdd_total_segmento = df_segmento['PDDTotal'].sum()
        if pd.notna(pdd_total_segmento) and vp_bruto > 0:
            pdd_rate = pdd_total_segmento / vp_bruto
        else:
            pdd_rate = 0.0 #  0 se não houver PDD ou VP

        df_segmento['Fluxo_PDD'] = df_segmento['ValorNominal'] * (1 - pdd_rate)
        df_segmento['Fluxo_Custos'] = df_segmento['ReceitaLiquida'] #  calculado antes do loop
        
        # subtrai os custos do fluxo já líquido de PDD.
        df_segmento['Fluxo_Completo'] = df_segmento['Fluxo_PDD'] - (df_segmento['CustoFixo'] + (df_segmento['CustoVariavel'] * df_segmento['ValorNominal']))


        # fluxos de caixa anuais
        fluxos_pdd = df_segmento.groupby('_DIAS_UTEIS_')['Fluxo_PDD'].sum()
        fluxos_custos = df_segmento.groupby('_DIAS_UTEIS_')['Fluxo_Custos'].sum()
        fluxos_completos = df_segmento.groupby('_DIAS_UTEIS_')['Fluxo_Completo'].sum()

        # TIRs líquidas com os fluxos de caixa
        tir_pdd = calculate_xirr([-vp_bruto] + fluxos_pdd.values.tolist(), [0] + fluxos_pdd.index.tolist())
        tir_custos = calculate_xirr([-vp_bruto] + fluxos_custos.values.tolist(), [0] + fluxos_custos.index.tolist())
        tir_completa = calculate_xirr([-vp_bruto] + fluxos_completos.values.tolist(), [0] + fluxos_completos.index.tolist())
        
    # A lógica de anexar os resultados permanece a mesma
    all_tirs.append({
        'DimensaoColuna': tipo_dimensao,
        'Segmento': segmento,
        'Valor Presente TIR (M)': vp_bruto / 1e6,
        'TIR Bruta \n(% a.m. )': tir_bruta * 100 if pd.notna(tir_bruta) else np.nan,
        'TIR Líquida de PDD \n(% a.m. )': tir_pdd * 100 if pd.notna(tir_pdd) else np.nan,
        'TIR Líquida de custos \n(% a.m. )': tir_custos * 100 if pd.notna(tir_custos) else np.nan,
        'TIR Líquida Final \n(% a.m. )': tir_completa * 100 if pd.notna(tir_completa) else np.nan,
    })

df_tir_summary = pd.DataFrame(all_tirs)
tir_cols_to_fill = [col for col in df_tir_summary.columns if 'TIR' in col]
df_tir_summary[tir_cols_to_fill] = df_tir_summary[tir_cols_to_fill].fillna(-100.0)
print("Cálculo de TIR concluído.")


INICIANDO CÁLCULO DA TAXA INTERNA DE RETORNO (TIR)
Data de Referência para o cálculo da TIR: 05/08/2025
Sucesso: 1264 feriados carregados.
Cálculo de TIR concluído.


#### <span style="color:#FFDAC1;">  Montagem do HTML

Uno todos os elementos: as verificações de sanidade, as tabelas de métricas e os resultados da TIR. Boto um estilo CSS para formatação, codo a logo em base64 e monto a estrutura HTML, que é salva em um arquivo local.
<br> CHANGELOG: <span style="color:#CFFFE5;">ATUALIZADO na versão 1.02</span>

In [12]:
#***********************
#* GERAÇÃO DO RELATÓRIO HTML
#***********************

print("\n" + "="*80)
print("GERANDO RELATÓRIO HTML FINAL COM AJUSTES DE ESTILO")
print("="*80)

# LOGO E DATA  
def encode_image_to_base64(image_path):
    try:
        with open(image_path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')
    except FileNotFoundError:
        print(f"[ATENÇÃO] Arquivo de imagem não encontrado em: {image_path}. A logo não será exibida.")
        return None


logo_base64 = encode_image_to_base64(logo_path)
report_date = ref_date.strftime('%d/%m/%Y')

# CSS - similar ao anterior, com algumas adições apenas 
html_css = """
<style>

    .checks-container {
        display: grid;
        grid-template-columns: repeat(auto-fit, minmax(350px, 1fr));
        gap: 15px;
        margin-bottom: 20px;
    }
    .check-item {
        background-color: #f5f5f5;
        padding: 12px;
        border-radius: 5px;
        border-left: 4px solid #76c6c5;
        font-size: 0.95em;
    }
    .check-item strong {
        color: #163f3f;
    }
    .dataframe-checks th {
        background-color: #5b8c8c; /* Um tom mais claro para diferenciar */
    }
    .check-table-wrapper h4 {
        margin-top: 20px;
        margin-bottom: 10px;
        color: #163f3f;
        border-bottom: 2px solid #eeeeee;
        padding-bottom: 5px;
    }

    /* Configurações Gerais */
    body {
        font-family: "Gill Sans MT", Arial, sans-serif;
        background-color: #f9f9f9;
        color: #313131;
        margin: 0;
        padding: 0;
    }
    .main-content {
        padding: 25px;
    }

    /* --- CABEÇALHO --- */
    header {
        background-color: #163f3f;
        color: #FFFFFF;
        padding: 20px 40px;
        display: flex;
        justify-content: space-between;
        align-items: center;
        border-bottom: 5px solid #76c6c5;
    }
    /* 3. LOGO MAIOR: Altura da logo aumentada novamente */
    header .logo img {
        height: 75px; /* Aumentado de 65px para 75px */
    }
    header .report-title {
        text-align: left;
        font-family: "Gill Sans MT", Arial, sans-serif;
    }
    header .report-title h1, header .report-title h2, header .report-title h3 {
        margin: 0;
        padding: 0;
        font-weight: normal;
    }
    /* 2. FONTE MENOR: Tamanho do título principal reduzido */
    header .report-title h1 { font-size: 1.6em; /* Reduzido de 1.8em para 1.6em */ }
    header .report-title h2 { font-size: 1.4em; color: #d0d0d0; }
    header .report-title h3 { font-size: 1.1em; color: #a0a0a0; }

    /* Estilos dos Botões e Tabelas (sem alterações) */
    .container-botoes { display: flex; flex-wrap: wrap; gap: 15px; margin-bottom: 25px; }
    .container-botoes > details { flex: 1 1 280px; border: 1px solid #76c6c5; border-radius: 8px; overflow: hidden; }
    .container-botoes > details[open] { flex-basis: 100%; }
    details summary { font-size: 1.1em; font-weight: bold; color: #FFFFFF; background-color: #163f3f; padding: 15px 20px; cursor: pointer; outline: none; list-style-type: none; }
    details summary:hover { background-color: #0e5d5f; }
    details[open] summary { background-color: #76c6c5; color: #313131; }
    details[open] summary:hover { filter: brightness(95%); }
    summary::-webkit-details-marker { display: none; }
    summary::before { content: '► '; margin-right: 8px; font-size: 0.8em;}
    details[open] summary::before { content: '▼ '; }
    details .content-wrapper { padding: 20px; background-color: #FFFFFF; }
    table.dataframe, th, td { border: 1px solid #bbbbbb; }
    table.dataframe { border-collapse: collapse; width: 100%; }
    th, td { text-align: left; padding: 10px; vertical-align: middle; }
    th { background-color: #163f3f; color: #FFFFFF; }
    tr:nth-child(even) { background-color: #eeeeee; }

    /* --- RODAPÉ --- */
    footer {
        background-color: #f0f0f0;
        color: #555555;
        font-size: 0.8em;
        line-height: 1.6;
        padding: 25px 40px;
        margin-top: 40px;
        border-top: 1px solid #dddddd;
    }
    footer .disclaimer {
        margin-top: 20px;
        font-style: italic;
        border-top: 1px solid #dddddd;
        padding-top: 20px;
    }
</style>
"""

# HTML COMPLETO:
html_parts = []


html_parts.append("<!DOCTYPE html><html lang='pt-BR'><head>")
html_parts.append("<meta charset='UTF-8'><title>Análise de Desempenho - FCT Consignado II</title>")
html_parts.append(html_css)
html_parts.append("</head><body>")

# --- add o Cabeçalho ---
html_parts.append("<header>")
html_parts.append(f"""
<div class="report-title">
    <h1>Análise de desempenho</h1>
    <h2>FCT CONSIGNADO II</h2>
    <h3>{report_date}</h3>
</div>
""")

if logo_base64:
    html_parts.append(f'<div class="logo"><img src="data:image/png;base64,{logo_base64}" alt="Logo"></div>')
html_parts.append("</header>")


html_parts.append("<div class='main-content'>")

# [ADAPTADO] Mapa de descrições atualizado com as novas dimensões
mapa_descricoes = {
    'Cedentes': 'Analisa as métricas de risco e retorno agrupadas por cada Cedente.',
    'Originadores': 'Compara o desempenho da carteira originada por cada parceiro.',
    'Promotoras': 'Métricas agrupadas pela promotora de vendas responsável pela operação.',
    'Produtos': 'Agrupa os dados pelo tipo de produto de crédito (Ex: Cartão Benefício).',
    'Convênios': 'Métricas detalhadas por cada Convênio onde a consignação é feita.',
    'Situação': 'Compara o desempenho dos títulos com base na sua situação atual (Ex: A vencer).',
    'UF': 'Agrega todas as métricas por Unidade Federativa (Estado) do cliente.',
    'CAPAG': 'Métricas baseadas na Capacidade de Pagamento (CAPAG) do município ou estado do convênio.',
    'Pagamento Parcial': 'Verifica se há impacto nas métricas para títulos que aceitam pagamento parcial.',
    'Tem Muitos Contratos': 'Compara clientes com um número baixo vs. alto de contratos ativos.',
    'Tem Muitos Entes': 'Compara clientes que operam em poucos vs. múltiplos convênios.'
}

# ==============================================================================
#  Parte nova- verificacao dos dados ==========================
# ==============================================================================
html_parts.append("<details open>") # 'open' ===  comece já expandido
html_parts.append("<summary>Verificações e Sanidade dos Dados</summary>")
html_parts.append("<div class='content-wrapper'>")

# resultados simples
simple_checks_html = "<div class='checks-container'>"
# Estrutura tabelas
table_checks_html = "<div class='check-table-wrapper'>"

for key, value in checks_results.items():
    # ratata tabela html difente por chave
    if '[TABELA]' in key:
        clean_key = key.replace('[TABELA]', '').strip()
        table_checks_html += f"<h4>{clean_key}</h4>"
        table_checks_html += str(value)
    else:
        simple_checks_html += f"<div class='check-item'><strong>{key}:</strong> {value}</div>"

simple_checks_html += "</div>"
table_checks_html += "</div>"

html_parts.append(simple_checks_html)
html_parts.append(table_checks_html)

html_parts.append("</div></details>")

html_parts.append("<div class='container-botoes'>")
dimensoes_ordem_alfabetica = ['CAPAG'] # add outras se necessário
"""
for nome_analise, coluna in dimensoes_analise.items():
    if coluna not in df_final2.columns or df_final2[coluna].isnull().all(): continue
    print(f"--> Processando e gerando HTML para o botão: '{nome_analise}'")
    
    # Junção tabelas de métricas
    df_pdd = tabelas_pdd.get(nome_analise)
    df_venc = tabelas_vencido.get(nome_analise)
    df_ticket = tabelas_ticket.get(nome_analise)
    df_tir = df_tir_summary[df_tir_summary['DimensaoColuna'] == coluna].set_index('Segmento')
    if df_pdd is None: continue

    df_final = df_pdd.join(df_venc.drop(columns=[vp_col_name]), how='outer')
    if df_ticket is not None: df_final = df_final.join(df_ticket, how='outer')
    df_final = df_final.join(df_tir.drop(columns=['DimensaoColuna']), how='outer')
    df_final.index.name = nome_analise
    df_final.reset_index(inplace=True)
    ANTIGO
    df_final = df_final.drop(columns=['ValorVencido (M)', 'Valor Presente TIR (M)'], errors='ignore')

    # Ordenação das cols
    colunas_ordem = [nome_analise, vl_col_name, vp_col_name]
    if 'Ticket Ponderado (R$)' in df_final.columns: colunas_ordem.append('Ticket Ponderado (R$)')
    colunas_ordem.extend(['%PDD', '%Vencido'])
    colunas_tir_existentes = sorted([col for col in df_tir.columns if col in df_final.columns and 'TIR' in col])
    colunas_finais = colunas_ordem + colunas_tir_existentes
    outras_colunas = [col for col in df_final.columns if col not in colunas_finais]
    df_final = df_final[colunas_finais + outras_colunas]

    # Ordenação linhas
    if nome_analise in dimensoes_ordem_alfabetica:
        df_final = df_final.sort_values(nome_analise, ascending=True).reset_index(drop=True)
    else:
        df_final = df_final.sort_values(vp_col_name, ascending=False).reset_index(drop=True)

    # Formatação
    formatters = {
        vl_col_name: lambda x: f'{x:,.2f}',
        vp_col_name: lambda x: f'{x:,.2f}',
        'Ticket Ponderado (R$)': lambda x: f'R$ {x:,.2f}',
        '%PDD': lambda x: f'{x:,.2f}%',
        '%Vencido': lambda x: f'{x:,.2f}%',
    }
    for col in colunas_tir_existentes: formatters[col] = lambda x: f'{x:,.2f}%'
    
    #quebra de linha com tag <br> 
    df_final.columns = [col.replace('\n', '<br>') for col in df_final.columns]
    
    """
for nome_analise, coluna in dimensoes_analise.items():
    if coluna not in df_final2.columns or df_final2[coluna].isnull().all(): continue
    print(f"--> Process e gerando HTML para : '{nome_analise}'")
    
    # table de métricas já pronta.
    df_final = tabelas_metricas.get(nome_analise)
    
    # Juntode Ticket e TIR.
    df_ticket = tabelas_ticket.get(nome_analise, pd.DataFrame())
    df_tir = df_tir_summary[df_tir_summary['DimensaoColuna'] == coluna].set_index('Segmento')
    
    df_final = df_final.join(df_ticket, how='outer')
    df_final = df_final.join(df_tir.drop(columns=['DimensaoColuna', 'Valor Presente TIR (M)']), how='outer')
    
    df_final.index.name = nome_analise
    df_final.reset_index(inplace=True)
    
    colunas_ordem = [
        nome_analise,
        'Nº Contratos',
        'Nº Contratos Vencidos',
        '% Contratos Vencidos',
        vl_col_name,
        vp_col_name,
        '%PDD',
        '% Vol. Vencido'
    ]
    if 'Ticket Ponderado (R$)' in df_final.columns: colunas_ordem.append('Ticket Ponderado (R$)')
    
    ordem_ideal_tir = [
        'TIR Bruta \n(% a.m. )',
        'TIR Líquida de PDD \n(% a.m. )',
        'TIR Líquida de custos \n(% a.m. )',
        'TIR Líquida Final \n(% a.m. )'
    ]
    
    colunas_tir_ordenadas = [col for col in ordem_ideal_tir if col in df_final.columns] # lsta na ordem, desde que esteja nos dados
    
    colunas_finais = colunas_ordem + colunas_tir_ordenadas
    outras_colunas = [col for col in df_final.columns if col not in colunas_finais]
    df_final = df_final[colunas_finais + outras_colunas]
    
   
    if nome_analise in ['CAPAG']:  # Orden  
        df_final = df_final.sort_values(nome_analise, ascending=True).reset_index(drop=True)
    else:
        df_final = df_final.sort_values(vp_col_name, ascending=False).reset_index(drop=True)

    # Format
    formatters = {
        vl_col_name: lambda x: f'{x:,.2f}',
        vp_col_name: lambda x: f'{x:,.2f}',
        'Nº Contratos': lambda x: f'{x:,.0f}'.replace(',', '.'),
        'Nº Contratos Vencidos': lambda x: f'{x:,.0f}'.replace(',', '.'),
        'Ticket Ponderado (R$)': lambda x: f'R$ {x:,.2f}'.replace(',', 'X').replace('.', ',').replace('X', '.'),
        '%PDD': lambda x: f'{x:,.2f}%',
        '% Vol. Vencido': lambda x: f'{x:,.2f}%',
        '% Contratos Vencidos': lambda x: f'{x:,.2f}%',
    }
    
    for col in colunas_tir_ordenadas:
        formatters[col] = lambda x: f'{x:,.2f}%'
    
    df_final.columns = [col.replace('\n', '<br>') for col in df_final.columns]
    
    # GERO HTML para a tabela
    html_parts.append("<details>")
    descricao = mapa_descricoes.get(nome_analise, 'Descrição não disponível.')
    html_parts.append(f'<summary title="{descricao}">{nome_analise}</summary>')
    html_parts.append("<div class='content-wrapper'>")
    html_table = df_final.to_html(index=False, classes='dataframe', formatters=formatters, na_rep='-', escape=False)
    html_parts.append(html_table)
    html_parts.append("</div></details>")

html_parts.append("</div>") # Fim do container-botoes
html_parts.append("</div>") # Fim do main-content

#! obs: herdado do outro código
# (Não sei se é o caso, se aplica aqui)
footer_main_text = """
Este documento tem como objetivo apresentar uma análise de desempenho do fundo FCT Consignado II (CNPJ 52.203.615/0001-19), realizada pelo Porto Real Investimentos na qualidade de cogestora. Os prestadores de serviço do fundo são: FICTOR ASSET (Gestor), Porto Real Investimentos (Cogestor), e VÓRTX DTVM (Administrador e Custodiante).
"""
footer_disclaimer = """
Disclaimer: Este relatório foi preparado pelo Porto Real Investimentos exclusivamente para fins informativos e não constitui uma oferta de venda, solicitação de compra ou recomendação para qualquer investimento. As informações aqui contidas são baseadas em fontes consideradas confiáveis na data de sua publicação, mas não há garantia de sua precisão ou completude. Rentabilidade passada não representa garantia de rentabilidade futura.
"""
html_parts.append("<footer>")
html_parts.append(f'<p>{footer_main_text.strip()}</p>')
html_parts.append(f'<div class="disclaimer">{footer_disclaimer.strip()}</div>')
html_parts.append("</footer>")


html_parts.append("</body></html>")

final_html_content = "\n".join(html_parts)
html_output_filename = os.path.join(output_path, 'analise_originadores_consolidada.html') # obs: Nome do arquivo novo diferente
try:
    with open(html_output_filename, 'w', encoding='utf-8') as f:
        f.write(final_html_content)
    print("\n" + "="*80)
    print("ANÁLISE CONCLUÍDA COM SUCESSO!")
    print(f"O relatório HTML final foi salvo em: {html_output_filename}")
    print("="*80)
except Exception as e:
    print(f"\n[ERRO GRAVE] Não foi possível salvar o arquivo HTML: {e}")


GERANDO RELATÓRIO HTML FINAL COM AJUSTES DE ESTILO
--> Process e gerando HTML para : 'Cedentes'
--> Process e gerando HTML para : 'Originadores'
--> Process e gerando HTML para : 'Promotoras'
--> Process e gerando HTML para : 'Produtos'
--> Process e gerando HTML para : 'Convênios'
--> Process e gerando HTML para : 'Situação'
--> Process e gerando HTML para : 'UF'
--> Process e gerando HTML para : 'CAPAG'
--> Process e gerando HTML para : 'Pagamento Parcial'
--> Process e gerando HTML para : 'Tem Muitos Contratos'
--> Process e gerando HTML para : 'Tem Muitos Entes'

ANÁLISE CONCLUÍDA COM SUCESSO!
O relatório HTML final foi salvo em: C:\Users\Leo\Desktop\Porto_Real\portoauto\src\originadores\output\analise_originadores_consolidada.html


# Verificações

## Verificações de Consistência

In [13]:
#%%
"""Executa uma série de análises na base de dados para fornecer um
# panorama geral de qualidade, consistência e estrutura dos dados."""

# Resumo
print("\nResumo")
valor_presente_formatado = f"R$ {df_final2['ValorPresente'].sum():,.2f}".replace(',', 'X').replace('.', ',').replace('X', '.')
total_registros_formatado = f"{len(df_final2):,}".replace(',', '.')
clientes_unicos_formatado = f"{df_final2['SacadoID'].nunique():,}".replace(',', '.')
ccbs_unicos_formatado = f"{df_final2['CCB'].nunique():,}".replace(',', '.')

print(f"\n>> Resumo :")
print(f"  - Número Total de Registros (Parcelas): {total_registros_formatado}")
print(f"  - Valor Presente Total da Carteira: {valor_presente_formatado}")
print(f"  - Período da Carteira (Data de Aquisição): {df_final2['DataAquisicao'].min().strftime('%d/%m/%Y')} a {df_final2['DataAquisicao'].max().strftime('%d/%m/%Y')}")
print(f"  - Número de Clientes Únicos: {clientes_unicos_formatado}")
print(f"  - Número de Contratos (CCBs) Únicos: {ccbs_unicos_formatado}")

print(f"\n>> Consistência:")
print(f"  - Registros com Duplicidade de CCB (problema se > 0): {df_final2.duplicated(subset='CCB').sum()}")
print(f"  - Registros com Valores Monetários Negativos: {(df_final2[['ValorAquisicao', 'ValorNominal', 'ValorPresente', 'PDDTotal']] < 0).any(axis=1).sum()}")
print(f"  - Registros com VP > Valor Nominal: {(df_final2['ValorPresente'] > df_final2['ValorNominal']).sum()}")
print(f"  - Registros com Data Aquisição > Data Vencimento: {(df_final2['DataAquisicao'] > df_final2['DataVencimento']).sum()}")


print("\n--- Sequencialidade (A questão da parcela 'pendurada') ")
parcelas_por_contrato = df_final2.groupby('CCB').size()
print("\n>> Estatísticas da qttd de parcelas por contrato na base:")
print(parcelas_por_contrato.describe())

analise_sequencia = df_final2.groupby('CCB')['PARCELA'].agg(['min', 'max', 'count']).reset_index()
analise_sequencia['numero_de_gaps'] = (analise_sequencia['max'] - analise_sequencia['min'] + 1) - analise_sequencia['count']
analise_sequencia['tem_gaps'] = analise_sequencia['numero_de_gaps'] > 0
contratos_com_gaps = analise_sequencia['tem_gaps'].sum()
total_contratos = len(analise_sequencia)
print(f"\n>> Gaps:")
print(f"  - Contratos com sequência de parcelas contínua: {total_contratos - contratos_com_gaps:,}".replace(',', '.'))
print(f"  - Contratos com 'gaps' (parcelas faltando na sequência): {contratos_com_gaps:,}".replace(',', '.'))
print(f"  - Quantidade total de parcelas faltantes (soma de todos os gaps): {analise_sequencia['numero_de_gaps'].sum():,}".replace(',', '.'))


print("\n verificação das Duplicatas de Parcela ")
contagem_parcelas_duplicadas = df_final2.groupby(['CCB', 'PARCELA']).size()
frequencia = contagem_parcelas_duplicadas.value_counts().sort_index()
print("\n>> Frequência de repetição de uma mesma parcela dentro de um contrato:")
for count, num_ocorrencias in frequencia.items():
    if count == 1:
        print(f"  - {num_ocorrencias:,} combinações (CCB, Parcela) são únicas.".replace(',', '.'))
    else:
        print(f"  - {num_ocorrencias:,} combinações (CCB, Parcela) aparecem exatamente {count} vezes.".replace(',', '.'))

pares_duplicados_idx = contagem_parcelas_duplicadas[contagem_parcelas_duplicadas > 1].index
if not pares_duplicados_idx.empty:  #>0
    #*""" Contar qts CCBs únicos estão envolvidos na duplicação"""
    contratos_com_duplicatas = pares_duplicados_idx.get_level_values('CCB').nunique()
    print(f"  - Número de contratos únicos que possuem pelo menos uma parcela duplicada: {contratos_com_duplicatas:,}".replace(',', '.'))
    
    df_detalhe_duplicados = df_final2[df_final2.set_index(['CCB', 'PARCELA']).index.isin(pares_duplicados_idx)]
    print("\n>> Intervalo de Tempo entre vencimentos de parcelas duplicadas:")
    gaps_de_vencimento = df_detalhe_duplicados.groupby(['CCB', 'PARCELA'])['DataVencimento'].agg(lambda datas: (datas.max() - datas.min()).days)
    media_anos = gaps_de_vencimento.mean() / 365.25
    mediana_anos = gaps_de_vencimento.median() / 365.25
    print(f"  - O intervalo médio de tempo entre os vencimentos é de aprox {media_anos:.1f} anos.")
    print(f"  - O intervalo mediano é de aprox {mediana_anos:.1f} anos.")
else:
    print("\n>> 0 parcelas duplicada foi encontrada na base.")


print("\n Relação com Status de Vencimento ---")

df_contratos_analise = pd.DataFrame(index=df_final2['CCB'].unique())

df_contratos_analise = df_contratos_analise.join(analise_sequencia.set_index('CCB')[['tem_gaps']])

ccbs_com_duplicatas = pares_duplicados_idx.get_level_values('CCB').unique()
df_contratos_analise['tem_duplicatas'] = df_contratos_analise.index.isin(ccbs_com_duplicatas).astype(int)

status_vencimento = df_final2.groupby('CCB')['_ContratoVencido_Flag'].max().to_frame('tem_vencimento')
df_contratos_analise = df_contratos_analise.join(status_vencimento)


print("\n>> Relç entre GAPS e VENCIMENTO:")
print("iE, lê-se: 'Dos X contratos COM GAPS (tem_gaps=True), Y estão vencidos.'")
crosstab_gaps = pd.crosstab(df_contratos_analise['tem_gaps'].rename('Contratos com Gaps'), df_contratos_analise['tem_vencimento'].rename('Contrato Vencido?'), margins=True, margins_name="Total")
print(crosstab_gaps)

print("\n(Em percentual por linha):")
crosstab_gaps_perc = pd.crosstab(df_contratos_analise['tem_gaps'].rename('Contratos com Gaps'), df_contratos_analise['tem_vencimento'].rename('Contrato Vencido?'), normalize='index') * 100
print(crosstab_gaps_perc.round(2))
print("-" * 60)

print("\n\n>> Relação entre PARCELAS DUPLICADAS e VENCIMENTO:")
print("Lê-se: 'Dos X contratos COM DUPLICATAS (tem_duplicatas=1), Y estão vencidos.'")
crosstab_dups = pd.crosstab(df_contratos_analise['tem_duplicatas'].rename('Contratos com Duplicatas'), df_contratos_analise['tem_vencimento'].rename('Contrato Vencido?'), margins=True, margins_name="Total")
print(crosstab_dups)

print("\n(Em percentual por linha):")
crosstab_dups_perc = pd.crosstab(df_contratos_analise['tem_duplicatas'].rename('Contratos com Duplicatas'), df_contratos_analise['tem_vencimento'].rename('Contrato Vencido?'), normalize='index') * 100
print(crosstab_dups_perc.round(2))


Resumo

>> Resumo :
  - Número Total de Registros (Parcelas): 248.413
  - Valor Presente Total da Carteira: R$ 35.160.953,80
  - Período da Carteira (Data de Aquisição): 23/09/2024 a 05/08/2025
  - Número de Clientes Únicos: 4.614
  - Número de Contratos (CCBs) Únicos: 6.838

>> Consistência:
  - Registros com Duplicidade de CCB (problema se > 0): 241575
  - Registros com Valores Monetários Negativos: 0
  - Registros com VP > Valor Nominal: 0
  - Registros com Data Aquisição > Data Vencimento: 0

--- Sequencialidade (A questão da parcela 'pendurada') 

>> Estatísticas da qttd de parcelas por contrato na base:
count    6838.000000
mean       36.328312
std        29.985794
min         5.000000
25%        18.000000
50%        18.000000
75%        36.000000
max       120.000000
dtype: float64

>> Gaps:
  - Contratos com sequência de parcelas contínua: 6.525
  - Contratos com 'gaps' (parcelas faltando na sequência): 313
  - Quantidade total de parcelas faltantes (soma de todos os gaps): -9

In [14]:
#%%
# =============================================================================
# CÉLULA DE GERAÇÃO DO RELATÓRIO DE DIAGNÓSTICO EM HTML
# =============================================================================
# Descrição: Executa todas as análises de diagnóstico e consolida os
# resultados em um único arquivo HTML, com estilo visual similar ao relatório principal.

import pandas as pd
import base64
import os

# --- FASE 1: CAPTURAR TODOS OS RESULTADOS DA ANÁLISE ---
# Em vez de imprimir, vamos armazenar tudo em um dicionário.

print("FASE 1: Executando análises e capturando resultados...")
diagnostico_results = {}

# --- 1. Resumo e Sanidade Geral ---
resumo = {}
resumo['Número Total de Registros (Parcelas)'] = f"{len(df_final2):,}".replace(',', '.')
resumo['Valor Presente Total da Carteira'] = f"R$ {df_final2['ValorPresente'].sum():,.2f}".replace(',', 'X').replace('.', ',').replace('X', '.')
resumo['Período da Carteira (Data de Aquisição)'] = f"{df_final2['DataAquisicao'].min().strftime('%d/%m/%Y')} a {df_final2['DataAquisicao'].max().strftime('%d/%m/%Y')}"
resumo['Número de Clientes Únicos'] = f"{df_final2['SacadoID'].nunique():,}".replace(',', '.')
resumo['Número de Contratos (CCBs) Únicos'] = f"{df_final2['CCB'].nunique():,}".replace(',', '.')
diagnostico_results['resumo'] = resumo

consistencia = {}
consistencia['Registros com Duplicidade de CCB (problema se > 0)'] = df_final2.duplicated(subset='CCB').sum()
consistencia['Registros com Valores Monetários Negativos'] = (df_final2[['ValorAquisicao', 'ValorNominal', 'ValorPresente', 'PDDTotal']] < 0).any(axis=1).sum()
consistencia['Registros com VP > Valor Nominal'] = (df_final2['ValorPresente'] > df_final2['ValorNominal']).sum()
consistencia['Registros com Data Aquisição > Data Vencimento'] = (df_final2['DataAquisicao'] > df_final2['DataVencimento']).sum()
diagnostico_results['consistencia'] = consistencia

# --- 2. Análise de Sequencialidade ---
sequencialidade = {}
parcelas_por_contrato = df_final2.groupby('CCB').size()
sequencialidade['describe_df'] = parcelas_por_contrato.describe().to_frame().T

analise_sequencia = df_final2.groupby('CCB')['PARCELA'].agg(['min', 'max', 'count']).reset_index()
analise_sequencia['numero_de_gaps'] = (analise_sequencia['max'] - analise_sequencia['min'] + 1) - analise_sequencia['count']
analise_sequencia['tem_gaps'] = analise_sequencia['numero_de_gaps'] > 0
contratos_com_gaps = analise_sequencia['tem_gaps'].sum()
total_contratos = len(analise_sequencia)

gaps = {}
gaps['Contratos com sequência de parcelas contínua'] = f"{total_contratos - contratos_com_gaps:,}".replace(',', '.')
gaps['Contratos com gaps (parcelas faltando na sequência)'] = f"{contratos_com_gaps:,}".replace(',', '.')
gaps['Quantidade total de parcelas faltantes (soma de todos os gaps)'] = f"{analise_sequencia['numero_de_gaps'].sum():,}".replace(',', '.')
sequencialidade['gaps'] = gaps
diagnostico_results['sequencialidade'] = sequencialidade

# --- 3. Análise de Duplicatas ---
duplicatas = {}
contagem_parcelas_duplicadas = df_final2.groupby(['CCB', 'PARCELA']).size()
frequencia = contagem_parcelas_duplicadas.value_counts().sort_index()
frequencia_text = []
for count, num_ocorrencias in frequencia.items():
    if count == 1:
        frequencia_text.append(f"<li>{num_ocorrencias:,} combinações (CCB, Parcela) são únicas.</li>".replace(',', '.'))
    else:
        frequencia_text.append(f"<li>{num_ocorrencias:,} combinações (CCB, Parcela) aparecem exatamente {count} vezes.</li>".replace(',', '.'))
duplicatas['frequencia_list'] = "<ul>" + "".join(frequencia_text) + "</ul>"

pares_duplicados_idx = contagem_parcelas_duplicadas[contagem_parcelas_duplicadas > 1].index
if not pares_duplicados_idx.empty:
    duplicatas['num_contratos_com_duplicatas'] = f"{pares_duplicados_idx.get_level_values('CCB').nunique():,}".replace(',', '.')
    df_detalhe_duplicados = df_final2[df_final2.set_index(['CCB', 'PARCELA']).index.isin(pares_duplicados_idx)]
    gaps_de_vencimento = df_detalhe_duplicados.groupby(['CCB', 'PARCELA'])['DataVencimento'].agg(lambda datas: (datas.max() - datas.min()).days)
    media_anos = gaps_de_vencimento.mean() / 365.25
    mediana_anos = gaps_de_vencimento.median() / 365.25
    duplicatas['intervalo_medio'] = f"{media_anos:.1f} anos"
    duplicatas['intervalo_mediano'] = f"{mediana_anos:.1f} anos"
else:
    duplicatas['num_contratos_com_duplicatas'] = 0
diagnostico_results['duplicatas'] = duplicatas

# --- 4. Relação com Vencimento ---
cruzamentos = {}
status_contrato = pd.DataFrame(index=df_final2['CCB'].unique())
status_contrato = status_contrato.join(analise_sequencia.set_index('CCB')[['tem_gaps']])
ccbs_com_duplicatas = pares_duplicados_idx.get_level_values('CCB').unique()
status_contrato['tem_duplicatas'] = status_contrato.index.isin(ccbs_com_duplicatas).astype(int)
status_vencimento = df_final2.groupby('CCB')['_ContratoVencido_Flag'].max().to_frame('tem_vencimento')
status_contrato = status_contrato.join(status_vencimento)

cruzamentos['crosstab_gaps_df'] = pd.crosstab(status_contrato['tem_gaps'].rename('Contratos com Gaps'), status_contrato['tem_vencimento'].rename('Contrato Vencido?'), margins=True, margins_name="Total")
cruzamentos['crosstab_gaps_perc_df'] = pd.crosstab(status_contrato['tem_gaps'].rename('Contratos com Gaps'), status_contrato['tem_vencimento'].rename('Contrato Vencido?'), normalize='index') * 100
cruzamentos['crosstab_dups_df'] = pd.crosstab(status_contrato['tem_duplicatas'].rename('Contratos com Duplicatas'), status_contrato['tem_vencimento'].rename('Contrato Vencido?'), margins=True, margins_name="Total")
cruzamentos['crosstab_dups_perc_df'] = pd.crosstab(status_contrato['tem_duplicatas'].rename('Contratos com Duplicatas'), status_contrato['tem_vencimento'].rename('Contrato Vencido?'), normalize='index') * 100
diagnostico_results['cruzamentos'] = cruzamentos

print("FASE 1 concluída.")

# --- FASE 2: CONSTRUIR E SALVAR O ARQUIVO HTML ---
print("FASE 2: Construindo o relatório HTML...")

# Reutilizar o CSS e a estrutura do relatório principal
html_css = """
<style>

    .checks-container {
        display: grid;
        grid-template-columns: repeat(auto-fit, minmax(350px, 1fr));
        gap: 15px;
        margin-bottom: 20px;
    }
    .check-item {
        background-color: #f5f5f5;
        padding: 12px;
        border-radius: 5px;
        border-left: 4px solid #76c6c5;
        font-size: 0.95em;
    }
    .check-item strong {
        color: #163f3f;
    }
    .dataframe-checks th {
        background-color: #5b8c8c; /* Um tom mais claro para diferenciar */
    }
    .check-table-wrapper h4 {
        margin-top: 20px;
        margin-bottom: 10px;
        color: #163f3f;
        border-bottom: 2px solid #eeeeee;
        padding-bottom: 5px;
    }

    /* Configurações Gerais */
    body {
        font-family: "Gill Sans MT", Arial, sans-serif;
        background-color: #f9f9f9;
        color: #313131;
        margin: 0;
        padding: 0;
    }
    .main-content {
        padding: 25px;
    }

    /* --- CABEÇALHO --- */
    header {
        background-color: #163f3f;
        color: #FFFFFF;
        padding: 20px 40px;
        display: flex;
        justify-content: space-between;
        align-items: center;
        border-bottom: 5px solid #76c6c5;
    }
    /* 3. LOGO MAIOR: Altura da logo aumentada novamente */
    header .logo img {
        height: 75px; /* Aumentado de 65px para 75px */
    }
    header .report-title {
        text-align: left;
        font-family: "Gill Sans MT", Arial, sans-serif;
    }
    header .report-title h1, header .report-title h2, header .report-title h3 {
        margin: 0;
        padding: 0;
        font-weight: normal;
    }
    /* 2. FONTE MENOR: Tamanho do título principal reduzido */
    header .report-title h1 { font-size: 1.6em; /* Reduzido de 1.8em para 1.6em */ }
    header .report-title h2 { font-size: 1.4em; color: #d0d0d0; }
    header .report-title h3 { font-size: 1.1em; color: #a0a0a0; }

    /* Estilos dos Botões e Tabelas  */
    .container-botoes { display: flex; flex-wrap: wrap; gap: 15px; margin-bottom: 25px; }
    .container-botoes > details { flex: 1 1 280px; border: 1px solid #76c6c5; border-radius: 8px; overflow: hidden; }
    .container-botoes > details[open] { flex-basis: 100%; }
    details summary { font-size: 1.1em; font-weight: bold; color: #FFFFFF; background-color: #163f3f; padding: 15px 20px; cursor: pointer; outline: none; list-style-type: none; }
    details summary:hover { background-color: #0e5d5f; }
    details[open] summary { background-color: #76c6c5; color: #313131; }
    details[open] summary:hover { filter: brightness(95%); }
    summary::-webkit-details-marker { display: none; }
    summary::before { content: '► '; margin-right: 8px; font-size: 0.8em;}
    details[open] summary::before { content: '▼ '; }
    details .content-wrapper { padding: 20px; background-color: #FFFFFF; }
    table.dataframe, th, td { border: 1px solid #bbbbbb; }
    table.dataframe { border-collapse: collapse; width: 100%; }
    th, td { text-align: left; padding: 10px; vertical-align: middle; }
    th { background-color: #163f3f; color: #FFFFFF; }
    tr:nth-child(even) { background-color: #eeeeee; }

    /* --- RODAPÉ --- */
    footer {
        background-color: #f0f0f0;
        color: #555555;
        font-size: 0.8em;
        line-height: 1.6;
        padding: 25px 40px;
        margin-top: 40px;
        border-top: 1px solid #dddddd;
    }
    footer .disclaimer {
        margin-top: 20px;
        font-style: italic;
        border-top: 1px solid #dddddd;
        padding-top: 20px;
    }
</style>
"""

html_parts = ["<!DOCTYPE html><html lang='pt-BR'><head><meta charset='UTF-8'><title>Diagnóstico da Carteira</title>", html_css, "</head><body>"]

# Cabeçalho (pode usar a mesma logo e data)
report_date = ref_date.strftime('%d/%m/%Y')
header_html = f"""
<header>
    <div class="report-title">
        <h1>Diagnóstico da Carteira</h1>
        <h2>Qualidade e Estrutura dos Dados</h2>
        <h3>{report_date}</h3>
    </div>
</header>
"""
html_parts.append(header_html)
html_parts.append("<div class='main-content'>")

# Seção de Resumo
html_parts.append("<details open><summary>Resumo e Sanidade Geral</summary><div class='content-wrapper'>")
html_parts.append("<h4>Resumo da Carteira</h4><div class='checks-container'>")
for key, value in diagnostico_results['resumo'].items():
    html_parts.append(f"<div class='check-item'><strong>{key}:</strong> {value}</div>")
html_parts.append("</div><h4>Verificações de Consistência</h4><div class='checks-container'>")
for key, value in diagnostico_results['consistencia'].items():
    html_parts.append(f"<div class='check-item'><strong>{key}:</strong> {value}</div>")
html_parts.append("</div></div></details>")

# Seção de Sequencialidade
html_parts.append("<details open><summary>Análise de Sequencialidade (Parcela 'Pendurada')</summary><div class='content-wrapper'>")
html_parts.append("<h4>Estatísticas da Quantidade de Parcelas por Contrato</h4>")
html_parts.append(diagnostico_results['sequencialidade']['describe_df'].to_html(classes='dataframe dataframe-checks'))
html_parts.append("<h4>Análise de Gaps</h4><div class='checks-container'>")
for key, value in diagnostico_results['sequencialidade']['gaps'].items():
    html_parts.append(f"<div class='check-item'><strong>{key}:</strong> {value}</div>")
html_parts.append("</div></div></details>")

# Seção de Duplicatas
html_parts.append("<details open><summary>Análise Avançada de Duplicatas de Parcela</summary><div class='content-wrapper'>")
html_parts.append("<h4>Frequência de Repetição</h4>")
html_parts.append(diagnostico_results['duplicatas']['frequencia_list'])
html_parts.append("<h4>Análise das Duplicatas</h4><div class='checks-container'>")
html_parts.append(f"<div class='check-item'><strong>Nº de contratos únicos com parcelas duplicadas:</strong> {diagnostico_results['duplicatas']['num_contratos_com_duplicatas']}</div>")
html_parts.append(f"<div class='check-item'><strong>Intervalo médio entre vencimentos:</strong> {diagnostico_results['duplicatas']['intervalo_medio']}</div>")
html_parts.append(f"<div class='check-item'><strong>Intervalo mediano entre vencimentos:</strong> {diagnostico_results['duplicatas']['intervalo_mediano']}</div>")
html_parts.append("</div></div></details>")

# Seção de Cruzamentos
html_parts.append("<details open><summary>Relação entre Anomalias e Status de Vencimento</summary><div class='content-wrapper'>")
html_parts.append("<div class='check-table-wrapper'><h4>Relação entre GAPS e VENCIMENTO</h4>")
html_parts.append(diagnostico_results['cruzamentos']['crosstab_gaps_df'].to_html(classes='dataframe dataframe-checks'))
html_parts.append("<h5>(Em percentual por linha)</h5>")
html_parts.append(diagnostico_results['cruzamentos']['crosstab_gaps_perc_df'].to_html(classes='dataframe dataframe-checks', float_format='{:.2f}%'.format))
html_parts.append("</div><div class='check-table-wrapper'><h4>Relação entre PARCELAS DUPLICADAS e VENCIMENTO</h4>")
html_parts.append(diagnostico_results['cruzamentos']['crosstab_dups_df'].to_html(classes='dataframe dataframe-checks'))
html_parts.append("<h5>(Em percentual por linha)</h5>")
html_parts.append(diagnostico_results['cruzamentos']['crosstab_dups_perc_df'].to_html(classes='dataframe dataframe-checks', float_format='{:.2f}%'.format))
html_parts.append("</div></div></details>")

html_parts.append("</div></body></html>")

# Salvar o arquivo final
final_html_content = "\n".join(html_parts)
html_output_filename = os.path.join(output_path, 'diagnostico_carteira.html')
try:
    with open(html_output_filename, 'w', encoding='utf-8') as f:
        f.write(final_html_content)
    print(f"\nFASE 2 concluída. Relatório de Diagnóstico salvo em: {html_output_filename}")
except Exception as e:
    print(f"\n[ERRO GRAVE] Não foi possível salvar o arquivo HTML de diagnóstico: {e}")

print("\n" + "="*80)
print("PROCESSO FINALIZADO")
print("="*80)

FASE 1: Executando análises e capturando resultados...
FASE 1 concluída.
FASE 2: Construindo o relatório HTML...

FASE 2 concluída. Relatório de Diagnóstico salvo em: C:\Users\Leo\Desktop\Porto_Real\portoauto\src\originadores\output\diagnostico_carteira.html

PROCESSO FINALIZADO


## Verificação de Vencidos e Performance

#### Verificação igual ao felipe (normaliza com valor máximo)

In [27]:
#%%
import pandas as pd
import numpy as np
import os
from datetime import datetime, date
from dateutil.relativedelta import relativedelta

##! obs: execute as células do nb para que "df_final2" esteja disponível


#! PATH DE SAÍDA (diretório)
output_path = r'C:\Users\Leo\Desktop\Porto_Real\portoauto\src\originadores\output'
#FIXME


df_report = df_final2.copy()
df_report['_total_carteira'] = 'Carteira Consolidada'
df_report['MesVencimento'] = pd.to_datetime(df_report['DataVencimento']).dt.to_period('M')
#```coluna geral da carteira```
""" versão super simples
segmentos_para_analise = {                           # defne os segmnt que querms analsr
    'Carteira Consolidada': '_total_carteira',
    'Originadores': 'Originador',
    'Produtos': 'Produto',
    'Convênios': 'Convênio',
    'UF': 'UF',
    'CAPAG': 'CAPAG'
}
"""

segmentos_para_analise = {
    'Carteira Consolidada': '_total_carteira',
    'Cedentes': 'Cedente',
    'Originadores': 'Originador',
    'Promotoras': 'Promotora',
    'Produtos': 'Produto',
    'Convênios': 'Convênio',
    'Situação': 'Situacao',
    'UF': 'UF',
    'CAPAG': 'CAPAG',
    'Pagamento Parcial': 'PagamentoParcial',
    'Tem Muitos Contratos':'_MuitosContratos',
    'Tem Muitos Entes':'_MuitosEntes'
}



ref_date_obj = df_report['DataGeracao'].max().date()
"""data de referencia"""

""" AC
if 'DataGeracao' in df_report.columns and not df_report['DataGeracao'].isna().all(): # to lendo a data de refeência dos dados
    ref_date_obj = df_report['DataGeracao'].max().date()
else:
    ref_date_obj = date.today()

"""
os.makedirs(output_path, exist_ok=True) #* --------


def convert_period_to_relative_month(period: pd.Period, ref_date: date) -> str: #?fc aux
    """uso aqueles meses com M - x"""
    venc_month = period.to_timestamp().to_pydatetime()
    ref_month = datetime(ref_date.year, ref_date.month, 1)
    
    delta = relativedelta(venc_month, ref_month)
    months_diff = delta.years * 12 + delta.months
    
    return f"M{months_diff:+}"

# FUNCAO PRINCIPAL: GERACAO DA TABELA

def generate_performance_heatmap(df_input: pd.DataFrame, segment_column: str, segment_friendly_name: str, ref_date: date) -> str:
    """
    vai retornar um html de tabela de cada uma das dimensoes que tamos analisando, para colcoar no relatório
    """
    print(f"---> Processando segmento: {segment_friendly_name}...")
    
    # pulo se só tiver uma única entrada
    unique_entries = df_input[segment_column].dropna().unique()
    if len(unique_entries) == 1 and segment_column != '_total_carteira':
        entry_name = unique_entries[0]
        print(f"     [INFO] Segmento '{segment_friendly_name}' tem apenas uma entrada: '{entry_name}'. Gerando mensagem simples.")
        message_html = f"""
        <div class="table-container">
            <h3 class="table-title">Análise por: {segment_friendly_name}</h3>
            <p class="single-entry-message">Em '{segment_friendly_name}' temos apenas uma entrada: <strong>{entry_name}</strong>. A análise detalhada deste item é refletida na tabela 'Carteira Consolidada'.</p>
        </div>
        """
        return message_html

    report_abs = df_input.pivot_table(
        index=segment_column,
        columns="MesVencimento",
        values="ValorNominal", # tab din
        fill_value=0,
        aggfunc='sum'
    )
    if report_abs.empty: return ""

    peak_value = report_abs.max().max() # isso aqui vai ser o "pico", que vou usar para normalizar
    print("valor de report_abs.max():")
    print(report_abs.max())

    print("valor de report_abs.max().max():")
    print(peak_value)

    if peak_value == 0: return ""
    print(f"     Pico de Valor Nominal para este segmento: R$ {peak_value:,.2f}")

    
    report_performance = (1 - (report_abs / peak_value)) * 100
    """# calclo da perfrm"""
    report_performance["Total Nominal (R$)"] = report_abs.sum(axis=1)
    report_performance.sort_values(by="Total Nominal (R$)", ascending=False, inplace=True)
    report_performance.index.name = segment_friendly_name
    report_performance.reset_index(inplace=True)

    
    period_cols = [c for c in report_performance.columns if isinstance(c, pd.Period)]
    period_to_relative_map = {p: convert_period_to_relative_month(p, ref_date) for p in period_cols}
    relative_to_html_map = {
        relative: f"{relative}<br><span class='sub-header'>{period.strftime('%Y-%m')}</span>"
        for period, relative in period_to_relative_map.items()
    }
    report_performance.rename(columns=period_to_relative_map, inplace=True) # acm: logc rnmco
    
    relative_month_cols = [c for c in report_performance.columns if isinstance(c, str) and c.startswith('M')]
    past_months = [c for c in relative_month_cols if int(c.replace('M','').replace('+','')) <= 0]
    
    sort_key = lambda col: int(col.replace('M', '').replace('+', ''))
    sorted_relative_cols = sorted(past_months, key=sort_key)
    
    final_cols_order = [segment_friendly_name] + sorted_relative_cols + ["Total Nominal (R$)"]
    report_final = report_performance[final_cols_order]

    report_final.rename(columns=relative_to_html_map, inplace=True)
    sorted_html_cols = [relative_to_html_map.get(c, c) for c in sorted_relative_cols]

    def format_br_percent(val): ## parte trazida de outros codes: estilizacao do html
        if pd.isna(val): return ""
        return f"{val:.2f}%".replace('.', ',')
        
    def format_br_number(val):
        if pd.isna(val): return ""
        return f"{val:,.2f}".replace(',', 'X').replace('.', ',').replace('X', '.')

    styler = (
        report_final.style
        .set_table_attributes('class="dataframe"')
        .set_properties(**{"padding": "8px", "text-align": "right", "min-width": "80px"})
        .set_table_styles([
            {"selector": "th, td", "props": [("border", "1px solid #ccc")]},
            {"selector": "th", "props": [("background-color", "#163f3f"), ("color", "#FFFFFF"), ("text-align", "center"), ("font-weight", "bold"), ("line-height", "1.3")]},
            {"selector": "thead th", "props": [("position", "sticky"), ("top", "0"), ("z-index", "1")]},
            {"selector": "tbody td:first-child", "props": [("position", "sticky"), ("left", "0"), ("background-color", "#f9f9f9"), ("text-align", "left"), ("font-weight", "bold"), ("white-space", "nowrap")]},
            {"selector": "thead th:first-child", "props": [("position", "sticky"), ("left", "0"), ("z-index", "2")]}
        ], overwrite=False)
        .format({col: format_br_percent for col in sorted_html_cols} | {"Total Nominal (R$)": format_br_number})
        .background_gradient(cmap="RdYlGn", subset=sorted_html_cols, vmin=0, vmax=100, axis=None)
        .background_gradient(cmap="Greys", subset=["Total Nominal (R$)"], axis=None)
        .hide(axis="index")
    )
    
    header_tabela = f"""
    <div class="table-container">
        <h3 class="table-title">Performance de Vencimentos por: {segment_friendly_name}</h3>
        {styler.to_html()}
    </div>
    """
    return header_tabela


#  int main(){ 

print("\n" + "="*80)
print("INICIANDO A GERAÇÃO DO RELATÓRIO DE PERFORMANCE DE VENCIMENTOS")
print(f"Data de referência para os dados: {ref_date_obj.strftime('%d/%m/%Y')}")
print("="*80)

html_tabelas = []
for nome_amigavel, nome_coluna in segmentos_para_analise.items():
    tabela_html = generate_performance_heatmap(df_report, nome_coluna, nome_amigavel, ref_date_obj)
    if tabela_html:
        html_tabelas.append(tabela_html)

# html final

if html_tabelas:
    html_css = """
    <style>
        body { font-family: "Segoe UI", Arial, sans-serif; background-color: #f4f4f4; margin: 0; padding: 20px; }
        .main-container { max-width: 95%; margin: auto; background-color: #ffffff; padding: 25px; border-radius: 8px; box-shadow: 0 4px 8px rgba(0,0,0,0.1); }
        .report-header { text-align: center; border-bottom: 2px solid #163f3f; padding-bottom: 15px; margin-bottom: 30px; }
        .report-header h1 { color: #163f3f; margin: 0; }
        .report-header p { color: #555; font-size: 1.1em; }
        .table-container { margin-bottom: 50px; }
        .table-title { color: #0e5d5f; border-bottom: 2px solid #76c6c5; padding-bottom: 8px; margin-bottom: 15px; }
        .dataframe { border-collapse: collapse; border-spacing: 0; width: 100%; font-size: 0.9em; }
        .dataframe td.na { background-color: transparent !important; }
        .single-entry-message {
            padding: 20px;
            background-color: #f8f9fa;
            border-left: 5px solid #76c6c5;
            margin-top: 15px;
            font-size: 1.1em;
            color: #333;
        }
        .sub-header {
            font-size: 0.8em;
            font-weight: normal;
            color: #dddddd;
        }
        /* estilop q e sempre uso no rodpé */
        footer {
            margin-top: 40px;
            padding-top: 20px;
            border-top: 1px solid #ddd;
            font-size: 0.9em;
            color: #777;
            text-align: center;
        }
    </style>
    """
    
    # resolvi deixano rodapé pq eu pensei em fazer outra forma, mas adaptei o do Felipe
    footer_text = """
    <p><strong>Nota sobre a Metodologia:</strong> Este relatório utiliza a lógica de análise de performance baseada no "mês de pico". 
    Ele calcula o volume nominal de cada mês como um percentual do mês de maior volume nominal da história para aquele segmento, usando a visão de "aging".</p>
    """

    html_final = f"""
    <!DOCTYPE html>
    <html lang="pt-BR">
    <head>
        <meta charset="UTF-8">
        <title>Relatório de Performance de Vencimentos (Aging)</title>
        {html_css}
    </head>
    <body>
        <div class="main-container">
            <div class="report-header">
                <h1>Relatório de Performance de Vencimentos</h1>
                <p>Data de referência dos dados: {ref_date_obj.strftime('%d/%m/%Y')}</p>
            </div>
            {''.join(html_tabelas)}
            
            <footer>
                {footer_text}
            </footer>
        </div>
    </body>
    </html>
    """

    nome_arquivo_saida = f"{date.today().strftime('%Y-%m-%d')}-relatorio_performance_vencimentos_aging.html"
    caminho_arquivo_saida = os.path.join(output_path, nome_arquivo_saida)
    


    with open(caminho_arquivo_saida, "w", encoding="utf-8") as f:
        f.write(html_final)
    print("\n" + "="*80)
    print("PROCESSO FINALIZADO COM SUCESSO!")
    print(f"Relatório de Performance (Aging) gerado em: {caminho_arquivo_saida}")
    print("="*80)

    """versão AC
    try:
        with open(caminho_arquivo_saida, "w", encoding="utf-8") as f:
            f.write(html_final)
        print("\n" + "="*80)
        print("PROCESSO FINALIZADO COM SUCESSO!")
        print(f"Relatório de Performance (Aging) gerado em: {caminho_arquivo_saida}")
        print("="*80)
    except Exception as e:
        print(f"\n[ERRO GRAVE] Não foi possível salvar o arquivo HTML: {e}")
else:
    print("\nNenhuma tabela foi gerada. O relatório final está vazio.")

"""


INICIANDO A GERAÇÃO DO RELATÓRIO DE PERFORMANCE DE VENCIMENTOS
Data de referência para os dados: 05/08/2025
---> Processando segmento: Carteira Consolidada...
valor de report_abs.max():
MesVencimento
2024-10        322.04
2024-11       2311.10
2024-12      11852.93
2025-01      65762.92
2025-02     150881.63
2025-03     184647.78
2025-04     479440.34
2025-05     754163.82
2025-06    1127829.35
2025-07    1336298.43
2025-08    1608560.94
2025-09    1678016.74
2025-10    1678416.74
2025-11    1678416.74
2025-12    1678065.63
2026-01    1677932.86
2026-02    1675842.38
2026-03    1675545.60
2026-04    1670950.85
2026-05    1665214.60
2026-06    1658951.55
2026-07    1622116.71
2026-08    1559265.06
2026-09    1471420.54
2026-10    1321892.19
2026-11    1144353.33
2026-12     885524.03
2027-01     788020.33
2027-02     749196.40
2027-03     744580.38
2027-04     730657.30
2027-05     706199.67
2027-06     688911.17
2027-07     661289.36
2027-08     596007.83
2027-09     545094.60
2027-10

C:\Users\Leo\AppData\Local\Temp\ipykernel_31472\3112482115.py:137: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  report_final.rename(columns=relative_to_html_map, inplace=True)


---> Processando segmento: Cedentes...
     [INFO] Segmento 'Cedentes' tem apenas uma entrada: 'BMP MONEY PLUS SOCIEDADE DE CREDITO DIRETO S.A'. Gerando mensagem simples.
---> Processando segmento: Originadores...
     [INFO] Segmento 'Originadores' tem apenas uma entrada: 'StarCard'. Gerando mensagem simples.
---> Processando segmento: Promotoras...
valor de report_abs.max():
MesVencimento
2024-10       322.04
2024-11      2311.10
2024-12      5406.15
2025-01     15909.20
2025-02     47886.07
2025-03     46509.74
2025-04    139492.54
2025-05    139696.70
2025-06    147350.12
2025-07    185072.86
2025-08    237836.07
2025-09    257810.27
2025-10    258210.27
2025-11    258210.27
2025-12    258151.38
2026-01    258151.38
2026-02    258151.38
2026-03    258022.60
2026-04    258022.60
2026-05    257753.64
2026-06    257249.35
2026-07    257100.24
2026-08    256722.49
2026-09    254280.82
2026-10    253956.06
2026-11    252796.87
2026-12    250671.08
2027-01    250627.04
2027-02    250627.

C:\Users\Leo\AppData\Local\Temp\ipykernel_31472\3112482115.py:137: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  report_final.rename(columns=relative_to_html_map, inplace=True)


valor de report_abs.max():
MesVencimento
2024-10       241.90
2024-11      1366.68
2024-12      4345.14
2025-01     18013.91
2025-02     48222.12
2025-03     75123.33
2025-04    123197.22
2025-05    212945.83
2025-06    333768.46
2025-07    437023.78
2025-08    454096.17
2025-09    469550.56
2025-10    469550.56
2025-11    469550.56
2025-12    469550.56
2026-01    469550.56
2026-02    468973.87
2026-03    468973.87
2026-04    468286.78
2026-05    463934.38
2026-06    461554.14
2026-07    449604.46
2026-08    441623.83
2026-09    416320.91
2026-10    364130.70
2026-11    275658.74
2026-12    275658.74
2027-01    275222.23
2027-02    275222.23
2027-03    273852.84
2027-04    270246.20
2027-05    265302.60
2027-06    260503.12
2027-07    258903.24
2027-08    255958.81
2027-09    251771.90
2027-10    250915.09
2027-11    248072.89
2027-12    240521.19
2028-01    228782.36
2028-02    219511.40
2028-03    214946.91
2028-04    210934.34
2028-05    206971.37
2028-06    204813.93
2028-07    202

...

C:\Users\Leo\AppData\Local\Temp\ipykernel_31472\3112482115.py:137: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  report_final.rename(columns=relative_to_html_map, inplace=True)
C:\Users\Leo\AppData\Local\Temp\ipykernel_31472\3112482115.py:137: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  report_final.rename(columns=relative_to_html_map, inplace=True)


valor de report_abs.max():
MesVencimento
2024-10        322.04
2024-11       2267.17
2024-12       5985.61
2025-01      41826.85
2025-02     113374.09
2025-03     131907.78
2025-04     368894.63
2025-05     558551.40
2025-06     739720.47
2025-07     898926.80
2025-08    1096859.28
2025-09    1097067.85
2025-10    1097067.85
2025-11    1097067.85
2025-12    1096716.74
2026-01    1096583.97
2026-02    1094493.49
2026-03    1094325.49
2026-04    1089730.74
2026-05    1083994.49
2026-06    1077731.44
2026-07    1040896.60
2026-08     978044.95
2026-09     890200.43
2026-10     740672.08
2026-11     563133.22
2026-12     415309.03
2027-01     414872.52
2027-02     414472.77
2027-03     413341.10
2027-04     406990.50
2027-05     397084.14
2027-06     383026.71
2027-07     376550.43
2027-08     369723.93
2027-09     354465.29
2027-10     352646.91
2027-11     347776.30
2027-12     338350.63
2028-01     323180.29
2028-02     310651.69
2028-03     304456.75
2028-04     296938.79
2028-05     2

C:\Users\Leo\AppData\Local\Temp\ipykernel_31472\3112482115.py:137: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  report_final.rename(columns=relative_to_html_map, inplace=True)
C:\Users\Leo\AppData\Local\Temp\ipykernel_31472\3112482115.py:137: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  report_final.rename(columns=relative_to_html_map, inplace=True)


---> Processando segmento: Pagamento Parcial...
valor de report_abs.max():
MesVencimento
2024-10        241.90
2024-11       2237.19
2024-12      11615.89
2025-01      64135.07
2025-02     145332.18
2025-03     175765.33
2025-04     469431.22
2025-05     738752.98
2025-06    1127829.35
2025-07    1336298.43
2025-08    1608560.94
2025-09    1678016.74
2025-10    1678416.74
2025-11    1678416.74
2025-12    1678065.63
2026-01    1677932.86
2026-02    1675842.38
2026-03    1675545.60
2026-04    1670950.85
2026-05    1665214.60
2026-06    1658951.55
2026-07    1622116.71
2026-08    1559265.06
2026-09    1471420.54
2026-10    1321892.19
2026-11    1144353.33
2026-12     885524.03
2027-01     788020.33
2027-02     749196.40
2027-03     744580.38
2027-04     730657.30
2027-05     706199.67
2027-06     688911.17
2027-07     661289.36
2027-08     596007.83
2027-09     545094.60
2027-10     543276.22
2027-11     538405.61
2027-12     528124.60
2028-01     511126.50
2028-02     495192.17
2028-03  

C:\Users\Leo\AppData\Local\Temp\ipykernel_31472\3112482115.py:137: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  report_final.rename(columns=relative_to_html_map, inplace=True)


In [29]:
#%%
# =============================================================================
# CÉLULA DE GERAÇÃO DO RELATÓRIO DE PERFORMANCE DE VENCIMENTOS (AGING)
# =============================================================================
# Descrição: Este script utiliza a lógica de análise de performance baseada no
# "mês de pico" de CADA LINHA. Ele calcula o volume nominal de cada mês como
# um percentual do mês de maior volume da história para aquele item específico.

import pandas as pd
import numpy as np
import os
from datetime import datetime, date
from dateutil.relativedelta import relativedelta


if 'df_final2' not in locals() or df_final2.empty:
    raise NameError("Você esqueceu de executar as céulas anteriores para definir o df_final2")


#! ==============================================================================
output_path = r'C:\Users\Leo\Desktop\Porto_Real\portoauto\src\originadores\output'
"""dir de saída"""
os.makedirs(output_path, exist_ok=True) 
#FIXME
#! ==============================================================================

df_report = df_final2.copy()
df_report['_total_carteira'] = 'Carteira Consolidada'
# ```coluna para permitir a análise da carteira total```
df_report['MesVencimento'] = pd.to_datetime(df_report['DataVencimento']).dt.to_period('M')


segmentos_para_analise = {
    'Carteira Consolidada': '_total_carteira',
    'Cedentes': 'Cedente',
    'Originadores': 'Originador',
    'Promotoras': 'Promotora',
    'Produtos': 'Produto',
    'Convênios': 'Convênio',
    'Situação': 'Situacao',
    'UF': 'UF',
    'CAPAG': 'CAPAG',
    'Pagamento Parcial': 'PagamentoParcial',
    'Tem Muitos Contratos':'_MuitosContratos',
    'Tem Muitos Entes':'_MuitosEntes'
} 
""" os segmentos que queremos analisar (variáveis categóricas)"""


ref_date_obj = df_report['DataGeracao'].max().date()
"""Define a data de referência a partir da tabela"""
# recomendação: coloque isso ou a data de hoje, para ficar mais robusto


#*=================================================================================
def convert_period_to_relative_month(period: pd.Period, ref_date: date) -> str:
    """converte  para um formato de mês relativo (eg M-1, M+0)."""
    venc_month = period.to_timestamp().to_pydatetime()
    ref_month = datetime(ref_date.year, ref_date.month, 1)
    delta = relativedelta(venc_month, ref_month)
    months_diff = delta.years * 12 + delta.months
    return f"M{months_diff:+}"

#*=================================================================================================================
def generate_performance_heatmap(df_input: pd.DataFrame, segment_column: str, segment_friendly_name: str, ref_date: date) -> str:
    """
    tabela HTML de performance de vencimentos para um segmento.
    """
    print(f"---> Processando segmento: {segment_friendly_name}...")

    # Lógica para pular segmentos com apenas uma entrada
    unique_entries = df_input[segment_column].dropna().unique()
    if len(unique_entries) == 1 and segment_column != '_total_carteira':
        entry_name = unique_entries[0]
        print(f"     [INFO] Segmento '{segment_friendly_name}' tem apenas uma entrada: '{entry_name}'. Gerando mensagem simples.")
        message_html = f"""
        <div class="table-container">
            <h3 class="table-title">Análise por: {segment_friendly_name}</h3>
            <p class="single-entry-message">Em '{segment_friendly_name}' temos apenas uma entrada: <strong>{entry_name}</strong>. A análise detalhada deste item é refletida na tabela 'Carteira Consolidada'.</p>
        </div>
        """
        return message_html

    report_abs = df_input.pivot_table(
        index=segment_column,
        columns="MesVencimento",
        values="ValorNominal",
        fill_value=0,
        aggfunc='sum'
    )
    """tabela dinâmica de valores absolutos AC"""

    if report_abs.empty: return ""

    #! CHANGELOG -------------------------------------------------------------------------
    # Em vez de um pico global da tabela inteira, calculo o pico para cada item do índice.
    # peak_value = report_abs.max().max() #! REMOVIDO
    row_peaks = report_abs.max(axis=1) #?  valor máximo de cada linha
    row_peaks[row_peaks == 0] = np.nan # para não dar divisão por zero
    #!------------------------------------------------------------------------------------
    # VP e VL
    summary_totals = df_input.groupby(segment_column)[['_ValorLiquido', 'ValorPresente']].sum()
    summary_totals.rename(columns={'_ValorLiquido': 'Valor Líquido Total (R$)', 'ValorPresente': 'Valor Presente Total (R$)'}, inplace=True)

    #! CHANGELOG -------------------------------------------------------------------------
    # .div com axis=0 garante que cada linha de report_abs
    #  seja dividida pelo seu pico correspondente em row_peaks.
    report_performance = (1 - report_abs.div(row_peaks, axis=0)) * 100
    report_performance["Total Nominal (R$)"] = report_abs.sum(axis=1)
    #!------------------------------------------------------------------------------------
    
    report_performance = report_performance.join(summary_totals)

    report_performance.sort_values(by="Total Nominal (R$)", ascending=False, inplace=True)
    report_performance.index.name = segment_friendly_name
    report_performance.reset_index(inplace=True)

    # renomear cols:
    period_cols = [c for c in report_performance.columns if isinstance(c, pd.Period)]
    period_to_relative_map = {p: convert_period_to_relative_month(p, ref_date) for p in period_cols}
    relative_to_html_map = {
        relative: f"{relative}<br><span class='sub-header'>{period.strftime('%Y-%m')}</span>"
        for period, relative in period_to_relative_map.items()
    }
    report_performance.rename(columns=period_to_relative_map, inplace=True)

    relative_month_cols = [c for c in report_performance.columns if isinstance(c, str) and c.startswith('M')]

    # (m+1)
    months_to_show = [c for c in relative_month_cols if int(c.replace('M','').replace('+','')) <= 1]

    sort_key = lambda col: int(col.replace('M', '').replace('+', ''))
    # Inverti a ordem das colunas aqui
    sorted_relative_cols = sorted(months_to_show, key=sort_key, reverse=True)

    # cols VP e VL no início da ordem
    final_cols_order = [
        segment_friendly_name,
        'Valor Presente Total (R$)',
        'Valor Líquido Total (R$)'
    ] + sorted_relative_cols + ["Total Nominal (R$)"]

    #AC: (garantir que as colunas certas sao colcadas, desde que existam)
    final_cols_order = [col for col in final_cols_order if col in report_performance.columns]
    report_final = report_performance[final_cols_order]

    report_final.rename(columns=relative_to_html_map, inplace=True)
    sorted_html_cols = [relative_to_html_map.get(c, c) for c in sorted_relative_cols]

    #? Styles
    def format_br_percent(val):
        if pd.isna(val): return ""
        return f"{val:.2f}%".replace('.', ',')

    def format_br_number(val):
        if pd.isna(val): return ""
        return f"{val:,.2f}".replace(',', 'X').replace('.', ',').replace('X', '.')

    styler = (
        report_final.style
        .set_table_attributes('class="dataframe"')
        .set_properties(**{"padding": "8px", "text-align": "right", "min-width": "80px"})
        .set_table_styles([
            {"selector": "th, td", "props": [("border", "1px solid #ccc")]},
            {"selector": "th", "props": [("background-color", "#163f3f"), ("color", "#FFFFFF"), ("text-align", "center"), ("font-weight", "bold"), ("line-height", "1.3")]},
            {"selector": "thead th", "props": [("position", "sticky"), ("top", "0"), ("z-index", "1")]},
            {"selector": "tbody td:first-child", "props": [("position", "sticky"), ("left", "0"), ("background-color", "#f9f9f9"), ("text-align", "left"), ("font-weight", "bold"), ("white-space", "nowrap")]},
            {"selector": "thead th:first-child", "props": [("position", "sticky"), ("left", "0"), ("z-index", "2")]}
        ], overwrite=False)
        .format({
            **{col: format_br_percent for col in sorted_html_cols},
            **{"Total Nominal (R$)": format_br_number,
               'Valor Presente Total (R$)': format_br_number,
               'Valor Líquido Total (R$)': format_br_number}
        })
        .background_gradient(cmap="RdYlGn", subset=sorted_html_cols, vmin=0, vmax=100, axis=None)
        .background_gradient(cmap="Greys", subset=["Total Nominal (R$)", 'Valor Presente Total (R$)', 'Valor Líquido Total (R$)'], axis=None)
        .hide(axis="index")
    )

    header_tabela = f"""
    <div class="table-container">
        <h3 class="table-title">Performance de Vencimentos por: {segment_friendly_name}</h3>
        {styler.to_html()}
    </div>
    """
    return header_tabela


#main:

print("\n" + "="*80)
print("INICIANDO A GERAÇÃO DO RELATÓRIO DE PERFORMANCE DE VENCIMENTOS")
print(f"Data de referência para os dados: {ref_date_obj.strftime('%d/%m/%Y')}")
print("="*80)

html_tabelas = []
for nome_amigavel, nome_coluna in segmentos_para_analise.items():
    tabela_html = generate_performance_heatmap(df_report, nome_coluna, nome_amigavel, ref_date_obj)
    if tabela_html:
        html_tabelas.append(tabela_html)

#? html final 

if html_tabelas:
    html_css = """
    <style>
        body { font-family: "Segoe UI", Arial, sans-serif; background-color: #f4f4f4; margin: 0; padding: 20px; }
        .main-container { max-width: 95%; margin: auto; background-color: #ffffff; padding: 25px; border-radius: 8px; box-shadow: 0 4px 8px rgba(0,0,0,0.1); }
        .report-header { text-align: center; border-bottom: 2px solid #163f3f; padding-bottom: 15px; margin-bottom: 30px; }
        .report-header h1 { color: #163f3f; margin: 0; }
        .report-header p { color: #555; font-size: 1.1em; }
        .table-container { margin-bottom: 50px; }
        .table-title { color: #0e5d5f; border-bottom: 2px solid #76c6c5; padding-bottom: 8px; margin-bottom: 15px; }
        .dataframe { border-collapse: collapse; border-spacing: 0; width: 100%; font-size: 0.9em; }
        .dataframe td.na { background-color: transparent !important; }
        .single-entry-message {
            padding: 20px;
            background-color: #f8f9fa;
            border-left: 5px solid #76c6c5;
            margin-top: 15px;
            font-size: 1.1em;
            color: #333;
        }
        .sub-header {
            font-size: 0.8em;
            font-weight: normal;
            color: #dddddd;
        }
        footer {
            margin-top: 40px;
            padding-top: 20px;
            border-top: 1px solid #ddd;
            font-size: 0.9em;
            color: #777;
            text-align: center;
        }
    </style>
    """

    # AC: nova metodologia abaixo
    footer_text = """
    <p><strong>Nota sobre a Metodologia:</strong> Este relatório analisa a performance de cada item (ex: cada Originador) de forma individual.
    Para cada linha da tabela, o relatório identifica o mês de maior 'Valor Nominal' ('mês de pico' daquela linha) e o usa como referência (100%).
    Os demais meses da mesma linha são então mostrados como um percentual desse pico pessoal. Isso permite avaliar a consistência e a tendência de cada item em relação ao seu próprio melhor desempenho histórico.</p>
    """

    html_final = f"""
    <!DOCTYPE html>
    <html lang="pt-BR">
    <head>
        <meta charset="UTF-8">
        <title>Relatório de Performance de Vencimentos (Aging)</title>
        {html_css}
    </head>
    <body>
        <div class="main-container">
            <div class="report-header">
                <h1>Relatório de Performance de Vencimentos</h1>
                <p>Data de referência dos dados: {ref_date_obj.strftime('%d/%m/%Y')}</p>
            </div>
            {''.join(html_tabelas)}

            <footer>
                {footer_text}
            </footer>
        </div>
    </body>
    </html>
    """

    nome_arquivo_saida = f"{date.today().strftime('%Y-%m-%d')}-relatorio_performance_vencimentos_aging.html"
    caminho_arquivo_saida = os.path.join(output_path, nome_arquivo_saida)

    with open(caminho_arquivo_saida, "w", encoding="utf-8") as f:
        f.write(html_final)
    print("\n" + "="*80)
    print("PROCESSO FINALIZADO COM SUCESSO!")
    print(f"Relatório de Performance (Aging) gerado em: {caminho_arquivo_saida}")
    print("="*80)

else:
    print("\nNenhuma tabela foi gerada. O relatório final está vazio.")


INICIANDO A GERAÇÃO DO RELATÓRIO DE PERFORMANCE DE VENCIMENTOS
Data de referência para os dados: 05/08/2025
---> Processando segmento: Carteira Consolidada...


C:\Users\Leo\AppData\Local\Temp\ipykernel_31472\768391313.py:147: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  report_final.rename(columns=relative_to_html_map, inplace=True)


---> Processando segmento: Cedentes...
     [INFO] Segmento 'Cedentes' tem apenas uma entrada: 'BMP MONEY PLUS SOCIEDADE DE CREDITO DIRETO S.A'. Gerando mensagem simples.
---> Processando segmento: Originadores...
     [INFO] Segmento 'Originadores' tem apenas uma entrada: 'StarCard'. Gerando mensagem simples.
---> Processando segmento: Promotoras...


C:\Users\Leo\AppData\Local\Temp\ipykernel_31472\768391313.py:147: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  report_final.rename(columns=relative_to_html_map, inplace=True)


---> Processando segmento: Produtos...


...

C:\Users\Leo\AppData\Local\Temp\ipykernel_31472\768391313.py:147: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  report_final.rename(columns=relative_to_html_map, inplace=True)
C:\Users\Leo\AppData\Local\Temp\ipykernel_31472\768391313.py:147: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  report_final.rename(columns=relative_to_html_map, inplace=True)


---> Processando segmento: CAPAG...
---> Processando segmento: Pagamento Parcial...


C:\Users\Leo\AppData\Local\Temp\ipykernel_31472\768391313.py:147: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  report_final.rename(columns=relative_to_html_map, inplace=True)
C:\Users\Leo\AppData\Local\Temp\ipykernel_31472\768391313.py:147: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  report_final.rename(columns=relative_to_html_map, inplace=True)


---> Processando segmento: Tem Muitos Contratos...
---> Processando segmento: Tem Muitos Entes...
     [INFO] Segmento 'Tem Muitos Entes' tem apenas uma entrada: 'False'. Gerando mensagem simples.

PROCESSO FINALIZADO COM SUCESSO!
Relatório de Performance (Aging) gerado em: C:\Users\Leo\Desktop\Porto_Real\portoauto\src\originadores\output\2025-08-12-relatorio_performance_vencimentos_aging.html


C:\Users\Leo\AppData\Local\Temp\ipykernel_31472\768391313.py:147: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  report_final.rename(columns=relative_to_html_map, inplace=True)


In [30]:
#%%
# =============================================================================
# CÉLULA DE GERAÇÃO DO RELATÓRIO DE PERFORMANCE DE VENCIMENTOS (AGING)
# =============================================================================
# Descrição: Este script utiliza a lógica de análise de performance baseada no
# "mês de pico" de CADA LINHA. Ele calcula o volume nominal de cada mês como
# um percentual do mês de maior volume da história para aquele item específico.

import pandas as pd
import numpy as np
import os
from datetime import datetime, date
from dateutil.relativedelta import relativedelta


if 'df_final2' not in locals() or df_final2.empty:
    raise NameError("Você esqueceu de executar as céulas anteriores para definir o df_final2")


#! ==============================================================================
output_path = r'C:\Users\Leo\Desktop\Porto_Real\portoauto\src\originadores\output'
"""dir de saída"""
os.makedirs(output_path, exist_ok=True)
#FIXME
#! ==============================================================================

df_report = df_final2.copy()
df_report['_total_carteira'] = 'Carteira Consolidada'
# ```coluna para permitir a análise da carteira total```
df_report['MesVencimento'] = pd.to_datetime(df_report['DataVencimento']).dt.to_period('M')


segmentos_para_analise = {
    'Carteira Consolidada': '_total_carteira',
    'Cedentes': 'Cedente',
    'Originadores': 'Originador',
    'Promotoras': 'Promotora',
    'Produtos': 'Produto',
    'Convênios': 'Convênio',
    'Situação': 'Situacao',
    'UF': 'UF',
    'CAPAG': 'CAPAG',
    'Pagamento Parcial': 'PagamentoParcial',
    'Tem Muitos Contratos':'_MuitosContratos',
    'Tem Muitos Entes':'_MuitosEntes'
}
""" os segmentos que queremos analisar (variáveis categóricas)"""


ref_date_obj = df_report['DataGeracao'].max().date()
"""Define a data de referência a partir da tabela"""
# recomendação: coloque isso ou a data de hoje, para ficar mais robusto


#*=================================================================================
def convert_period_to_relative_month(period: pd.Period, ref_date: date) -> str:
    """converte  para um formato de mês relativo (eg M-1, M+0)."""
    venc_month = period.to_timestamp().to_pydatetime()
    ref_month = datetime(ref_date.year, ref_date.month, 1)
    delta = relativedelta(venc_month, ref_month)
    months_diff = delta.years * 12 + delta.months
    return f"M{months_diff:+}"

#*=================================================================================================================
def generate_performance_heatmap(df_input: pd.DataFrame, segment_column: str, segment_friendly_name: str, ref_date: date) -> str:
    """
    tabela HTML de performance de vencimentos para um segmento.
    """
    print(f"---> Processando segmento: {segment_friendly_name}...")

    # Lógica para pular segmentos com apenas uma entrada
    unique_entries = df_input[segment_column].dropna().unique()
    if len(unique_entries) == 1 and segment_column != '_total_carteira':
        entry_name = unique_entries[0]
        print(f"     [INFO] Segmento '{segment_friendly_name}' tem apenas uma entrada: '{entry_name}'. Gerando mensagem simples.")
        message_html = f"""
        <div class="table-container">
            <h3 class="table-title">Análise por: {segment_friendly_name}</h3>
            <p class="single-entry-message">Em '{segment_friendly_name}' temos apenas uma entrada: <strong>{entry_name}</strong>. A análise detalhada deste item é refletida na tabela 'Carteira Consolidada'.</p>
        </div>
        """
        return message_html

    report_abs = df_input.pivot_table(
        index=segment_column,
        columns="MesVencimento",
        values="ValorNominal",
        fill_value=0,
        aggfunc='sum'
    )
    """tabela dinâmica de valores absolutos AC"""

    if report_abs.empty: return ""

    #! CHANGELOG -------------------------------------------------------------------------
    # Em vez de um pico global da tabela inteira, calculo o pico para cada item do índice.
    # peak_value = report_abs.max().max() #! REMOVIDO
    row_peaks = report_abs.max(axis=1) #?  valor máximo de cada linha
    row_peaks[row_peaks == 0] = np.nan # para não dar divisão por zero
    #!------------------------------------------------------------------------------------
    # VP e VL
    summary_totals = df_input.groupby(segment_column)[['_ValorLiquido', 'ValorPresente']].sum()
    summary_totals.rename(columns={'_ValorLiquido': 'Valor Líquido Total (R$)', 'ValorPresente': 'Valor Presente Total (R$)'}, inplace=True)

    #! CHANGELOG -------------------------------------------------------------------------
    # .div com axis=0 garante que cada linha de report_abs
    #  seja dividida pelo seu pico correspondente em row_peaks.
    report_performance = (1 - report_abs.div(row_peaks, axis=0)) * 100
    # ALTERAÇÃO 1: Linha que calculava o Total Nominal foi removida.
    # report_performance["Total Nominal (R$)"] = report_abs.sum(axis=1)
    #!------------------------------------------------------------------------------------

    report_performance = report_performance.join(summary_totals)

    # ALTERAÇÃO 1: Ordenando por 'Valor Presente Total' ao invés de 'Total Nominal'
    report_performance.sort_values(by="Valor Presente Total (R$)", ascending=False, inplace=True)
    report_performance.index.name = segment_friendly_name
    report_performance.reset_index(inplace=True)

    # renomear cols:
    period_cols = [c for c in report_performance.columns if isinstance(c, pd.Period)]
    period_to_relative_map = {p: convert_period_to_relative_month(p, ref_date) for p in period_cols}
    relative_to_html_map = {
        relative: f"{relative}<br><span class='sub-header'>{period.strftime('%Y-%m')}</span>"
        for period, relative in period_to_relative_map.items()
    }
    report_performance.rename(columns=period_to_relative_map, inplace=True)

    relative_month_cols = [c for c in report_performance.columns if isinstance(c, str) and c.startswith('M')]

    # (m+1)
    months_to_show = [c for c in relative_month_cols if int(c.replace('M','').replace('+','')) <= 1]

    sort_key = lambda col: int(col.replace('M', '').replace('+', ''))
    # Inverti a ordem das colunas aqui
    sorted_relative_cols = sorted(months_to_show, key=sort_key, reverse=True)

    # ALTERAÇÃO 1: Removido "Total Nominal (R$)" da ordem final das colunas
    final_cols_order = [
        segment_friendly_name,
        'Valor Presente Total (R$)',
        'Valor Líquido Total (R$)'
    ] + sorted_relative_cols

    #AC: (garantir que as colunas certas sao colcadas, desde que existam)
    final_cols_order = [col for col in final_cols_order if col in report_performance.columns]
    report_final = report_performance[final_cols_order]

    report_final.rename(columns=relative_to_html_map, inplace=True)
    sorted_html_cols = [relative_to_html_map.get(c, c) for c in sorted_relative_cols]

    #? Styles
    def format_br_percent(val):
        if pd.isna(val): return ""
        return f"{val:.2f}%".replace('.', ',')

    def format_br_number(val):
        if pd.isna(val): return ""
        return f"{val:,.2f}".replace(',', 'X').replace('.', ',').replace('X', '.')

    styler = (
        report_final.style
        .set_table_attributes('class="dataframe"')
        .set_properties(**{"padding": "8px", "text-align": "right", "min-width": "80px"})
        .set_table_styles([
            {"selector": "th, td", "props": [("border", "1px solid #ccc")]},
            {"selector": "th", "props": [("background-color", "#163f3f"), ("color", "#FFFFFF"), ("text-align", "center"), ("font-weight", "bold"), ("line-height", "1.3")]},
            {"selector": "thead th", "props": [("position", "sticky"), ("top", "0"), ("z-index", "1")]},
            {"selector": "tbody td:first-child", "props": [("position", "sticky"), ("left", "0"), ("background-color", "#f9f9f9"), ("text-align", "left"), ("font-weight", "bold"), ("white-space", "nowrap")]},
            {"selector": "thead th:first-child", "props": [("position", "sticky"), ("left", "0"), ("z-index", "2")]}
        ], overwrite=False)
        .format({
            **{col: format_br_percent for col in sorted_html_cols},
            # ALTERAÇÃO 1: Removido formatação do "Total Nominal (R$)"
            **{'Valor Presente Total (R$)': format_br_number,
               'Valor Líquido Total (R$)': format_br_number}
        })
        .background_gradient(cmap="RdYlGn", subset=sorted_html_cols, vmin=0, vmax=100, axis=None)
        # ALTERAÇÃO 1: Removido background do "Total Nominal (R$)"
        .background_gradient(cmap="Greys", subset=['Valor Presente Total (R$)', 'Valor Líquido Total (R$)'], axis=None)
        .hide(axis="index")
    )

    header_tabela = f"""
    <div class="table-container">
        <h3 class="table-title">Performance de Vencimentos por: {segment_friendly_name}</h3>
        {styler.to_html()}
    </div>
    """
    return header_tabela


#main:

print("\n" + "="*80)
print("INICIANDO A GERAÇÃO DO RELATÓRIO DE PERFORMANCE DE VENCIMENTOS")
print(f"Data de referência para os dados: {ref_date_obj.strftime('%d/%m/%Y')}")
print("="*80)

html_tabelas = []
for nome_amigavel, nome_coluna in segmentos_para_analise.items():
    tabela_html = generate_performance_heatmap(df_report, nome_coluna, nome_amigavel, ref_date_obj)
    if tabela_html:
        html_tabelas.append(tabela_html)

#? html final

if html_tabelas:
    html_css = """
    <style>
        body { font-family: "Segoe UI", Arial, sans-serif; background-color: #f4f4f4; margin: 0; padding: 20px; }
        .main-container { max-width: 95%; margin: auto; background-color: #ffffff; padding: 25px; border-radius: 8px; box-shadow: 0 4px 8px rgba(0,0,0,0.1); }
        .report-header { text-align: center; border-bottom: 2px solid #163f3f; padding-bottom: 15px; margin-bottom: 30px; }
        .report-header h1 { color: #163f3f; margin: 0; }
        .report-header p { color: #555; font-size: 1.1em; }
        /* ALTERAÇÃO 2: Adicionado 'overflow-x: auto' para criar rolagem horizontal na tabela */
        .table-container {
            margin-bottom: 50px;
            overflow-x: auto;
        }
        .table-title { color: #0e5d5f; border-bottom: 2px solid #76c6c5; padding-bottom: 8px; margin-bottom: 15px; }
        .dataframe { border-collapse: collapse; border-spacing: 0; width: 100%; font-size: 0.9em; }
        .dataframe td.na { background-color: transparent !important; }
        .single-entry-message {
            padding: 20px;
            background-color: #f8f9fa;
            border-left: 5px solid #76c6c5;
            margin-top: 15px;
            font-size: 1.1em;
            color: #333;
        }
        .sub-header {
            font-size: 0.8em;
            font-weight: normal;
            color: #dddddd;
        }
        footer {
            margin-top: 40px;
            padding-top: 20px;
            border-top: 1px solid #ddd;
            font-size: 0.9em;
            color: #777;
            text-align: center;
        }
    </style>
    """

    # AC: nova metodologia abaixo
    footer_text = """
    <p><strong>Nota sobre a Metodologia:</strong> Este relatório analisa a performance de cada item (ex: cada Originador) de forma individual.
    Para cada linha da tabela, o relatório identifica o mês de maior 'Valor Nominal' ('mês de pico' daquela linha) e o usa como referência (100%).
    Os demais meses da mesma linha são então mostrados como um percentual desse pico pessoal. Isso permite avaliar a consistência e a tendência de cada item em relação ao seu próprio melhor desempenho histórico.</p>
    """

    html_final = f"""
    <!DOCTYPE html>
    <html lang="pt-BR">
    <head>
        <meta charset="UTF-8">
        <title>Relatório de Performance de Vencimentos (Aging)</title>
        {html_css}
    </head>
    <body>
        <div class="main-container">
            <div class="report-header">
                <h1>Relatório de Performance de Vencimentos</h1>
                <p>Data de referência dos dados: {ref_date_obj.strftime('%d/%m/%Y')}</p>
            </div>
            {''.join(html_tabelas)}

            <footer>
                {footer_text}
            </footer>
        </div>
    </body>
    </html>
    """

    nome_arquivo_saida = f"{date.today().strftime('%Y-%m-%d')}-relatorio_performance_vencimentos_aging.html"
    caminho_arquivo_saida = os.path.join(output_path, nome_arquivo_saida)

    with open(caminho_arquivo_saida, "w", encoding="utf-8") as f:
        f.write(html_final)
    print("\n" + "="*80)
    print("PROCESSO FINALIZADO COM SUCESSO!")
    print(f"Relatório de Performance (Aging) gerado em: {caminho_arquivo_saida}")
    print("="*80)

else:
    print("\nNenhuma tabela foi gerada. O relatório final está vazio.")


INICIANDO A GERAÇÃO DO RELATÓRIO DE PERFORMANCE DE VENCIMENTOS
Data de referência para os dados: 05/08/2025
---> Processando segmento: Carteira Consolidada...


C:\Users\Leo\AppData\Local\Temp\ipykernel_31472\990333329.py:149: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  report_final.rename(columns=relative_to_html_map, inplace=True)


---> Processando segmento: Cedentes...
     [INFO] Segmento 'Cedentes' tem apenas uma entrada: 'BMP MONEY PLUS SOCIEDADE DE CREDITO DIRETO S.A'. Gerando mensagem simples.
---> Processando segmento: Originadores...
     [INFO] Segmento 'Originadores' tem apenas uma entrada: 'StarCard'. Gerando mensagem simples.
---> Processando segmento: Promotoras...


C:\Users\Leo\AppData\Local\Temp\ipykernel_31472\990333329.py:149: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  report_final.rename(columns=relative_to_html_map, inplace=True)


---> Processando segmento: Produtos...


...

C:\Users\Leo\AppData\Local\Temp\ipykernel_31472\990333329.py:149: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  report_final.rename(columns=relative_to_html_map, inplace=True)


---> Processando segmento: Pagamento Parcial...


C:\Users\Leo\AppData\Local\Temp\ipykernel_31472\990333329.py:149: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  report_final.rename(columns=relative_to_html_map, inplace=True)


---> Processando segmento: Tem Muitos Contratos...
---> Processando segmento: Tem Muitos Entes...
     [INFO] Segmento 'Tem Muitos Entes' tem apenas uma entrada: 'False'. Gerando mensagem simples.

PROCESSO FINALIZADO COM SUCESSO!
Relatório de Performance (Aging) gerado em: C:\Users\Leo\Desktop\Porto_Real\portoauto\src\originadores\output\2025-08-12-relatorio_performance_vencimentos_aging.html


C:\Users\Leo\AppData\Local\Temp\ipykernel_31472\990333329.py:149: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  report_final.rename(columns=relative_to_html_map, inplace=True)


In [31]:
#%%
# =============================================================================
# CÉLULA DE GERAÇÃO DO RELATÓRIO DE PERFORMANCE DE VENCIMENTOS (AGING)
# =============================================================================
# Descrição: Este script utiliza a lógica de análise de performance baseada no
# "mês de pico" de CADA LINHA. Ele calcula o volume nominal de cada mês como
# um percentual do mês de maior volume da história para aquele item específico.

import pandas as pd
import numpy as np
import os
from datetime import datetime, date
from dateutil.relativedelta import relativedelta


if 'df_final2' not in locals() or df_final2.empty:
    raise NameError("Você esqueceu de executar as céulas anteriores para definir o df_final2")


#! ==============================================================================
output_path = r'C:\Users\Leo\Desktop\Porto_Real\portoauto\src\originadores\output'
"""dir de saída"""
os.makedirs(output_path, exist_ok=True)
#FIXME
#! ==============================================================================

df_report = df_final2.copy()
df_report['_total_carteira'] = 'Carteira Consolidada'
# ```coluna para permitir a análise da carteira total```
df_report['MesVencimento'] = pd.to_datetime(df_report['DataVencimento']).dt.to_period('M')


segmentos_para_analise = {
    'Carteira Consolidada': '_total_carteira',
    'Cedentes': 'Cedente',
    'Originadores': 'Originador',
    'Promotoras': 'Promotora',
    'Produtos': 'Produto',
    'Convênios': 'Convênio',
    'Situação': 'Situacao',
    'UF': 'UF',
    'CAPAG': 'CAPAG',
    'Pagamento Parcial': 'PagamentoParcial',
    'Tem Muitos Contratos':'_MuitosContratos',
    'Tem Muitos Entes':'_MuitosEntes'
}
""" os segmentos que queremos analisar (variáveis categóricas)"""


ref_date_obj = df_report['DataGeracao'].max().date()
"""Define a data de referência a partir da tabela"""
# recomendação: coloque isso ou a data de hoje, para ficar mais robusto


#*=================================================================================
def convert_period_to_relative_month(period: pd.Period, ref_date: date) -> str:
    """converte  para um formato de mês relativo (eg M-1, M+0)."""
    venc_month = period.to_timestamp().to_pydatetime()
    ref_month = datetime(ref_date.year, ref_date.month, 1)
    delta = relativedelta(venc_month, ref_month)
    months_diff = delta.years * 12 + delta.months
    return f"M{months_diff:+}"

#*=================================================================================================================
def generate_performance_heatmap(df_input: pd.DataFrame, segment_column: str, segment_friendly_name: str, ref_date: date) -> str:
    """
    tabela HTML de performance de vencimentos para um segmento.
    """
    print(f"---> Processando segmento: {segment_friendly_name}...")

    # Lógica para pular segmentos com apenas uma entrada
    unique_entries = df_input[segment_column].dropna().unique()
    if len(unique_entries) == 1 and segment_column != '_total_carteira':
        entry_name = unique_entries[0]
        print(f"     [INFO] Segmento '{segment_friendly_name}' tem apenas uma entrada: '{entry_name}'. Gerando mensagem simples.")
        message_html = f"""
        <div class="table-container">
            <h3 class="table-title">Análise por: {segment_friendly_name}</h3>
            <p class="single-entry-message">Em '{segment_friendly_name}' temos apenas uma entrada: <strong>{entry_name}</strong>. A análise detalhada deste item é refletida na tabela 'Carteira Consolidada'.</p>
        </div>
        """
        return message_html

    report_abs = df_input.pivot_table(
        index=segment_column,
        columns="MesVencimento",
        values="ValorNominal",
        fill_value=0,
        aggfunc='sum'
    )
    """tabela dinâmica de valores absolutos AC"""

    if report_abs.empty: return ""

    #! CHANGELOG -------------------------------------------------------------------------
    # Em vez de um pico global da tabela inteira, calculo o pico para cada item do índice.
    # peak_value = report_abs.max().max() #! REMOVIDO
    row_peaks = report_abs.max(axis=1) #?  valor máximo de cada linha
    row_peaks[row_peaks == 0] = np.nan # para não dar divisão por zero
    #!------------------------------------------------------------------------------------
    # VP e VL
    summary_totals = df_input.groupby(segment_column)[['_ValorLiquido', 'ValorPresente']].sum()
    summary_totals.rename(columns={'_ValorLiquido': 'Valor Líquido Total (R$)', 'ValorPresente': 'Valor Presente Total (R$)'}, inplace=True)

    #! CHANGELOG -------------------------------------------------------------------------
    # .div com axis=0 garante que cada linha de report_abs
    #  seja dividida pelo seu pico correspondente em row_peaks.
    report_performance = (1 - report_abs.div(row_peaks, axis=0)) * 100
    #!------------------------------------------------------------------------------------

    report_performance = report_performance.join(summary_totals)

    report_performance.sort_values(by="Valor Presente Total (R$)", ascending=False, inplace=True)
    report_performance.index.name = segment_friendly_name
    report_performance.reset_index(inplace=True)

    # renomear cols:
    period_cols = [c for c in report_performance.columns if isinstance(c, pd.Period)]
    period_to_relative_map = {p: convert_period_to_relative_month(p, ref_date) for p in period_cols}
    relative_to_html_map = {
        relative: f"{relative}<br><span class='sub-header'>{period.strftime('%Y-%m')}</span>"
        for period, relative in period_to_relative_map.items()
    }
    report_performance.rename(columns=period_to_relative_map, inplace=True)

    relative_month_cols = [c for c in report_performance.columns if isinstance(c, str) and c.startswith('M')]

    # (m+1)
    months_to_show = [c for c in relative_month_cols if int(c.replace('M','').replace('+','')) <= 1]

    sort_key = lambda col: int(col.replace('M', '').replace('+', ''))
    # Inverti a ordem das colunas aqui
    sorted_relative_cols = sorted(months_to_show, key=sort_key, reverse=True)

    final_cols_order = [
        segment_friendly_name,
        'Valor Presente Total (R$)',
        'Valor Líquido Total (R$)'
    ] + sorted_relative_cols

    #AC: (garantir que as colunas certas sao colcadas, desde que existam)
    final_cols_order = [col for col in final_cols_order if col in report_performance.columns]
    report_final = report_performance[final_cols_order]

    # ALTERAÇÃO 3: Truncar os nomes na coluna de índice (a primeira coluna) para 30 caracteres.
    report_final[segment_friendly_name] = report_final[segment_friendly_name].astype(str).apply(lambda x: (x[:27] + '...') if len(x) > 30 else x)

    report_final.rename(columns=relative_to_html_map, inplace=True)
    sorted_html_cols = [relative_to_html_map.get(c, c) for c in sorted_relative_cols]

    #? Styles
    def format_br_percent(val):
        if pd.isna(val): return ""
        return f"{val:.2f}%".replace('.', ',')

    def format_br_number(val):
        if pd.isna(val): return ""
        return f"{val:,.2f}".replace(',', 'X').replace('.', ',').replace('X', '.')

    styler = (
        report_final.style
        .set_table_attributes('class="dataframe"')
        .set_properties(**{"padding": "8px", "text-align": "right", "min-width": "80px"})
        .set_table_styles([
            {"selector": "th, td", "props": [("border", "1px solid #ccc")]},
            {"selector": "th", "props": [("background-color", "#163f3f"), ("color", "#FFFFFF"), ("text-align", "center"), ("font-weight", "bold"), ("line-height", "1.3")]},
            {"selector": "thead th", "props": [("position", "sticky"), ("top", "0"), ("z-index", "1")]},
            {"selector": "tbody td:first-child", "props": [("position", "sticky"), ("left", "0"), ("background-color", "#f9f9f9"), ("text-align", "left"), ("font-weight", "bold"), ("white-space", "nowrap")]},
            {"selector": "thead th:first-child", "props": [("position", "sticky"), ("left", "0"), ("z-index", "2")]}
        ], overwrite=False)
        .format({
            **{col: format_br_percent for col in sorted_html_cols},
            **{'Valor Presente Total (R$)': format_br_number,
               'Valor Líquido Total (R$)': format_br_number}
        })
        .background_gradient(cmap="RdYlGn", subset=sorted_html_cols, vmin=0, vmax=100, axis=None)
        .background_gradient(cmap="Greys", subset=['Valor Presente Total (R$)', 'Valor Líquido Total (R$)'], axis=None)
        .hide(axis="index")
    )

    header_tabela = f"""
    <div class="table-container">
        <h3 class="table-title">Performance de Vencimentos por: {segment_friendly_name}</h3>
        {styler.to_html()}
    </div>
    """
    return header_tabela


#main:

print("\n" + "="*80)
print("INICIANDO A GERAÇÃO DO RELATÓRIO DE PERFORMANCE DE VENCIMENTOS")
print(f"Data de referência para os dados: {ref_date_obj.strftime('%d/%m/%Y')}")
print("="*80)

html_tabelas = []
for nome_amigavel, nome_coluna in segmentos_para_analise.items():
    tabela_html = generate_performance_heatmap(df_report, nome_coluna, nome_amigavel, ref_date_obj)
    if tabela_html:
        html_tabelas.append(tabela_html)

#? html final

if html_tabelas:
    html_css = """
    <style>
        body { font-family: "Segoe UI", Arial, sans-serif; background-color: #f4f4f4; margin: 0; padding: 20px; }
        .main-container { max-width: 95%; margin: auto; background-color: #ffffff; padding: 25px; border-radius: 8px; box-shadow: 0 4px 8px rgba(0,0,0,0.1); }
        .report-header { text-align: center; border-bottom: 2px solid #163f3f; padding-bottom: 15px; margin-bottom: 30px; }
        .report-header h1 { color: #163f3f; margin: 0; }
        .report-header p { color: #555; font-size: 1.1em; }
        .table-container {
            margin-bottom: 50px;
            overflow-x: auto;
        }
        .table-title { color: #0e5d5f; border-bottom: 2px solid #76c6c5; padding-bottom: 8px; margin-bottom: 15px; }
        .dataframe { border-collapse: collapse; border-spacing: 0; width: 100%; font-size: 0.9em; }
        .dataframe td.na { background-color: transparent !important; }
        .single-entry-message {
            padding: 20px;
            background-color: #f8f9fa;
            border-left: 5px solid #76c6c5;
            margin-top: 15px;
            font-size: 1.1em;
            color: #333;
        }
        .sub-header {
            font-size: 0.8em;
            font-weight: normal;
            color: #dddddd;
        }
        footer {
            margin-top: 40px;
            padding-top: 20px;
            border-top: 1px solid #ddd;
            font-size: 0.9em;
            color: #777;
            text-align: center;
        }
    </style>
    """

    # AC: nova metodologia abaixo
    footer_text = """
    <p><strong>Nota sobre a Metodologia:</strong> Este relatório analisa a performance de cada item (ex: cada Originador) de forma individual.
    Para cada linha da tabela, o relatório identifica o mês de maior 'Valor Nominal' ('mês de pico' daquela linha) e o usa como referência (100%).
    Os demais meses da mesma linha são então mostrados como um percentual desse pico pessoal. Isso permite avaliar a consistência e a tendência de cada item em relação ao seu próprio melhor desempenho histórico.</p>
    """

    html_final = f"""
    <!DOCTYPE html>
    <html lang="pt-BR">
    <head>
        <meta charset="UTF-8">
        <title>Relatório de Performance de Vencimentos (Aging)</title>
        {html_css}
    </head>
    <body>
        <div class="main-container">
            <div class="report-header">
                <h1>Relatório de Performance de Vencimentos</h1>
                <p>Data de referência dos dados: {ref_date_obj.strftime('%d/%m/%Y')}</p>
            </div>
            {''.join(html_tabelas)}

            <footer>
                {footer_text}
            </footer>
        </div>
    </body>
    </html>
    """

    nome_arquivo_saida = f"{date.today().strftime('%Y-%m-%d')}-relatorio_performance_vencimentos_aging.html"
    caminho_arquivo_saida = os.path.join(output_path, nome_arquivo_saida)

    with open(caminho_arquivo_saida, "w", encoding="utf-8") as f:
        f.write(html_final)
    print("\n" + "="*80)
    print("PROCESSO FINALIZADO COM SUCESSO!")
    print(f"Relatório de Performance (Aging) gerado em: {caminho_arquivo_saida}")
    print("="*80)

else:
    print("\nNenhuma tabela foi gerada. O relatório final está vazio.")


INICIANDO A GERAÇÃO DO RELATÓRIO DE PERFORMANCE DE VENCIMENTOS
Data de referência para os dados: 05/08/2025
---> Processando segmento: Carteira Consolidada...


C:\Users\Leo\AppData\Local\Temp\ipykernel_31472\14761674.py:146: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  report_final[segment_friendly_name] = report_final[segment_friendly_name].astype(str).apply(lambda x: (x[:27] + '...') if len(x) > 30 else x)
C:\Users\Leo\AppData\Local\Temp\ipykernel_31472\14761674.py:148: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  report_final.rename(columns=relative_to_html_map, inplace=True)


---> Processando segmento: Cedentes...
     [INFO] Segmento 'Cedentes' tem apenas uma entrada: 'BMP MONEY PLUS SOCIEDADE DE CREDITO DIRETO S.A'. Gerando mensagem simples.
---> Processando segmento: Originadores...
     [INFO] Segmento 'Originadores' tem apenas uma entrada: 'StarCard'. Gerando mensagem simples.
---> Processando segmento: Promotoras...


C:\Users\Leo\AppData\Local\Temp\ipykernel_31472\14761674.py:146: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  report_final[segment_friendly_name] = report_final[segment_friendly_name].astype(str).apply(lambda x: (x[:27] + '...') if len(x) > 30 else x)
C:\Users\Leo\AppData\Local\Temp\ipykernel_31472\14761674.py:148: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  report_final.rename(columns=relative_to_html_map, inplace=True)


---> Processando segmento: Produtos...


...

C:\Users\Leo\AppData\Local\Temp\ipykernel_31472\14761674.py:146: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  report_final[segment_friendly_name] = report_final[segment_friendly_name].astype(str).apply(lambda x: (x[:27] + '...') if len(x) > 30 else x)
C:\Users\Leo\AppData\Local\Temp\ipykernel_31472\14761674.py:148: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  report_final.rename(columns=relative_to_html_map, inplace=True)


---> Processando segmento: CAPAG...
---> Processando segmento: Pagamento Parcial...


C:\Users\Leo\AppData\Local\Temp\ipykernel_31472\14761674.py:146: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  report_final[segment_friendly_name] = report_final[segment_friendly_name].astype(str).apply(lambda x: (x[:27] + '...') if len(x) > 30 else x)
C:\Users\Leo\AppData\Local\Temp\ipykernel_31472\14761674.py:148: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  report_final.rename(columns=relative_to_html_map, inplace=True)
C:\Users\Leo\AppData\Local\Temp\ipykernel_31472\14761674.py:146: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fr

---> Processando segmento: Tem Muitos Contratos...
---> Processando segmento: Tem Muitos Entes...
     [INFO] Segmento 'Tem Muitos Entes' tem apenas uma entrada: 'False'. Gerando mensagem simples.

PROCESSO FINALIZADO COM SUCESSO!
Relatório de Performance (Aging) gerado em: C:\Users\Leo\Desktop\Porto_Real\portoauto\src\originadores\output\2025-08-12-relatorio_performance_vencimentos_aging.html


C:\Users\Leo\AppData\Local\Temp\ipykernel_31472\14761674.py:146: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  report_final[segment_friendly_name] = report_final[segment_friendly_name].astype(str).apply(lambda x: (x[:27] + '...') if len(x) > 30 else x)
C:\Users\Leo\AppData\Local\Temp\ipykernel_31472\14761674.py:148: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  report_final.rename(columns=relative_to_html_map, inplace=True)


In [35]:
# =============================================================================
# BLOCO DE DEPURAÇÃO PARA O SEGMENTO 'Pagamento Parcial'
# =============================================================================
print("--- INICIANDO DEPURAÇÃO DO SEGMENTO 'Pagamento Parcial' ---")

# 1. DEFINIÇÕES BÁSICAS (copiadas do seu script)
# -----------------------------------------------------------------------------
segment_column = 'PagamentoParcial'
segment_friendly_name = 'Pagamento Parcial'
ref_date = df_report['DataGeracao'].max().date()

def convert_period_to_relative_month(period: pd.Period, ref_date: date) -> str:
    venc_month = period.to_timestamp().to_pydatetime()
    ref_month = datetime(ref_date.year, ref_date.month, 1)
    delta = relativedelta(venc_month, ref_month)
    months_diff = delta.years * 12 + delta.months
    return f"M{months_diff:+}"

# 2. ISOLAR OS DADOS E CRIAR A TABELA PIVOT ABSOLUTA
# Vamos ver os valores nominais brutos para cada mês.
# -----------------------------------------------------------------------------
df_segmento = df_report[df_report[segment_column].notna()].copy()

report_abs = df_segmento.pivot_table(
    index=segment_column,
    columns="MesVencimento",
    values="ValorNominal",
    fill_value=0,
    aggfunc='sum'
)

print("\n\nPASSO 1: Tabela de Valor Nominal Absoluto (R$)\n" + "="*50)
print("Aqui estão os valores nominais brutos que entram no cálculo.")
display(report_abs)


# 3. CALCULAR O PICO DE CADA LINHA
# Vamos ver qual é o valor máximo encontrado para a linha 'SIM' e 'NÃO'.
# -----------------------------------------------------------------------------
row_peaks = report_abs.max(axis=1)
print("\n\nPASSO 2: Pico de Valor Nominal por Categoria (R$)\n" + "="*50)
print("Este é o valor usado como referência (denominador) para cada linha.")
display(row_peaks)


# 4. CALCULAR A TABELA DE PERFORMANCE
# Agora, aplicamos a fórmula para ver o resultado percentual.
# -----------------------------------------------------------------------------
# Adicionamos um valor pequeno para evitar divisão por zero de forma segura
row_peaks_safe = row_peaks.replace(0, np.nan)
report_performance = (1 - report_abs.div(row_peaks_safe, axis=0)) * 100

# Renomear colunas para o formato M-1, M+0 etc., para facilitar a leitura
period_cols = [c for c in report_performance.columns if isinstance(c, pd.Period)]
period_to_relative_map = {p: convert_period_to_relative_month(p, ref_date) for p in period_cols}
report_performance.rename(columns=period_to_relative_map, inplace=True)


print("\n\nPASSO 3: Tabela de Performance Calculada (%)\n" + "="*50)
print("Este é o resultado final que aparece no relatório.")
# Filtrar para mostrar apenas os meses mais relevantes
months_to_show = [c for c in report_performance.columns if isinstance(c, str) and c.startswith('M')]
sort_key = lambda col: int(col.replace('M', '').replace('+', ''))
sorted_relative_cols = sorted(months_to_show, key=sort_key)
display(report_performance[sorted_relative_cols].style.format("{:.2f}%"))


print("\n--- FIM DA DEPURAÇÃO ---")

--- INICIANDO DEPURAÇÃO DO SEGMENTO 'Pagamento Parcial' ---


PASSO 1: Tabela de Valor Nominal Absoluto (R$)
Aqui estão os valores nominais brutos que entram no cálculo.


MesVencimento,2024-10,2024-11,2024-12,2025-01,2025-02,2025-03,2025-04,2025-05,2025-06,2025-07,2025-08,2025-09,2025-10,2025-11,2025-12,2026-01,2026-02,2026-03,2026-04,2026-05,2026-06,2026-07,2026-08,2026-09,2026-10,2026-11,2026-12,2027-01,2027-02,2027-03,2027-04,2027-05,2027-06,2027-07,2027-08,2027-09,2027-10,2027-11,2027-12,2028-01,2028-02,2028-03,2028-04,2028-05,2028-06,2028-07,2028-08,2028-09,2028-10,2028-11,...,2031-07,2031-08,2031-09,2031-10,2031-11,2031-12,2032-01,2032-02,2032-03,2032-04,2032-05,2032-06,2032-07,2032-08,2032-09,2032-10,2032-11,2032-12,2033-01,2033-02,2033-03,2033-04,2033-05,2033-06,2033-07,2033-08,2033-09,2033-10,2033-11,2033-12,2034-01,2034-02,2034-03,2034-04,2034-05,2034-06,2034-07,2034-08,2034-09,2034-10,2034-11,2034-12,2035-01,2035-02,2035-03,2035-04,2035-05,2035-06,2035-07,2035-08
PagamentoParcial,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
NAO,80.14,2237.19,11615.89,64135.07,145332.18,175765.33,469431.22,738752.98,1127829.35,1336298.43,1608560.94,1678016.74,1678416.74,1678416.74,1678065.63,1677932.86,1675842.38,1675545.6,1670950.85,1665214.6,1658951.55,1622116.71,1559265.06,1471420.54,1321892.19,1144353.33,885524.03,788020.33,749196.4,744580.38,730657.3,706199.67,688911.17,661289.36,596007.83,545094.6,543276.22,538405.61,528124.6,511126.5,495192.17,485316.83,469076.13,455094.08,450943.17,424251.4,339688.95,334770.69,334082.14,331245.0,...,184074.94,183305.0,182081.35,182081.35,179887.94,178873.95,175820.37,173835.54,173397.2,172182.34,171007.44,170074.51,167638.7,165905.62,165463.99,165313.85,164255.48,161862.98,152032.28,140439.39,135687.41,134420.78,130066.99,125555.61,119810.88,116123.84,114771.45,114771.45,114771.45,114771.45,114771.45,114771.45,114771.45,114771.45,114771.45,114771.45,114771.45,114771.45,114771.45,114771.45,113094.78,112778.68,76480.4,64912.18,52172.36,42608.93,30113.69,22269.99,17704.55,4507.57
SIM,241.90,73.91,237.04,1627.85,5549.45,8882.45,10009.12,15410.84,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.0,...,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00




PASSO 2: Pico de Valor Nominal por Categoria (R$)
Este é o valor usado como referência (denominador) para cada linha.


PagamentoParcial
NAO    1678416.74
SIM      15410.84
dtype: float64



PASSO 3: Tabela de Performance Calculada (%)
Este é o resultado final que aparece no relatório.


MesVencimento,M-10,M-9,M-8,M-7,M-6,M-5,M-4,M-3,M-2,M-1,M+0,M+1,M+2,M+3,M+4,M+5,M+6,M+7,M+8,M+9,M+10,M+11,M+12,M+13,M+14,M+15,M+16,M+17,M+18,M+19,M+20,M+21,M+22,M+23,M+24,M+25,M+26,M+27,M+28,M+29,M+30,M+31,M+32,M+33,M+34,M+35,M+36,M+37,M+38,M+39,M+40,M+41,M+42,M+43,M+44,M+45,M+46,M+47,M+48,M+49,M+50,M+51,M+52,M+53,M+54,M+55,M+56,M+57,M+58,M+59,M+60,M+61,M+62,M+63,M+64,M+65,M+66,M+67,M+68,M+69,M+70,M+71,M+72,M+73,M+74,M+75,M+76,M+77,M+78,M+79,M+80,M+81,M+82,M+83,M+84,M+85,M+86,M+87,M+88,M+89,M+90,M+91,M+92,M+93,M+94,M+95,M+96,M+97,M+98,M+99,M+100,M+101,M+102,M+103,M+104,M+105,M+106,M+107,M+108,M+109,M+110,M+111,M+112,M+113,M+114,M+115,M+116,M+117,M+118,M+119,M+120
PagamentoParcial,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
NAO,100.00%,99.87%,99.31%,96.18%,91.34%,89.53%,72.03%,55.99%,32.80%,20.38%,4.16%,0.02%,0.00%,0.00%,0.02%,0.03%,0.15%,0.17%,0.44%,0.79%,1.16%,3.35%,7.10%,12.33%,21.24%,31.82%,47.24%,53.05%,55.36%,55.64%,56.47%,57.92%,58.95%,60.60%,64.49%,67.52%,67.63%,67.92%,68.53%,69.55%,70.50%,71.08%,72.05%,72.89%,73.13%,74.72%,79.76%,80.05%,80.10%,80.26%,80.50%,81.35%,81.75%,82.14%,82.54%,82.92%,83.37%,84.11%,84.82%,85.07%,85.08%,85.14%,85.22%,85.68%,85.99%,86.52%,86.98%,87.20%,87.42%,87.60%,87.76%,87.83%,87.83%,87.89%,87.89%,88.07%,88.30%,88.42%,88.53%,88.89%,88.97%,89.03%,89.08%,89.15%,89.15%,89.28%,89.34%,89.52%,89.64%,89.67%,89.74%,89.81%,89.87%,90.01%,90.12%,90.14%,90.15%,90.21%,90.36%,90.94%,91.63%,91.92%,91.99%,92.25%,92.52%,92.86%,93.08%,93.16%,93.16%,93.16%,93.16%,93.16%,93.16%,93.16%,93.16%,93.16%,93.16%,93.16%,93.16%,93.16%,93.16%,93.26%,93.28%,95.44%,96.13%,96.89%,97.46%,98.21%,98.67%,98.95%,99.73%
SIM,98.43%,99.52%,98.46%,89.44%,63.99%,42.36%,35.05%,0.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%



--- FIM DA DEPURAÇÃO ---


In [21]:
#%%
# =============================================================================
# CÉLULA DE GERAÇÃO DO RELATÓRIO DE PERFORMANCE DE VENCIMENTOS (AGING)
# =============================================================================
# Descrição: Este script utiliza a lógica de análise de performance baseada no
# "mês de pico". Ele calcula o volume nominal de cada mês como um percentual
# do mês de maior volume nominal da história, usando a visão de "aging".

import pandas as pd
import numpy as np
import os
from datetime import datetime, date
from dateutil.relativedelta import relativedelta

# --- CONFIGURAÇÕES INICIAIS ---

# Verifica se o DataFrame principal (df_final2) existe.
if 'df_final2' not in locals() or df_final2.empty:
    raise NameError("O DataFrame 'df_final2' não foi encontrado ou está vazio. Por favor, execute as células anteriores primeiro.")

# Adiciona uma coluna para permitir a análise da carteira total
df_report = df_final2.copy()
df_report['_total_carteira'] = 'Carteira Consolidada'
df_report['MesVencimento'] = pd.to_datetime(df_report['DataVencimento']).dt.to_period('M')

# Define os segmentos que queremos analisar
segmentos_para_analise = {
    'Carteira Consolidada': '_total_carteira',
    'Cedentes': 'Cedente',
    'Originadores': 'Originador',
    'Promotoras': 'Promotora',
    'Produtos': 'Produto',
    'Convênios': 'Convênio',
    'Situação': 'Situacao',
    'UF': 'UF',
    'CAPAG': 'CAPAG',
    'Pagamento Parcial': 'PagamentoParcial',
    'Tem Muitos Contratos':'_MuitosContratos',
    'Tem Muitos Entes':'_MuitosEntes'
}

# Define a data de referência
ref_date_obj = df_report['DataGeracao'].max().date()

# Define o caminho de saída
output_path = r'C:\Users\Leo\Desktop\Porto_Real\portoauto\src\originadores\output'
os.makedirs(output_path, exist_ok=True)


# --- FUNÇÕES AUXILIARES ---

def convert_period_to_relative_month(period: pd.Period, ref_date: date) -> str:
    """Converte um objeto Period para um formato de mês relativo (ex: M-1, M+0)."""
    venc_month = period.to_timestamp().to_pydatetime()
    ref_month = datetime(ref_date.year, ref_date.month, 1)
    delta = relativedelta(venc_month, ref_month)
    months_diff = delta.years * 12 + delta.months
    return f"M{months_diff:+}"

# --- FUNÇÃO PRINCIPAL DE GERAÇÃO DA TABELA ---

def generate_performance_heatmap(df_input: pd.DataFrame, segment_column: str, segment_friendly_name: str, ref_date: date) -> str:
    """
    Gera uma tabela HTML de performance de vencimentos para um segmento.
    """
    print(f"---> Processando segmento: {segment_friendly_name}...")
    
    # Lógica para pular segmentos com apenas uma entrada
    unique_entries = df_input[segment_column].dropna().unique()
    if len(unique_entries) == 1 and segment_column != '_total_carteira':
        entry_name = unique_entries[0]
        print(f"     [INFO] Segmento '{segment_friendly_name}' tem apenas uma entrada: '{entry_name}'. Gerando mensagem simples.")
        message_html = f"""
        <div class="table-container">
            <h3 class="table-title">Análise por: {segment_friendly_name}</h3>
            <p class="single-entry-message">Em '{segment_friendly_name}' temos apenas uma entrada: <strong>{entry_name}</strong>. A análise detalhada deste item é refletida na tabela 'Carteira Consolidada'.</p>
        </div>
        """
        return message_html

    # 1. Criação da Tabela Dinâmica com Valor Nominal
    report_abs = df_input.pivot_table(
        index=segment_column,
        columns="MesVencimento",
        values="ValorNominal",
        fill_value=0,
        aggfunc='sum'
    )
    if report_abs.empty: return ""

    # 2. Cálculo de Pico Dinâmico
    peak_value = report_abs.max().max()
    if peak_value == 0: return ""
    print(f"     Pico de Valor Nominal para este segmento: R$ {peak_value:,.2f}")

    # 3. Cálculo dos Totais de VP e VL
    summary_totals = df_input.groupby(segment_column)[['_ValorLiquido', 'ValorPresente']].sum()
    summary_totals.rename(columns={'_ValorLiquido': 'Valor Líquido Total (R$)', 'ValorPresente': 'Valor Presente Total (R$)'}, inplace=True)

    # 4. Cálculo da Performance
    report_performance = (1 - (report_abs / peak_value)) * 100
    report_performance["Total Nominal (R$)"] = report_abs.sum(axis=1)
    
    # Junta os totais de VP e VL
    report_performance = report_performance.join(summary_totals)
    
    report_performance.sort_values(by="Total Nominal (R$)", ascending=False, inplace=True)
    report_performance.index.name = segment_friendly_name
    report_performance.reset_index(inplace=True)

    # 5. Lógica de Renomeação e Ordenação de Colunas
    period_cols = [c for c in report_performance.columns if isinstance(c, pd.Period)]
    period_to_relative_map = {p: convert_period_to_relative_month(p, ref_date) for p in period_cols}
    relative_to_html_map = {
        relative: f"{relative}<br><span class='sub-header'>{period.strftime('%Y-%m')}</span>"
        for period, relative in period_to_relative_map.items()
    }
    report_performance.rename(columns=period_to_relative_map, inplace=True)
    
    relative_month_cols = [c for c in report_performance.columns if isinstance(c, str) and c.startswith('M')]
    
    # [ALTERADO] Inclui o M+1 na análise para ver adiantamentos
    months_to_show = [c for c in relative_month_cols if int(c.replace('M','').replace('+','')) <= 1]
    
    sort_key = lambda col: int(col.replace('M', '').replace('+', ''))
    # Inverte a ordem das colunas de mês
    sorted_relative_cols = sorted(months_to_show, key=sort_key, reverse=True)
    
    # Adiciona as colunas de VP e VL no início da ordem
    final_cols_order = [
        segment_friendly_name, 
        'Valor Presente Total (R$)', 
        'Valor Líquido Total (R$)'
    ] + sorted_relative_cols + ["Total Nominal (R$)"]
    
    # Garante que apenas colunas existentes sejam selecionadas
    final_cols_order = [col for col in final_cols_order if col in report_performance.columns]
    report_final = report_performance[final_cols_order]

    report_final.rename(columns=relative_to_html_map, inplace=True)
    sorted_html_cols = [relative_to_html_map.get(c, c) for c in sorted_relative_cols]

    # 6. Estilização e Geração do HTML
    def format_br_percent(val):
        if pd.isna(val): return ""
        return f"{val:.2f}%".replace('.', ',')
        
    def format_br_number(val):
        if pd.isna(val): return ""
        return f"{val:,.2f}".replace(',', 'X').replace('.', ',').replace('X', '.')

    styler = (
        report_final.style
        .set_table_attributes('class="dataframe"')
        .set_properties(**{"padding": "8px", "text-align": "right", "min-width": "80px"})
        .set_table_styles([
            {"selector": "th, td", "props": [("border", "1px solid #ccc")]},
            {"selector": "th", "props": [("background-color", "#163f3f"), ("color", "#FFFFFF"), ("text-align", "center"), ("font-weight", "bold"), ("line-height", "1.3")]},
            {"selector": "thead th", "props": [("position", "sticky"), ("top", "0"), ("z-index", "1")]},
            {"selector": "tbody td:first-child", "props": [("position", "sticky"), ("left", "0"), ("background-color", "#f9f9f9"), ("text-align", "left"), ("font-weight", "bold"), ("white-space", "nowrap")]},
            {"selector": "thead th:first-child", "props": [("position", "sticky"), ("left", "0"), ("z-index", "2")]}
        ], overwrite=False)
        .format({
            **{col: format_br_percent for col in sorted_html_cols}, 
            **{"Total Nominal (R$)": format_br_number, 
               'Valor Presente Total (R$)': format_br_number, 
               'Valor Líquido Total (R$)': format_br_number}
        })
        .background_gradient(cmap="RdYlGn", subset=sorted_html_cols, vmin=0, vmax=100, axis=None)
        .background_gradient(cmap="Greys", subset=["Total Nominal (R$)", 'Valor Presente Total (R$)', 'Valor Líquido Total (R$)'], axis=None)
        .hide(axis="index")
    )
    
    header_tabela = f"""
    <div class="table-container">
        <h3 class="table-title">Performance de Vencimentos por: {segment_friendly_name}</h3>
        {styler.to_html()}
    </div>
    """
    return header_tabela


# --- BLOCO PRINCIPAL DE EXECUÇÃO ---

print("\n" + "="*80)
print("INICIANDO A GERAÇÃO DO RELATÓRIO DE PERFORMANCE DE VENCIMENTOS")
print(f"Data de referência para os dados: {ref_date_obj.strftime('%d/%m/%Y')}")
print("="*80)

html_tabelas = []
for nome_amigavel, nome_coluna in segmentos_para_analise.items():
    tabela_html = generate_performance_heatmap(df_report, nome_coluna, nome_amigavel, ref_date_obj)
    if tabela_html:
        html_tabelas.append(tabela_html)

# --- MONTAGEM DO ARQUIVO HTML FINAL ---

if html_tabelas:
    html_css = """
    <style>
        body { font-family: "Segoe UI", Arial, sans-serif; background-color: #f4f4f4; margin: 0; padding: 20px; }
        .main-container { max-width: 95%; margin: auto; background-color: #ffffff; padding: 25px; border-radius: 8px; box-shadow: 0 4px 8px rgba(0,0,0,0.1); }
        .report-header { text-align: center; border-bottom: 2px solid #163f3f; padding-bottom: 15px; margin-bottom: 30px; }
        .report-header h1 { color: #163f3f; margin: 0; }
        .report-header p { color: #555; font-size: 1.1em; }
        .table-container { margin-bottom: 50px; }
        .table-title { color: #0e5d5f; border-bottom: 2px solid #76c6c5; padding-bottom: 8px; margin-bottom: 15px; }
        .dataframe { border-collapse: collapse; border-spacing: 0; width: 100%; font-size: 0.9em; }
        .dataframe td.na { background-color: transparent !important; }
        .single-entry-message {
            padding: 20px;
            background-color: #f8f9fa;
            border-left: 5px solid #76c6c5;
            margin-top: 15px;
            font-size: 1.1em;
            color: #333;
        }
        .sub-header {
            font-size: 0.8em;
            font-weight: normal;
            color: #dddddd;
        }
        footer {
            margin-top: 40px;
            padding-top: 20px;
            border-top: 1px solid #ddd;
            font-size: 0.9em;
            color: #777;
            text-align: center;
        }
    </style>
    """
    
    footer_text = """
    <p><strong>Nota sobre a Metodologia:</strong> Este relatório utiliza a lógica de análise de performance baseada no "mês de pico". 
    Ele calcula o volume nominal de cada mês como um percentual do mês de maior volume nominal da história para aquele segmento, usando a visão de "aging". Ele faz isso porque precisamos recupearar a PMT, assumida contante, então assumimos que tomar o máximo vai funcionar como estimativa</p>
    """

    html_final = f"""
    <!DOCTYPE html>
    <html lang="pt-BR">
    <head>
        <meta charset="UTF-8">
        <title>Relatório de Performance de Vencimentos (Aging)</title>
        {html_css}
    </head>
    <body>
        <div class="main-container">
            <div class="report-header">
                <h1>Relatório de Performance de Vencimentos</h1>
                <p>Data de referência dos dados: {ref_date_obj.strftime('%d/%m/%Y')}</p>
            </div>
            {''.join(html_tabelas)}
            
            <footer>
                {footer_text}
            </footer>
        </div>
    </body>
    </html>
    """

    nome_arquivo_saida = f"{date.today().strftime('%Y-%m-%d')}-relatorio_performance_vencimentos_aging.html"
    caminho_arquivo_saida = os.path.join(output_path, nome_arquivo_saida)
    
    with open(caminho_arquivo_saida, "w", encoding="utf-8") as f:
        f.write(html_final)
    print("\n" + "="*80)
    print("PROCESSO FINALIZADO COM SUCESSO!")
    print(f"Relatório de Performance (Aging) gerado em: {caminho_arquivo_saida}")
    print("="*80)

else:
    print("\nNenhuma tabela foi gerada. O relatório final está vazio.")




INICIANDO A GERAÇÃO DO RELATÓRIO DE PERFORMANCE DE VENCIMENTOS
Data de referência para os dados: 05/08/2025
---> Processando segmento: Carteira Consolidada...
     Pico de Valor Nominal para este segmento: R$ 1,678,416.74


C:\Users\Leo\AppData\Local\Temp\ipykernel_31472\1593312765.py:140: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  report_final.rename(columns=relative_to_html_map, inplace=True)


---> Processando segmento: Cedentes...
     [INFO] Segmento 'Cedentes' tem apenas uma entrada: 'BMP MONEY PLUS SOCIEDADE DE CREDITO DIRETO S.A'. Gerando mensagem simples.
---> Processando segmento: Originadores...
     [INFO] Segmento 'Originadores' tem apenas uma entrada: 'StarCard'. Gerando mensagem simples.
---> Processando segmento: Promotoras...
     Pico de Valor Nominal para este segmento: R$ 258,210.27


C:\Users\Leo\AppData\Local\Temp\ipykernel_31472\1593312765.py:140: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  report_final.rename(columns=relative_to_html_map, inplace=True)


---> Processando segmento: Produtos...
     Pico de Valor Nominal para este segmento: R$ 469,550.56


...

---> Processando segmento: Pagamento Parcial...
     Pico de Valor Nominal para este segmento: R$ 1,678,416.74
---> Processando segmento: Tem Muitos Contratos...


C:\Users\Leo\AppData\Local\Temp\ipykernel_31472\1593312765.py:140: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  report_final.rename(columns=relative_to_html_map, inplace=True)


     Pico de Valor Nominal para este segmento: R$ 1,288,457.76
---> Processando segmento: Tem Muitos Entes...


C:\Users\Leo\AppData\Local\Temp\ipykernel_31472\1593312765.py:140: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  report_final.rename(columns=relative_to_html_map, inplace=True)


     [INFO] Segmento 'Tem Muitos Entes' tem apenas uma entrada: 'False'. Gerando mensagem simples.

PROCESSO FINALIZADO COM SUCESSO!
Relatório de Performance (Aging) gerado em: C:\Users\Leo\Desktop\Porto_Real\portoauto\src\originadores\output\2025-08-11-relatorio_performance_vencimentos_aging.html


In [26]:
#%%
# =============================================================================
# CÉLULA DE GERAÇÃO DO RELATÓRIO DE CURVA DE VENCIMENTOS (AGING)
# =============================================================================
# Descrição: Este script gera um relatório que analisa a curva completa de
# vencimentos (passado e futuro) e tenta inferir adiantamentos ao detetar
# "gaps" na sequência de parcelas a vencer.

import pandas as pd
import numpy as np
import os
from datetime import datetime, date
from dateutil.relativedelta import relativedelta

# --- CONFIGURAÇÕES INICIAIS ---

# Verifica se o DataFrame principal (df_final2) existe.
if 'df_final2' not in locals() or df_final2.empty:
    raise NameError("O DataFrame 'df_final2' não foi encontrado ou está vazio. Por favor, execute as células anteriores primeiro.")

# Adiciona uma coluna para permitir a análise da carteira total
df_report = df_final2.copy()
df_report['_total_carteira'] = 'Carteira Consolidada'
df_report['MesVencimento'] = pd.to_datetime(df_report['DataVencimento']).dt.to_period('M')

# Define os segmentos que queremos analisar
segmentos_para_analise = {
    'Carteira Consolidada': '_total_carteira',
    'Cedentes': 'Cedente',
    'Originadores': 'Originador',
    'Promotoras': 'Promotora',
    'Produtos': 'Produto',
    'Convênios': 'Convênio',
    'Situação': 'Situacao',
    'UF': 'UF',
    'CAPAG': 'CAPAG'
}

# Define a data de referência
ref_date_obj = df_report['DataGeracao'].max().date()

# Define o caminho de saída
output_path = r'C:\Users\Leo\Desktop\Porto_Real\portoauto\src\originadores\output'
os.makedirs(output_path, exist_ok=True)


# --- FUNÇÕES AUXILIARES ---

def convert_period_to_relative_month(period: pd.Period, ref_date: date) -> str:
    """Converte um objeto Period para um formato de mês relativo (ex: M+1, M+2)."""
    venc_month = period.to_timestamp().to_pydatetime()
    ref_month = datetime(ref_date.year, ref_date.month, 1)
    delta = relativedelta(venc_month, ref_month)
    months_diff = delta.years * 12 + delta.months
    return f"M{months_diff:+}"

# --- FUNÇÃO PRINCIPAL DE GERAÇÃO DA TABELA ---

def generate_due_curve_heatmap(df_input: pd.DataFrame, segment_column: str, segment_friendly_name: str, ref_date: date) -> str:
    """
    Gera uma tabela HTML com a curva de vencimentos completa e uma análise de gaps.
    """
    print(f"---> Processando segmento: {segment_friendly_name}...")
    
    # Lógica para pular segmentos com apenas uma entrada
    unique_entries = df_input[segment_column].dropna().unique()
    if len(unique_entries) == 1 and segment_column != '_total_carteira':
        entry_name = unique_entries[0]
        message_html = f"""
        <div class="table-container">
            <h3 class="table-title">Análise por: {segment_friendly_name}</h3>
            <p class="single-entry-message">Em '{segment_friendly_name}' temos apenas uma entrada: <strong>{entry_name}</strong>. A análise detalhada deste item é refletida na tabela 'Carteira Consolidada'.</p>
        </div>
        """
        return message_html

    # 1. Criação da Tabela Dinâmica com o volume a vencer (usando a carteira toda)
    report_abs = df_input.pivot_table(
        index=segment_column,
        columns="MesVencimento",
        values="ValorPresente",
        fill_value=0,
        aggfunc='sum'
    )
    if report_abs.empty: return ""

    report_abs["Total em Carteira (R$)"] = report_abs.sum(axis=1)
    report_abs.sort_values(by="Total em Carteira (R$)", ascending=False, inplace=True)
    
    # 2. Análise de Gaps (Potenciais Adiantamentos) - Focada apenas no futuro
    gaps_analysis = []
    for index, row in report_abs.iterrows():
        meses_com_valor = row[row > 0].index.drop("Total em Carteira (R$)", errors='ignore')
        
        # Converte os períodos para números de meses relativos e filtra apenas M>=0
        meses_relativos_num = [
            relativedelta(p.to_timestamp().to_pydatetime(), datetime(ref_date.year, ref_date.month, 1)).months + 
            relativedelta(p.to_timestamp().to_pydatetime(), datetime(ref_date.year, ref_date.month, 1)).years * 12 
            for p in meses_com_valor
        ]
        meses_futuros_num = sorted([m for m in meses_relativos_num if m >= 0])
        
        if len(meses_futuros_num) < 2: continue
        
        seq_esperada = set(range(min(meses_futuros_num), max(meses_futuros_num) + 1))
        meses_faltantes_num = sorted(list(seq_esperada - set(meses_futuros_num)))
        
        if meses_faltantes_num:
            gaps_encontrados = [f"M+{m}" for m in meses_faltantes_num]
            gaps_analysis.append({"Segmento": index, "Meses Futuros Faltantes (Potenciais Adiantamentos)": ", ".join(gaps_encontrados)})

    gaps_df = pd.DataFrame(gaps_analysis)
    
    report_final = report_abs.reset_index()
    report_final.rename(columns={segment_column: segment_friendly_name}, inplace=True)

    # 3. Lógica de Renomeação e Ordenação de Colunas
    period_cols = [c for c in report_final.columns if isinstance(c, pd.Period)]
    period_to_relative_map = {p: convert_period_to_relative_month(p, ref_date) for p in period_cols}
    relative_to_html_map = {
        relative: f"{relative}<br><span class='sub-header'>{period.strftime('%Y-%m')}</span>"
        for period, relative in period_to_relative_map.items()
    }
    report_final.rename(columns=period_to_relative_map, inplace=True)
    
    relative_month_cols = [c for c in report_final.columns if isinstance(c, str) and c.startswith('M')]
    
    # [ALTERADO] Filtra para mostrar M-x até M+1
    months_to_show = [c for c in relative_month_cols if int(c.replace('M','').replace('+','')) <= 1]
    
    sort_key = lambda col: int(col.replace('M', '').replace('+', ''))
    sorted_relative_cols = sorted(months_to_show, key=sort_key, reverse=True) # M+1, M+0, M-1...
    
    final_cols_order = [segment_friendly_name] + sorted_relative_cols + ["Total em Carteira (R$)"]
    report_final = report_final[[col for col in final_cols_order if col in report_final.columns]]

    report_final.rename(columns=relative_to_html_map, inplace=True)
    sorted_html_cols = [relative_to_html_map.get(c, c) for c in sorted_relative_cols]

    # 4. Estilização e Geração do HTML
    def format_br_number(val):
        if pd.isna(val) or val == 0: return ""
        return f"{val:,.2f}".replace(',', 'X').replace('.', ',').replace('X', '.')

    styler = (
        report_final.style
        .set_table_attributes('class="dataframe"')
        .set_properties(**{"padding": "8px", "text-align": "right", "min-width": "80px"})
        .set_table_styles([
            {"selector": "th, td", "props": [("border", "1px solid #ccc")]},
            {"selector": "th", "props": [("background-color", "#163f3f"), ("color", "#FFFFFF"), ("text-align", "center"), ("font-weight", "bold"), ("line-height", "1.3")]},
            {"selector": "thead th", "props": [("position", "sticky"), ("top", "0"), ("z-index", "1")]},
            {"selector": "tbody td:first-child", "props": [("position", "sticky"), ("left", "0"), ("background-color", "#f9f9f9"), ("text-align", "left"), ("font-weight", "bold"), ("white-space", "nowrap")]},
            {"selector": "thead th:first-child", "props": [("position", "sticky"), ("left", "0"), ("z-index", "2")]}
        ], overwrite=False)
        .format({col: format_br_number for col in sorted_html_cols + ["Total em Carteira (R$)"]})
        .background_gradient(cmap="YlGn", subset=sorted_html_cols, axis=None) # Verde para futuro
        .background_gradient(cmap="Greys", subset=["Total em Carteira (R$)"], axis=None)
        .hide(axis="index")
    )
    
    # Monta o HTML final para este segmento
    html_final_segmento = f"""
    <div class="table-container">
        <h3 class="table-title">Curva de Vencimentos (Aging) por: {segment_friendly_name}</h3>
        {styler.to_html()}
    """
    
    if not gaps_df.empty:
        html_final_segmento += f"""
        <h4 class="gaps-title">Análise de Adiantamentos (Gaps na Sequência Futura)</h4>
        {gaps_df.to_html(classes='dataframe dataframe-gaps', index=False)}
        """
        
    html_final_segmento += "</div>"
    return html_final_segmento


# --- BLOCO PRINCIPAL DE EXECUÇÃO ---

print("\n" + "="*80)
print("INICIANDO A GERAÇÃO DO RELATÓRIO DE CURVA DE VENCIMENTOS")
print(f"Data de referência para os dados: {ref_date_obj.strftime('%d/%m/%Y')}")
print("="*80)

html_tabelas = []
for nome_amigavel, nome_coluna in segmentos_para_analise.items():
    tabela_html = generate_due_curve_heatmap(df_report, nome_coluna, nome_amigavel, ref_date_obj)
    if tabela_html:
        html_tabelas.append(tabela_html)

# --- MONTAGEM DO ARQUIVO HTML FINAL ---

if html_tabelas:
    html_css = """
    <style>
        body { font-family: "Segoe UI", Arial, sans-serif; background-color: #f4f4f4; margin: 0; padding: 20px; }
        .main-container { max-width: 95%; margin: auto; background-color: #ffffff; padding: 25px; border-radius: 8px; box-shadow: 0 4px 8px rgba(0,0,0,0.1); }
        .report-header { text-align: center; border-bottom: 2px solid #163f3f; padding-bottom: 15px; margin-bottom: 30px; }
        .report-header h1 { color: #163f3f; margin: 0; }
        .report-header p { color: #555; font-size: 1.1em; }
        .table-container { margin-bottom: 50px; }
        .table-title { color: #0e5d5f; border-bottom: 2px solid #76c6c5; padding-bottom: 8px; margin-bottom: 15px; }
        .dataframe { border-collapse: collapse; border-spacing: 0; width: 100%; font-size: 0.9em; }
        .dataframe td.na { background-color: transparent !important; }
        .single-entry-message { padding: 20px; background-color: #f8f9fa; border-left: 5px solid #76c6c5; margin-top: 15px; font-size: 1.1em; color: #333; }
        .sub-header { font-size: 0.8em; font-weight: normal; color: #dddddd; }
        .gaps-title { margin-top: 25px; color: #163f3f; }
        .dataframe-gaps { width: auto; margin-top: 10px; border-collapse: collapse; }
        .dataframe-gaps th, .dataframe-gaps td { border: 1px solid #ccc; padding: 6px 10px; }
        .dataframe-gaps th { background-color: #5b8c8c; color: white; }
        footer { margin-top: 40px; padding-top: 20px; border-top: 1px solid #ddd; font-size: 0.9em; color: #777; text-align: center; }
    </style>
    """
    
    footer_text = """
    <p><strong>Nota sobre a Metodologia:</strong> Este relatório analisa o <strong>Valor Presente</strong> de todas as parcelas em carteira, vencidas e a vencer.
    A tabela "Análise de Adiantamentos" infere potenciais pagamentos antecipados ao identificar meses futuros que estão em falta na sequência de vencimentos de um segmento.</p>
    """

    html_final = f"""
    <!DOCTYPE html>
    <html lang="pt-BR">
    <head>
        <meta charset="UTF-8">
        <title>Relatório de Curva de Vencimentos (Aging)</title>
        {html_css}
    </head>
    <body>
        <div class="main-container">
            <div class="report-header">
                <h1>Relatório de Curva de Vencimentos (Aging)</h1>
                <p>Data de referência dos dados: {ref_date_obj.strftime('%d/%m/%Y')}</p>
            </div>
            {''.join(html_tabelas)}
            
            <footer>
                {footer_text}
            </footer>
        </div>
    </body>
    </html>
    """

    nome_arquivo_saida = f"{date.today().strftime('%Y-%m-%d')}-relatorio_curva_vencimentos.html"
    caminho_arquivo_saida = os.path.join(output_path, nome_arquivo_saida)
    
    with open(caminho_arquivo_saida, "w", encoding="utf-8") as f:
        f.write(html_final)
    print("\n" + "="*80)
    print("PROCESSO FINALIZADO COM SUCESSO!")
    print(f"Relatório de Curva de Vencimentos gerado em: {caminho_arquivo_saida}")
    print("="*80)

else:
    print("\nNenhuma tabela foi gerada. O relatório final está vazio.")




INICIANDO A GERAÇÃO DO RELATÓRIO DE CURVA DE VENCIMENTOS
Data de referência para os dados: 05/08/2025
---> Processando segmento: Carteira Consolidada...
---> Processando segmento: Cedentes...
---> Processando segmento: Originadores...
---> Processando segmento: Promotoras...
---> Processando segmento: Produtos...
---> Processando segmento: Convênios...
---> Processando segmento: Situação...
---> Processando segmento: UF...
---> Processando segmento: CAPAG...

PROCESSO FINALIZADO COM SUCESSO!
Relatório de Curva de Vencimentos gerado em: C:\Users\Leo\Desktop\Porto_Real\portoauto\src\originadores\output\2025-08-11-relatorio_curva_vencimentos.html


In [22]:
#%%
# =============================================================================
# CÉLULA DE GERAÇÃO DO RELATÓRIO DE PERFORMANCE DE VENCIMENTOS (AGING)
# =============================================================================
# Descrição: Este script utiliza a lógica de análise de performance baseada no
# "mês de pico". Ele calcula o volume nominal de cada mês como um percentual
# do mês de maior volume nominal da história, usando a visão de "aging".

import pandas as pd
import numpy as np
import os
from datetime import datetime, date
from dateutil.relativedelta import relativedelta

# --- CONFIGURAÇÕES INICIAIS ---

# Verifica se o DataFrame principal (df_final2) existe.
if 'df_final2' not in locals() or df_final2.empty:
    raise NameError("O DataFrame 'df_final2' não foi encontrado ou está vazio. Por favor, execute as células anteriores primeiro.")

# Adiciona uma coluna para permitir a análise da carteira total
df_report = df_final2.copy()
df_report['_total_carteira'] = 'Carteira Consolidada'
df_report['MesVencimento'] = pd.to_datetime(df_report['DataVencimento']).dt.to_period('M')

# Define os segmentos que queremos analisar
segmentos_para_analise = {
    'Carteira Consolidada': '_total_carteira',
    'Cedentes': 'Cedente',
    'Originadores': 'Originador',
    'Promotoras': 'Promotora',
    'Produtos': 'Produto',
    'Convênios': 'Convênio',
    'Situação': 'Situacao',
    'UF': 'UF',
    'CAPAG': 'CAPAG',
    'Pagamento Parcial': 'PagamentoParcial',
    'Tem Muitos Contratos':'_MuitosContratos',
    'Tem Muitos Entes':'_MuitosEntes'
}

# Define a data de referência
ref_date_obj = df_report['DataGeracao'].max().date()

# Define o caminho de saída
output_path = r'C:\Users\Leo\Desktop\Porto_Real\portoauto\src\originadores\output'
os.makedirs(output_path, exist_ok=True)


# --- FUNÇÕES AUXILIARES ---

def convert_period_to_relative_month(period: pd.Period, ref_date: date) -> str:
    """Converte um objeto Period para um formato de mês relativo (ex: M-1, M+0)."""
    venc_month = period.to_timestamp().to_pydatetime()
    ref_month = datetime(ref_date.year, ref_date.month, 1)
    delta = relativedelta(venc_month, ref_month)
    months_diff = delta.years * 12 + delta.months
    return f"M{months_diff:+}"

# --- FUNÇÃO PRINCIPAL DE GERAÇÃO DA TABELA ---

def generate_performance_heatmap(df_input: pd.DataFrame, segment_column: str, segment_friendly_name: str, ref_date: date) -> str:
    """
    Gera uma tabela HTML de performance de vencimentos para um segmento.
    """
    print(f"---> Processando segmento: {segment_friendly_name}...")
    
    # Lógica para pular segmentos com apenas uma entrada
    unique_entries = df_input[segment_column].dropna().unique()
    if len(unique_entries) == 1 and segment_column != '_total_carteira':
        entry_name = unique_entries[0]
        print(f"     [INFO] Segmento '{segment_friendly_name}' tem apenas uma entrada: '{entry_name}'. Gerando mensagem simples.")
        message_html = f"""
        <div class="table-container">
            <h3 class="table-title">Análise por: {segment_friendly_name}</h3>
            <p class="single-entry-message">Em '{segment_friendly_name}' temos apenas uma entrada: <strong>{entry_name}</strong>. A análise detalhada deste item é refletida na tabela 'Carteira Consolidada'.</p>
        </div>
        """
        return message_html

    # 1. Criação da Tabela Dinâmica com Valor Nominal (sobre a carteira completa)
    report_abs = df_input.pivot_table(
        index=segment_column,
        columns="MesVencimento",
        values="ValorNominal",
        fill_value=0,
        aggfunc='sum'
    )
    if report_abs.empty: return ""

    # 2. Cálculo de Pico Dinâmico
    peak_value = report_abs.max().max()
    if peak_value == 0: return ""
    print(f"     Pico de Valor Nominal para este segmento: R$ {peak_value:,.2f}")

    # 3. Cálculo dos Totais de VP e VL
    summary_totals = df_input.groupby(segment_column)[['_ValorLiquido', 'ValorPresente']].sum()
    summary_totals.rename(columns={'_ValorLiquido': 'Valor Líquido Total (R$)', 'ValorPresente': 'Valor Presente Total (R$)'}, inplace=True)

    # 4. Cálculo da Performance
    report_performance = (1 - (report_abs / peak_value)) * 100
    report_performance["Total Nominal (R$)"] = report_abs.sum(axis=1)
    report_performance = report_performance.join(summary_totals)
    report_performance.sort_values(by="Total Nominal (R$)", ascending=False, inplace=True)
    report_performance.index.name = segment_friendly_name
    report_performance.reset_index(inplace=True)

    # 5. Lógica de Renomeação e Ordenação de Colunas
    period_cols = [c for c in report_performance.columns if isinstance(c, pd.Period)]
    period_to_relative_map = {p: convert_period_to_relative_month(p, ref_date) for p in period_cols}
    relative_to_html_map = {
        relative: f"{relative}<br><span class='sub-header'>{period.strftime('%Y-%m')}</span>"
        for period, relative in period_to_relative_map.items()
    }
    report_performance.rename(columns=period_to_relative_map, inplace=True)
    
    relative_month_cols = [c for c in report_performance.columns if isinstance(c, str) and c.startswith('M')]
    
    # [CORREÇÃO] Volta a incluir M+0 e M+1 na análise
    months_to_show = [c for c in relative_month_cols if int(c.replace('M','').replace('+','')) <= 1]
    
    sort_key = lambda col: int(col.replace('M', '').replace('+', ''))
    sorted_relative_cols = sorted(months_to_show, key=sort_key, reverse=True)
    
    final_cols_order = [
        segment_friendly_name, 
        'Valor Presente Total (R$)', 
        'Valor Líquido Total (R$)'
    ] + sorted_relative_cols + ["Total Nominal (R$)"]
    
    final_cols_order = [col for col in final_cols_order if col in report_performance.columns]
    report_final = report_performance[final_cols_order]

    report_final.rename(columns=relative_to_html_map, inplace=True)
    sorted_html_cols = [relative_to_html_map.get(c, c) for c in sorted_relative_cols]

    # 6. Estilização e Geração do HTML
    def format_br_percent(val):
        if pd.isna(val): return ""
        return f"{val:.2f}%".replace('.', ',')
        
    def format_br_number(val):
        if pd.isna(val): return ""
        return f"{val:,.2f}".replace(',', 'X').replace('.', ',').replace('X', '.')

    styler = (
        report_final.style
        .set_table_attributes('class="dataframe"')
        .set_properties(**{"padding": "8px", "text-align": "right", "min-width": "80px"})
        .set_table_styles([
            {"selector": "th, td", "props": [("border", "1px solid #ccc")]},
            {"selector": "th", "props": [("background-color", "#163f3f"), ("color", "#FFFFFF"), ("text-align", "center"), ("font-weight", "bold"), ("line-height", "1.3")]},
            {"selector": "thead th", "props": [("position", "sticky"), ("top", "0"), ("z-index", "1")]},
            {"selector": "tbody td:first-child", "props": [("position", "sticky"), ("left", "0"), ("background-color", "#f9f9f9"), ("text-align", "left"), ("font-weight", "bold"), ("white-space", "nowrap")]},
            {"selector": "thead th:first-child", "props": [("position", "sticky"), ("left", "0"), ("z-index", "2")]}
        ], overwrite=False)
        .format({
            **{col: format_br_percent for col in sorted_html_cols}, 
            **{"Total Nominal (R$)": format_br_number, 
               'Valor Presente Total (R$)': format_br_number, 
               'Valor Líquido Total (R$)': format_br_number}
        })
        .background_gradient(cmap="RdYlGn", subset=sorted_html_cols, vmin=0, vmax=100, axis=None)
        .background_gradient(cmap="Greys", subset=["Total Nominal (R$)", 'Valor Presente Total (R$)', 'Valor Líquido Total (R$)'], axis=None)
        .hide(axis="index")
    )
    
    header_tabela = f"""
    <div class="table-container">
        <h3 class="table-title">Performance de Vencimentos por: {segment_friendly_name}</h3>
        {styler.to_html()}
    </div>
    """
    return header_tabela


# --- BLOCO PRINCIPAL DE EXECUÇÃO ---

print("\n" + "="*80)
print("INICIANDO A GERAÇÃO DO RELATÓRIO DE PERFORMANCE DE VENCIMENTOS")
print(f"Data de referência para os dados: {ref_date_obj.strftime('%d/%m/%Y')}")
print("="*80)

html_tabelas = []
for nome_amigavel, nome_coluna in segmentos_para_analise.items():
    tabela_html = generate_performance_heatmap(df_report, nome_coluna, nome_amigavel, ref_date_obj)
    if tabela_html:
        html_tabelas.append(tabela_html)

# --- MONTAGEM DO ARQUIVO HTML FINAL ---

if html_tabelas:
    html_css = """
    <style>
        body { font-family: "Segoe UI", Arial, sans-serif; background-color: #f4f4f4; margin: 0; padding: 20px; }
        .main-container { max-width: 95%; margin: auto; background-color: #ffffff; padding: 25px; border-radius: 8px; box-shadow: 0 4px 8px rgba(0,0,0,0.1); }
        .report-header { text-align: center; border-bottom: 2px solid #163f3f; padding-bottom: 15px; margin-bottom: 30px; }
        .report-header h1 { color: #163f3f; margin: 0; }
        .report-header p { color: #555; font-size: 1.1em; }
        .table-container { margin-bottom: 50px; }
        .table-title { color: #0e5d5f; border-bottom: 2px solid #76c6c5; padding-bottom: 8px; margin-bottom: 15px; }
        .dataframe { border-collapse: collapse; border-spacing: 0; width: 100%; font-size: 0.9em; }
        .dataframe td.na { background-color: transparent !important; }
        .single-entry-message {
            padding: 20px;
            background-color: #f8f9fa;
            border-left: 5px solid #76c6c5;
            margin-top: 15px;
            font-size: 1.1em;
            color: #333;
        }
        .sub-header {
            font-size: 0.8em;
            font-weight: normal;
            color: #dddddd;
        }
        footer {
            margin-top: 40px;
            padding-top: 20px;
            border-top: 1px solid #ddd;
            font-size: 0.9em;
            color: #777;
            text-align: center;
        }
    </style>
    """
    
    footer_text = """
    <p><strong>Nota sobre a Metodologia:</strong> Este relatório utiliza a lógica de análise de performance baseada no "mês de pico". 
    Ele calcula o volume nominal de cada mês (passado e futuro) como um percentual do mês de maior volume nominal da história para aquele segmento, usando a visão de "aging".</p>
    """

    html_final = f"""
    <!DOCTYPE html>
    <html lang="pt-BR">
    <head>
        <meta charset="UTF-8">
        <title>Relatório de Performance de Vencimentos (Aging)</title>
        {html_css}
    </head>
    <body>
        <div class="main-container">
            <div class="report-header">
                <h1>Relatório de Performance de Vencimentos</h1>
                <p>Data de referência dos dados: {ref_date_obj.strftime('%d/%m/%Y')}</p>
            </div>
            {''.join(html_tabelas)}
            
            <footer>
                {footer_text}
            </footer>
        </div>
    </body>
    </html>
    """

    nome_arquivo_saida = f"{date.today().strftime('%Y-%m-%d')}-relatorio_performance_vencimentos_aging.html"
    caminho_arquivo_saida = os.path.join(output_path, nome_arquivo_saida)
    
    with open(caminho_arquivo_saida, "w", encoding="utf-8") as f:
        f.write(html_final)
    print("\n" + "="*80)
    print("PROCESSO FINALIZADO COM SUCESSO!")
    print(f"Relatório de Performance (Aging) gerado em: {caminho_arquivo_saida}")
    print("="*80)

else:
    print("\nNenhuma tabela foi gerada. O relatório final está vazio.")




INICIANDO A GERAÇÃO DO RELATÓRIO DE PERFORMANCE DE VENCIMENTOS
Data de referência para os dados: 05/08/2025
---> Processando segmento: Carteira Consolidada...
     Pico de Valor Nominal para este segmento: R$ 1,678,416.74


C:\Users\Leo\AppData\Local\Temp\ipykernel_31472\164892374.py:134: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  report_final.rename(columns=relative_to_html_map, inplace=True)


---> Processando segmento: Cedentes...
     [INFO] Segmento 'Cedentes' tem apenas uma entrada: 'BMP MONEY PLUS SOCIEDADE DE CREDITO DIRETO S.A'. Gerando mensagem simples.
---> Processando segmento: Originadores...
     [INFO] Segmento 'Originadores' tem apenas uma entrada: 'StarCard'. Gerando mensagem simples.
---> Processando segmento: Promotoras...
     Pico de Valor Nominal para este segmento: R$ 258,210.27


C:\Users\Leo\AppData\Local\Temp\ipykernel_31472\164892374.py:134: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  report_final.rename(columns=relative_to_html_map, inplace=True)


---> Processando segmento: Produtos...
     Pico de Valor Nominal para este segmento: R$ 469,550.56


...

C:\Users\Leo\AppData\Local\Temp\ipykernel_31472\164892374.py:134: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  report_final.rename(columns=relative_to_html_map, inplace=True)


---> Processando segmento: Pagamento Parcial...
     Pico de Valor Nominal para este segmento: R$ 1,678,416.74


C:\Users\Leo\AppData\Local\Temp\ipykernel_31472\164892374.py:134: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  report_final.rename(columns=relative_to_html_map, inplace=True)


---> Processando segmento: Tem Muitos Contratos...
     Pico de Valor Nominal para este segmento: R$ 1,288,457.76
---> Processando segmento: Tem Muitos Entes...
     [INFO] Segmento 'Tem Muitos Entes' tem apenas uma entrada: 'False'. Gerando mensagem simples.

PROCESSO FINALIZADO COM SUCESSO!
Relatório de Performance (Aging) gerado em: C:\Users\Leo\Desktop\Porto_Real\portoauto\src\originadores\output\2025-08-11-relatorio_performance_vencimentos_aging.html


C:\Users\Leo\AppData\Local\Temp\ipykernel_31472\164892374.py:134: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  report_final.rename(columns=relative_to_html_map, inplace=True)


In [23]:
#%%
# =============================================================================
# CÉLULA DE GERAÇÃO DO RELATÓRIO DE VENCIMENTOS FUTUROS E ADIANTAMENTOS
# =============================================================================
# Descrição: Este script gera um relatório que analisa a curva de vencimentos
# futuros e tenta inferir adiantamentos ao detetar "gaps" (meses em falta)
# na sequência de parcelas a vencer.

import pandas as pd
import numpy as np
import os
from datetime import datetime, date
from dateutil.relativedelta import relativedelta

# --- CONFIGURAÇÕES INICIAIS ---

# Verifica se o DataFrame principal (df_final2) existe.
if 'df_final2' not in locals() or df_final2.empty:
    raise NameError("O DataFrame 'df_final2' não foi encontrado ou está vazio. Por favor, execute as células anteriores primeiro.")

# Adiciona uma coluna para permitir a análise da carteira total
df_report = df_final2.copy()
df_report['_total_carteira'] = 'Carteira Consolidada'
df_report['MesVencimento'] = pd.to_datetime(df_report['DataVencimento']).dt.to_period('M')

# Define os segmentos que queremos analisar
segmentos_para_analise = {
    'Carteira Consolidada': '_total_carteira',
    'Cedentes': 'Cedente',
    'Originadores': 'Originador',
    'Promotoras': 'Promotora',
    'Produtos': 'Produto',
    'Convênios': 'Convênio',
    'Situação': 'Situacao',
    'UF': 'UF',
    'CAPAG': 'CAPAG'
}

# Define a data de referência
ref_date_obj = df_report['DataGeracao'].max().date()

# Define o caminho de saída
output_path = r'C:\Users\Leo\Desktop\Porto_Real\portoauto\src\originadores\output'
os.makedirs(output_path, exist_ok=True)


# --- FUNÇÕES AUXILIARES ---

def convert_period_to_relative_month(period: pd.Period, ref_date: date) -> str:
    """Converte um objeto Period para um formato de mês relativo (ex: M+1, M+2)."""
    venc_month = period.to_timestamp().to_pydatetime()
    ref_month = datetime(ref_date.year, ref_date.month, 1)
    delta = relativedelta(venc_month, ref_month)
    months_diff = delta.years * 12 + delta.months
    return f"M{months_diff:+}"

# --- FUNÇÃO PRINCIPAL DE GERAÇÃO DA TABELA ---

def generate_future_due_heatmap(df_input: pd.DataFrame, segment_column: str, segment_friendly_name: str, ref_date: date) -> str:
    """
    Gera uma tabela HTML com a curva de vencimentos futuros e uma análise de gaps.
    """
    print(f"---> Processando segmento: {segment_friendly_name}...")
    
    # Lógica para pular segmentos com apenas uma entrada
    unique_entries = df_input[segment_column].dropna().unique()
    if len(unique_entries) == 1 and segment_column != '_total_carteira':
        entry_name = unique_entries[0]
        message_html = f"""
        <div class="table-container">
            <h3 class="table-title">Análise por: {segment_friendly_name}</h3>
            <p class="single-entry-message">Em '{segment_friendly_name}' temos apenas uma entrada: <strong>{entry_name}</strong>. A análise detalhada deste item é refletida na tabela 'Carteira Consolidada'.</p>
        </div>
        """
        return message_html

    # 1. Filtra apenas as parcelas COM VENCIMENTO NO FUTURO (incluindo o mês atual)
    ref_timestamp = pd.to_datetime(ref_date)
    future_due_df = df_input[pd.to_datetime(df_input['DataVencimento']).dt.normalize() >= ref_timestamp].copy()
    if future_due_df.empty:
        print(f"     [INFO] Nenhuma parcela com vencimento no futuro encontrada para '{segment_friendly_name}'.")
        return ""

    # 2. Criação da Tabela Dinâmica com o volume a vencer
    report_abs = future_due_df.pivot_table(
        index=segment_column,
        columns="MesVencimento",
        values="ValorPresente",
        fill_value=0,
        aggfunc='sum'
    )
    if report_abs.empty: return ""

    report_abs["Total a Vencer (R$)"] = report_abs.sum(axis=1)
    report_abs.sort_values(by="Total a Vencer (R$)", ascending=False, inplace=True)
    report_abs.index.name = segment_friendly_name
    report_final = report_abs.reset_index()

    # 3. [NOVO] Análise de Gaps (Potenciais Adiantamentos)
    gaps_analysis = []
    for index, row in report_abs.iterrows():
        # Pega as colunas de meses que têm valor > 0 para esta linha
        meses_com_valor = row[row > 0].index.drop("Total a Vencer (R$)", errors='ignore')
        if len(meses_com_valor) < 2: continue

        # Converte os períodos para números de meses relativos
        meses_relativos_num = sorted([relativedelta(p.to_timestamp(), ref_timestamp).months + relativedelta(p.to_timestamp(), ref_timestamp).years * 12 for p in meses_com_valor])
        
        # Encontra os "buracos" na sequência
        gaps_encontrados = []
        for i in range(len(meses_relativos_num) - 1):
            diff = meses_relativos_num[i+1] - meses_relativos_num[i]
            if diff > 1:
                for j in range(1, diff):
                    mes_faltante_num = meses_relativos_num[i] + j
                    gaps_encontrados.append(f"M+{mes_faltante_num}")
        
        if gaps_encontrados:
            gaps_analysis.append({"Segmento": index, "Meses Faltantes (Potenciais Adiantamentos)": ", ".join(gaps_encontrados)})

    gaps_df = pd.DataFrame(gaps_analysis)

    # 4. Lógica de Renomeação e Ordenação de Colunas
    period_cols = [c for c in report_final.columns if isinstance(c, pd.Period)]
    period_to_relative_map = {p: convert_period_to_relative_month(p, ref_date) for p in period_cols}
    relative_to_html_map = {
        relative: f"{relative}<br><span class='sub-header'>{period.strftime('%Y-%m')}</span>"
        for period, relative in period_to_relative_map.items()
    }
    report_final.rename(columns=period_to_relative_map, inplace=True)
    
    relative_month_cols = [c for c in report_final.columns if isinstance(c, str) and c.startswith('M')]
    sort_key = lambda col: int(col.replace('M', '').replace('+', ''))
    sorted_relative_cols = sorted(relative_month_cols, key=sort_key) # Ordem crescente M+0, M+1...
    
    final_cols_order = [segment_friendly_name] + sorted_relative_cols + ["Total a Vencer (R$)"]
    report_final = report_final[[col for col in final_cols_order if col in report_final.columns]]

    report_final.rename(columns=relative_to_html_map, inplace=True)
    sorted_html_cols = [relative_to_html_map.get(c, c) for c in sorted_relative_cols]

    # 5. Estilização e Geração do HTML
    def format_br_number(val):
        if pd.isna(val) or val == 0: return ""
        return f"{val:,.2f}".replace(',', 'X').replace('.', ',').replace('X', '.')

    styler = (
        report_final.style
        .set_table_attributes('class="dataframe"')
        .set_properties(**{"padding": "8px", "text-align": "right", "min-width": "80px"})
        .set_table_styles([
            {"selector": "th, td", "props": [("border", "1px solid #ccc")]},
            {"selector": "th", "props": [("background-color", "#163f3f"), ("color", "#FFFFFF"), ("text-align", "center"), ("font-weight", "bold"), ("line-height", "1.3")]},
            {"selector": "thead th", "props": [("position", "sticky"), ("top", "0"), ("z-index", "1")]},
            {"selector": "tbody td:first-child", "props": [("position", "sticky"), ("left", "0"), ("background-color", "#f9f9f9"), ("text-align", "left"), ("font-weight", "bold"), ("white-space", "nowrap")]},
            {"selector": "thead th:first-child", "props": [("position", "sticky"), ("left", "0"), ("z-index", "2")]}
        ], overwrite=False)
        .format({col: format_br_number for col in sorted_html_cols + ["Total a Vencer (R$)"]})
        .background_gradient(cmap="Greens", subset=sorted_html_cols, axis=None)
        .background_gradient(cmap="Greys", subset=["Total a Vencer (R$)"], axis=None)
        .hide(axis="index")
    )
    
    # Monta o HTML final para este segmento
    html_final_segmento = f"""
    <div class="table-container">
        <h3 class="table-title">Curva de Vencimentos Futuros por: {segment_friendly_name}</h3>
        {styler.to_html()}
    """
    
    if not gaps_df.empty:
        html_final_segmento += f"""
        <h4 class="gaps-title">Análise de Adiantamentos (Gaps na Sequência Futura)</h4>
        {gaps_df.to_html(classes='dataframe dataframe-gaps', index=False)}
        """
        
    html_final_segmento += "</div>"
    return html_final_segmento


# --- BLOCO PRINCIPAL DE EXECUÇÃO ---

print("\n" + "="*80)
print("INICIANDO A GERAÇÃO DO RELATÓRIO DE VENCIMENTOS FUTUROS")
print(f"Data de referência para os dados: {ref_date_obj.strftime('%d/%m/%Y')}")
print("="*80)

html_tabelas = []
for nome_amigavel, nome_coluna in segmentos_para_analise.items():
    tabela_html = generate_future_due_heatmap(df_report, nome_coluna, nome_amigavel, ref_date_obj)
    if tabela_html:
        html_tabelas.append(tabela_html)

# --- MONTAGEM DO ARQUIVO HTML FINAL ---

if html_tabelas:
    html_css = """
    <style>
        body { font-family: "Segoe UI", Arial, sans-serif; background-color: #f4f4f4; margin: 0; padding: 20px; }
        .main-container { max-width: 95%; margin: auto; background-color: #ffffff; padding: 25px; border-radius: 8px; box-shadow: 0 4px 8px rgba(0,0,0,0.1); }
        .report-header { text-align: center; border-bottom: 2px solid #163f3f; padding-bottom: 15px; margin-bottom: 30px; }
        .report-header h1 { color: #163f3f; margin: 0; }
        .report-header p { color: #555; font-size: 1.1em; }
        .table-container { margin-bottom: 50px; }
        .table-title { color: #0e5d5f; border-bottom: 2px solid #76c6c5; padding-bottom: 8px; margin-bottom: 15px; }
        .dataframe { border-collapse: collapse; border-spacing: 0; width: 100%; font-size: 0.9em; }
        .dataframe td.na { background-color: transparent !important; }
        .single-entry-message { padding: 20px; background-color: #f8f9fa; border-left: 5px solid #76c6c5; margin-top: 15px; font-size: 1.1em; color: #333; }
        .sub-header { font-size: 0.8em; font-weight: normal; color: #dddddd; }
        .gaps-title { margin-top: 25px; color: #163f3f; }
        .dataframe-gaps { width: auto; margin-top: 10px; }
        footer { margin-top: 40px; padding-top: 20px; border-top: 1px solid #ddd; font-size: 0.9em; color: #777; text-align: center; }
    </style>
    """
    
    footer_text = """
    <p><strong>Nota sobre a Metodologia:</strong> Este relatório analisa o <strong>Valor Presente</strong> das parcelas com vencimento a partir do mês de referência. 
    A tabela "Análise de Adiantamentos" infere potenciais pagamentos antecipados ao identificar meses futuros que estão em falta na sequência de vencimentos de um segmento.</p>
    """

    html_final = f"""
    <!DOCTYPE html>
    <html lang="pt-BR">
    <head>
        <meta charset="UTF-8">
        <title>Relatório de Vencimentos Futuros e Adiantamentos</title>
        {html_css}
    </head>
    <body>
        <div class="main-container">
            <div class="report-header">
                <h1>Relatório de Vencimentos Futuros e Adiantamentos</h1>
                <p>Data de referência dos dados: {ref_date_obj.strftime('%d/%m/%Y')}</p>
            </div>
            {''.join(html_tabelas)}
            
            <footer>
                {footer_text}
            </footer>
        </div>
    </body>
    </html>
    """

    nome_arquivo_saida = f"{date.today().strftime('%Y-%m-%d')}-relatorio_vencimentos_futuros.html"
    caminho_arquivo_saida = os.path.join(output_path, nome_arquivo_saida)
    
    with open(caminho_arquivo_saida, "w", encoding="utf-8") as f:
        f.write(html_final)
    print("\n" + "="*80)
    print("PROCESSO FINALIZADO COM SUCESSO!")
    print(f"Relatório de Vencimentos Futuros gerado em: {caminho_arquivo_saida}")
    print("="*80)

else:
    print("\nNenhuma tabela foi gerada. O relatório final está vazio.")




INICIANDO A GERAÇÃO DO RELATÓRIO DE VENCIMENTOS FUTUROS
Data de referência para os dados: 05/08/2025
---> Processando segmento: Carteira Consolidada...
---> Processando segmento: Cedentes...
---> Processando segmento: Originadores...
---> Processando segmento: Promotoras...
---> Processando segmento: Produtos...
---> Processando segmento: Convênios...
---> Processando segmento: Situação...
---> Processando segmento: UF...
---> Processando segmento: CAPAG...

PROCESSO FINALIZADO COM SUCESSO!
Relatório de Vencimentos Futuros gerado em: C:\Users\Leo\Desktop\Porto_Real\portoauto\src\originadores\output\2025-08-11-relatorio_vencimentos_futuros.html


In [37]:
print("--- Verificação Direta no DataFrame ---")
print("Analisando se PagamentoParcial == 'SIM' tem ValorNominal == 0 após Junho de 2025...")

# 1. Definir a data de corte para o final de Junho de 2025
data_corte = pd.to_datetime('2025-06-30')

# 2. Criar o filtro com as duas condições
filtro_verificacao = (
    (df_final2['PagamentoParcial'] == 'SIM') &
    (df_final2['DataVencimento'] > data_corte)
)

# 3. Aplicar o filtro no DataFrame
df_filtrado = df_final2.loc[filtro_verificacao]

# 4. Analisar o resultado
if df_filtrado.empty:
    print("\n[RESULTADO]: O DataFrame filtrado está VAZIO.")
    print("Isso confirma que NÃO EXISTE NENHUM título com PagamentoParcial='SIM' e vencimento após 30/06/2025.")

else:
    print(f"\n[RESULTADO]: Foram encontrados {len(df_filtrado)} títulos que atendem às condições.")
    # Usar .describe() é uma forma rápida de checar os valores
    resumo_valor_nominal = df_filtrado['ValorNominal'].describe()

    print("\nResumo estatístico do 'ValorNominal' para os títulos encontrados:")
    print(resumo_valor_nominal)

    # Verificação final baseada no valor máximo
    if resumo_valor_nominal['max'] == 0:
        print("\n[CONFIRMAÇÃO]: Sim, o valor máximo encontrado é 0. Todos os títulos nessa condição têm ValorNominal igual a zero.")
    else:
        print("\n[ALERTA]: Não! Foi encontrado pelo menos um título com ValorNominal diferente de zero.")
        # Mostra os registros com ValorNominal > 0 para análise
        display(df_filtrado[df_filtrado['ValorNominal'] > 0][['PagamentoParcial', 'DataVencimento', 'ValorNominal']])

--- Verificação Direta no DataFrame ---
Analisando se PagamentoParcial == 'SIM' tem ValorNominal == 0 após Junho de 2025...

[RESULTADO]: O DataFrame filtrado está VAZIO.
Isso confirma que NÃO EXISTE NENHUM título com PagamentoParcial='SIM' e vencimento após 30/06/2025.


#### Versão com % de vencidos

In [17]:
#%%
## Versão: 1-concentr vencidos
import pandas as pd
import numpy as np
import os
from datetime import datetime, date
from dateutil.relativedelta import relativedelta

#! exec as ceclulas anteriores para ter o df_final2 salvo 
#TODO
output_path = r'C:\Users\Leo\Desktop\Porto_Real\portoauto\src\originadores\output'
""" Saída """
os.makedirs(output_path, exist_ok=True)

df_report = df_final2.copy()
df_report['_total_carteira'] = 'Carteira Consolidada'
df_report['MesVencimento'] = pd.to_datetime(df_report['DataVencimento']).dt.to_period('M')

segmentos_para_analise = {
    'Carteira Consolidada': '_total_carteira',
    'Cedentes': 'Cedente',
    'Originadores': 'Originador',
    'Promotoras': 'Promotora',
    'Produtos': 'Produto',
    'Convênios': 'Convênio',
    'Situação': 'Situacao',
    'UF': 'UF',
    'CAPAG': 'CAPAG',
    'Pagamento Parcial': 'PagamentoParcial',
    'Tem Muitos Contratos':'_MuitosContratos',
    'Tem Muitos Entes':'_MuitosEntes'
}

ref_date_obj = df_report['DataGeracao'].max().date()





def convert_period_to_relative_month(period: pd.Period, ref_date: date) -> str:
    venc_month = period.to_timestamp().to_pydatetime()
    ref_month = datetime(ref_date.year, ref_date.month, 1)
    delta = relativedelta(venc_month, ref_month)
    months_diff = delta.years * 12 + delta.months
    return f"M{months_diff:+}"


def generate_performance_heatmap(df_input: pd.DataFrame, segment_column: str, segment_friendly_name: str, ref_date: date) -> str:
    print(f"---> Processando segmento: {segment_friendly_name}...")
    
    unique_entries = df_input[segment_column].dropna().unique()
    if len(unique_entries) == 1 and segment_column != '_total_carteira':
        entry_name = unique_entries[0]
        print(f"     [INFO] Segmento '{segment_friendly_name}' tem apenas uma entrada: '{entry_name}'. Gerando mensagem simples.")
        message_html = f"""
        <div class="table-container">
            <h3 class="table-title">Análise por: {segment_friendly_name}</h3>
            <p class="single-entry-message">Em '{segment_friendly_name}' temos apenas uma entrada: <strong>{entry_name}</strong>. A análise detalhada deste item é refletida na tabela 'Carteira Consolidada'.</p>
        </div>
        """
        return message_html

    ref_timestamp = pd.to_datetime(ref_date)
    venc = df_input[pd.to_datetime(df_input['DataVencimento']).dt.normalize() < ref_timestamp].copy()
    if venc.empty:
        print(f"     [INFO] Nenhum item vencido encontrado para o segmento '{segment_friendly_name}'.")
        return ""

    report_abs = venc.pivot_table(
        index=segment_column,
        columns="MesVencimento",
        values="ValorPresente",
        fill_value=0,
        aggfunc='sum'
    )
    if report_abs.empty: return ""

    report_abs["Total Vencido (R$)"] = report_abs.sum(axis=1)
    report_abs = report_abs[report_abs["Total Vencido (R$)"] > 0].copy()
    if report_abs.empty: return ""

    report_concentration_pct = report_abs.drop(columns="Total Vencido (R$)").div(report_abs["Total Vencido (R$)"], axis=0) * 100
    report_performance = 100 - report_concentration_pct
    
    report_final = report_performance.join(report_abs[["Total Vencido (R$)"]])
    report_final.sort_values(by="Total Vencido (R$)", ascending=False, inplace=True)
    report_final.index.name = segment_friendly_name
    report_final.reset_index(inplace=True)

    period_cols = [c for c in report_final.columns if isinstance(c, pd.Period)]
    period_to_relative_map = {p: convert_period_to_relative_month(p, ref_date) for p in period_cols}
    relative_to_html_map = {
        relative: f"{relative}<br><span class='sub-header'>{period.strftime('%Y-%m')}</span>"
        for period, relative in period_to_relative_map.items()
    }
    report_final.rename(columns=period_to_relative_map, inplace=True)
    
    relative_month_cols = [c for c in report_final.columns if isinstance(c, str) and c.startswith('M')]
    
    # obs: removi o M+0 da análise, mostrando apenas meses passados.
    past_months = [c for c in relative_month_cols if int(c.replace('M','').replace('+','')) < 0]
    
    sort_key = lambda col: int(col.replace('M', '').replace('+', ''))
    sorted_relative_cols = sorted(past_months, key=sort_key)
    
    final_cols_order = [segment_friendly_name] + sorted_relative_cols + ["Total Vencido (R$)"]
    report_final = report_final[final_cols_order]

    report_final.rename(columns=relative_to_html_map, inplace=True)
    sorted_html_cols = [relative_to_html_map.get(c, c) for c in sorted_relative_cols]

    # obs:  se tiver vazia, vai colocar zero
    def format_br_percent(val):
        if pd.isna(val): return ""
        return f"{val:.2f}%".replace('.', ',')
        
    def format_br_number(val):
        if pd.isna(val): return ""
        return f"{val:,.2f}".replace(',', 'X').replace('.', ',').replace('X', '.')

    styler = (
        report_final.style
        .set_table_attributes('class="dataframe"')
        .set_properties(**{"padding": "8px", "text-align": "right", "min-width": "80px"})
        .set_table_styles([
            {"selector": "th, td", "props": [("border", "1px solid #ccc")]},
            {"selector": "th", "props": [("background-color", "#163f3f"), ("color", "#FFFFFF"), ("text-align", "center"), ("font-weight", "bold"), ("line-height", "1.3")]},
            {"selector": "thead th", "props": [("position", "sticky"), ("top", "0"), ("z-index", "1")]},
            {"selector": "tbody td:first-child", "props": [("position", "sticky"), ("left", "0"), ("background-color", "#f9f9f9"), ("text-align", "left"), ("font-weight", "bold"), ("white-space", "nowrap")]},
            {"selector": "thead th:first-child", "props": [("position", "sticky"), ("left", "0"), ("z-index", "2")]}
        ], overwrite=False)
        .format({col: format_br_percent for col in sorted_html_cols} | {"Total Vencido (R$)": format_br_number})
        .background_gradient(cmap="RdYlGn", subset=sorted_html_cols, vmin=0, vmax=100, axis=None)
        .background_gradient(cmap="Greys", subset=["Total Vencido (R$)"], axis=None)
        .hide(axis="index")
    )
    
    header_tabela = f"""
    <div class="table-container">
        <h3 class="table-title">Performance de Pagamento (Aging) por: {segment_friendly_name}</h3>
        {styler.to_html()}
    </div>
    """
    return header_tabela


# public static void main(String[] args) {

print("\n" + "="*80)
print("INICIANDO A GERAÇÃO DO RELATÓRIO DE PERFORMANCE DE PAGAMENTO")
print(f"Data de referência para os dados: {ref_date_obj.strftime('%d/%m/%Y')}")
print("="*80)

html_tabelas = []
for nome_amigavel, nome_coluna in segmentos_para_analise.items():
    tabela_html = generate_performance_heatmap(df_report, nome_coluna, nome_amigavel, ref_date_obj)
    if tabela_html:
        html_tabelas.append(tabela_html)


if html_tabelas:
    html_css = """
    <style>
        body { font-family: "Segoe UI", Arial, sans-serif; background-color: #f4f4f4; margin: 0; padding: 20px; }
        .main-container { max-width: 95%; margin: auto; background-color: #ffffff; padding: 25px; border-radius: 8px; box-shadow: 0 4px 8px rgba(0,0,0,0.1); }
        .report-header { text-align: center; border-bottom: 2px solid #163f3f; padding-bottom: 15px; margin-bottom: 30px; }
        .report-header h1 { color: #163f3f; margin: 0; }
        .report-header p { color: #555; font-size: 1.1em; }
        .table-container { margin-bottom: 50px; }
        .table-title { color: #0e5d5f; border-bottom: 2px solid #76c6c5; padding-bottom: 8px; margin-bottom: 15px; }
        .dataframe { border-collapse: collapse; border-spacing: 0; width: 100%; font-size: 0.9em; }
        .dataframe td.na { background-color: transparent !important; }
        .single-entry-message {
            padding: 20px;
            background-color: #f8f9fa;
            border-left: 5px solid #76c6c5;
            margin-top: 15px;
            font-size: 1.1em;
            color: #333;
        }
        .sub-header {
            font-size: 0.8em;
            font-weight: normal;
            color: #dddddd;
        }
        footer {
            margin-top: 40px;
            padding-top: 20px;
            border-top: 1px solid #ddd;
            font-size: 0.9em;
            color: #777;
            text-align: center;
        }
    </style>
    """
    
    footer_text = """
    <p><strong>Nota sobre a Metodologia:</strong> Este relatório utiliza a lógica de <strong>Performance de Pagamento</strong>. 
    Cada célula é calculada como (100% - a Concentração de Vencidos daquele mês). Um valor alto (verde) indica que o problema de inadimplência do segmento está menos concentrado naquele mês específico.</p>
    """

    html_final = f"""
    <!DOCTYPE html>
    <html lang="pt-BR">
    <head>
        <meta charset="UTF-8">
        <title>Relatório de Performance de Pagamento (Aging)</title>
        {html_css}
    </head>
    <body>
        <div class="main-container">
            <div class="report-header">
                <h1>Relatório de Performance de Pagamento</h1>
                <p>Data de referência dos dados: {ref_date_obj.strftime('%d/%m/%Y')}</p>
            </div>
            {''.join(html_tabelas)}
            
            <footer>
                {footer_text}
            </footer>
        </div>
    </body>
    </html>
    """

    nome_arquivo_saida = f"{date.today().strftime('%Y-%m-%d')}-relatorio_performance_pagamento_aging.html"
    caminho_arquivo_saida = os.path.join(output_path, nome_arquivo_saida)
    
    with open(caminho_arquivo_saida, "w", encoding="utf-8") as f:
        f.write(html_final)
    print("\n" + "="*80)
    print("PROCESSO FINALIZADO COM SUCESSO!")
    print(f"Relatório de Performance (Aging) gerado em: {caminho_arquivo_saida}")
    print("="*80)

else:
    print("\nNenhuma tabela foi gerada. O relatório final está vazio.")




INICIANDO A GERAÇÃO DO RELATÓRIO DE PERFORMANCE DE PAGAMENTO
Data de referência para os dados: 05/08/2025
---> Processando segmento: Carteira Consolidada...
---> Processando segmento: Cedentes...
     [INFO] Segmento 'Cedentes' tem apenas uma entrada: 'BMP MONEY PLUS SOCIEDADE DE CREDITO DIRETO S.A'. Gerando mensagem simples.
---> Processando segmento: Originadores...
     [INFO] Segmento 'Originadores' tem apenas uma entrada: 'StarCard'. Gerando mensagem simples.
---> Processando segmento: Promotoras...
---> Processando segmento: Produtos...
---> Processando segmento: Convênios...
---> Processando segmento: Situação...
---> Processando segmento: UF...
---> Processando segmento: CAPAG...
---> Processando segmento: Pagamento Parcial...
---> Processando segmento: Tem Muitos Contratos...
---> Processando segmento: Tem Muitos Entes...
     [INFO] Segmento 'Tem Muitos Entes' tem apenas uma entrada: 'False'. Gerando mensagem simples.

PROCESSO FINALIZADO COM SUCESSO!
Relatório de Performanc

#### Ajuda na depuração (AI GEN)

In [18]:
#%%
# =============================================================================
# CÉLULA DE DEPURAÇÃO: VERIFICAÇÃO DA CONSISTÊNCIA DA PERFORMANCE PONDERADA
# =============================================================================
# Descrição: Este script verifica se a performance da "Carteira Consolidada" é
# consistente com a média ponderada das performances dos segmentos individuais
# (neste caso, "Convênios").

import pandas as pd
import numpy as np

print("\n" + "="*80)
print("INICIANDO DEPURAÇÃO DA CONSISTÊNCIA DA PERFORMANCE")
print("="*80)

# --- CONFIGURAÇÕES ---
SEGMENTO_PARA_ANALISAR = 'Convênio'
NOME_AMIGAVEL = 'Convênios'

# --- 1. Análise da Carteira Consolidada (Visão Macro) ---
print(f"\n--- 1. Análise da Carteira Consolidada ---")
# Agrupa todo o DataFrame por mês para obter o volume nominal mensal
volume_total_mensal = df_report.groupby('MesVencimento')['ValorNominal'].sum()
# Encontra o pico de faturamento da carteira inteira
pico_total_carteira = volume_total_mensal.max()
# Pega o mês mais recente para nossa análise
mes_recente = volume_total_mensal.index.max()
# Calcula o volume e a performance da carteira consolidada para o mês recente
volume_total_mes_recente = volume_total_mensal.get(mes_recente, 0)
performance_real_consolidada = (1 - (volume_total_mes_recente / pico_total_carteira)) * 100

print(f"Mês de referência para a análise: {mes_recente.strftime('%Y-%m')}")
print(f"Pico de Valor Nominal (Carteira Total): R$ {pico_total_carteira:,.2f}")
print(f"Volume Nominal no mês de referência (Carteira Total): R$ {volume_total_mes_recente:,.2f}")
print(f"PERFORMANCE REAL CONSOLIDADA: {performance_real_consolidada:.2f}%")
print("-" * 50)


# --- 2. Análise Ponderada por Segmento (Visão Micro) ---
print(f"\n--- 2. Análise Ponderada por '{NOME_AMIGAVEL}' ---")
# Cria a tabela dinâmica para o segmento
tabela_segmento = df_report.pivot_table(
    index=SEGMENTO_PARA_ANALISAR,
    columns="MesVencimento",
    values="ValorNominal",
    fill_value=0,
    aggfunc='sum'
)

# Calcula o pico de faturamento DENTRO do universo deste segmento
pico_do_segmento = tabela_segmento.max().max()
print(f"Pico de Valor Nominal (dentro de '{NOME_AMIGAVEL}'): R$ {pico_do_segmento:,.2f}")

# Calcula a performance individual de cada item do segmento para o mês de referência
volume_segmento_mes_recente = tabela_segmento.get(mes_recente, 0)
performance_individual = (1 - (volume_segmento_mes_recente / pico_do_segmento)) * 100

# Calcula o peso de cada item do segmento (baseado no seu volume total)
peso_segmento = tabela_segmento.sum(axis=1) / tabela_segmento.sum().sum()

# Calcula a performance ponderada
performance_ponderada = (performance_individual * peso_segmento).sum()
print(f"PERFORMANCE PONDERADA CALCULADA: {performance_ponderada:.2f}%")
print("-" * 50)


# --- 3. Conclusão ---
print("\n--- 3. Conclusão da Depuração ---")
print(f"Performance Real Consolidada: {performance_real_consolidada:.2f}%")
print(f"Performance Ponderada (por {NOME_AMIGAVEL}): {performance_ponderada:.2f}%")

diferenca = abs(performance_real_consolidada - performance_ponderada)
print(f"\nDiferença: {diferenca:.2f} pontos percentuais.")

print("\nPor que os valores não são idênticos?")
print("A pequena diferença ocorre porque o 'pico' de referência usado para a Carteira Consolidada (o máximo de um mês da soma de todos os entes) é ligeiramente diferente do 'pico' usado para a análise de Convênios (o máximo de um mês de um único ente).")
print("No entanto, os valores devem ser muito próximos, validando que a lógica está consistente e que a performance consolidada reflete corretamente o peso de cada segmento.")

print("\n" + "="*80)



INICIANDO DEPURAÇÃO DA CONSISTÊNCIA DA PERFORMANCE

--- 1. Análise da Carteira Consolidada ---
Mês de referência para a análise: 2035-08
Pico de Valor Nominal (Carteira Total): R$ 1,678,416.74
Volume Nominal no mês de referência (Carteira Total): R$ 4,507.57
PERFORMANCE REAL CONSOLIDADA: 99.73%
--------------------------------------------------

--- 2. Análise Ponderada por 'Convênios' ---
Pico de Valor Nominal (dentro de 'Convênios'): R$ 1,097,067.85
PERFORMANCE PONDERADA CALCULADA: 99.97%
--------------------------------------------------

--- 3. Conclusão da Depuração ---
Performance Real Consolidada: 99.73%
Performance Ponderada (por Convênios): 99.97%

Diferença: 0.24 pontos percentuais.

Por que os valores não são idênticos?
A pequena diferença ocorre porque o 'pico' de referência usado para a Carteira Consolidada (o máximo de um mês da soma de todos os entes) é ligeiramente diferente do 'pico' usado para a análise de Convênios (o máximo de um mês de um único ente).
No entanto, 